- [llama.cpp源码解读--ggml框架学习](https://zhuanlan.zhihu.com/p/19968327329)
- [llama.cpp源码解读--推理流程总览](https://zhuanlan.zhihu.com/p/25774381094)
- [llama.cpp源码解读--cgraph计算图与sched后端调度机制详解](https://zhuanlan.zhihu.com/p/1893801096918585567)
- [llama.cpp中的量化方法简介](https://zhuanlan.zhihu.com/p/12729759086)

```c++
common_params params;
--> common_params_parse;
common_init();
llama_backend_init();
llama_numa_init(params.numa);
// load the model and apply lora adapter, if any
--> common_init_from_params;
llama_model * model = llama_init.model.get();
llama_context * ctx = llama_init.context.get();
struct results_perplexity results;
--> perplexity;
llama_perf_context_print(ctx);
--> llama_backend_free --> ggml_quantize_free;
```

# [`common_params_parse`](https://github.com/ggml-org/llama.cpp/blob/master/common/arg.cpp#L1179)

```c++
common_params_parse(argc, argv, params, LLAMA_EXAMPLE_PERPLEXITY);
----------
bool common_params_parse(int argc, char ** argv, common_params & params, llama_example ex, void(*print_usage)(int, char **)) {}
```

<details>
<summary>struct common_params_context</summary>

```c++
struct common_params_context {
    enum llama_example ex = LLAMA_EXAMPLE_COMMON;
    common_params & params;
    std::vector<common_arg> options;
    void(*print_usage)(int, char **) = nullptr;
    common_params_context(common_params & params) : params(params) {}
};
```

</details>

```c++
--> common_params_parser_init --> ggml_backend_load_all --> ggml_backend_load_all_from_path --> common_params_context ctx_arg;
const common_params params_org = ctx_arg.params; // the example can modify the default params
--> common_params_parse_ex;
--> common_params_print_usage;
--> common_params_print_completion;
```

## [`ggml_backend_load_all_from_path`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend-reg.cpp#L567)

```c++
auto ctx_arg = common_params_parser_init(params, ex, print_usage);
----------
common_params_context common_params_parser_init(common_params & params, llama_example ex, void(*print_usage)(int, char **)) {
    // load dynamic backends
    --> ggml_backend_load_all();
    /**
     * filter options by example
     * rules:
     * - all examples inherit options from LLAMA_EXAMPLE_COMMON
     * - if LLAMA_EXAMPLE_* is set (other than COMMON), we only show the option in the corresponding example
     * - if both {LLAMA_EXAMPLE_COMMON, LLAMA_EXAMPLE_*,} are set, we will prioritize the LLAMA_EXAMPLE_* matching current example
     */
    auto add_opt = [&](common_arg arg) {
        if ((arg.in_example(ex) || arg.in_example(LLAMA_EXAMPLE_COMMON)) && !arg.is_exclude(ex)) {
            ctx_arg.options.push_back(std::move(arg));
        }
    };
    common_arg & common_arg::set_examples(std::initializer_list<enum llama_example> examples) {
        this->examples = std::move(examples);
        return *this;
    }
    common_arg & common_arg::set_excludes(std::initializer_list<enum llama_example> excludes) {
        this->excludes = std::move(excludes);
        return *this;
    }
    common_arg & common_arg::set_env(const char * env) {
        help = help + "\n(env: " + env + ")";
        this->env = env;
        return *this;
    }
    common_arg & common_arg::set_sparam() {
        is_sparam = true;
        return *this;
    }
}

void ggml_backend_load_all() {
    --> ggml_backend_load_all_from_path(nullptr);
}

void ggml_backend_load_all_from_path(const char * dir_path) {
    --> ggml_backend_load_best("cuda", silent, dir_path);
    --> ggml_backend_load_best("cpu", silent, dir_path);
    // check the environment variable GGML_BACKEND_PATH to load an out-of-tree backend
    const char * backend_path = std::getenv("GGML_BACKEND_PATH");
    ggml_backend_load(backend_path); //if (backend_path)
        --> return get_reg().load_backend(path, false);
}
```

[`ggml_backend_load_best`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend-reg.cpp#L493)

```c++
static ggml_backend_reg_t ggml_backend_load_best(const char * name, bool silent, const char * user_search_path) {
    // enumerate all the files that match [lib]ggml-name-*.[so|dll] in the search paths
    const fs::path name_path = fs::u8path(name); //filesystem::path "cuda"
    const fs::path file_prefix = backend_filename_prefix().native() + name_path.native() + fs::u8path("-").native(); // filesystem::path "libggml-cuda-"
    const fs::path file_extension = backend_filename_extension(); // filesystem::path ".so"
    std::vector<fs::path> search_paths;
    // default search paths: executable directory, current directory
    search_paths.push_back(get_executable_path());
    //[0] = filesystem::path "/mnt/nfsdir_client/zhouyingkun/llama.cpp/cuda_debug/bin/"
    search_paths.push_back(fs::current_path());
    fs::path filename = backend_filename_prefix().native() + name_path.native() + backend_filename_extension().native();
    //filesystem::path "libggml-cuda.so"
    fs::path path = search_path / filename;
    --> return get_reg().load_backend(path, silent);
}
```

[`ggml_backend_registry::ggml_backend_registry`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend-reg.cpp#L167)

```c++
get_reg();
----------
static ggml_backend_registry & get_reg() {
    --> static ggml_backend_registry reg;
    return reg;
}

ggml_backend_registry::ggml_backend_registry() {
#ifdef GGML_USE_CUDA
        register_backend(ggml_backend_cuda_reg());
#endif
#ifdef GGML_USE_CPU
        register_backend(ggml_backend_cpu_reg());
#endif
}
```

### [`ggml_backend_cuda_reg`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L3504)

<details>
<summary>struct ggml_backend_reg/ggml_backend_reg_t</summary>

```c++
struct ggml_backend_reg {
    int api_version; // initialize to GGML_BACKEND_API_VERSION
    struct ggml_backend_reg_i iface;
    void * context;
};

typedef struct ggml_backend_reg * ggml_backend_reg_t;
```

</details>

<details>
<summary>struct ggml_backend_device/ggml_backend_dev_t</summary>

```c++
struct ggml_backend_device {
    struct ggml_backend_device_i iface;
    ggml_backend_reg_t reg;
    void * context;
};

typedef struct ggml_backend_device * ggml_backend_dev_t;
```

</details>


<details>
<summary>struct ggml_backend_cuda_reg_context</summary>

```c++
struct ggml_backend_cuda_reg_context {
    std::vector<ggml_backend_dev_t> devices;
};
```

</details>


<details>
<summary>struct ggml_backend_cuda_device_context</summary>

```c++
struct ggml_backend_cuda_device_context {
    int device;
    std::string name;
    std::string description;
};
```

</details>


```c++
static ggml_backend_reg reg;
static bool initialized = false;
{
    static std::mutex mutex;
    std::lock_guard<std::mutex> lock(mutex);
    if (!initialized) {
        ggml_backend_cuda_reg_context * ctx = new ggml_backend_cuda_reg_context;

        for (int i = 0; i < ggml_cuda_info().device_count; i++) {
                ---> return static ggml_cuda_device_info info = ggml_cuda_init();
            ggml_backend_cuda_device_context * dev_ctx = new ggml_backend_cuda_device_context;
            dev_ctx->device = i;
            // CUDA0
            dev_ctx->name = GGML_CUDA_NAME + std::to_string(i); //#define GGML_CUDA_NAME "CUDA"

            ggml_cuda_set_device(i);
            cudaDeviceProp prop;
            CUDA_CHECK(cudaGetDeviceProperties(&prop, i));
            dev_ctx->description = prop.name; //

            ggml_backend_dev_t dev = new ggml_backend_device {
                /* .iface   = */ ggml_backend_cuda_device_interface,
                /* .reg     = */ &reg,
                /* .context = */ dev_ctx
            };
            ctx->devices.push_back(dev);
        }

        reg = ggml_backend_reg {
            /* .api_version = */ GGML_BACKEND_API_VERSION,
            /* .iface       = */ ggml_backend_cuda_reg_interface,
            /* .context     = */ ctx
        };
    }

    initialized = true;
}
return &reg;
```

### [`ggml_backend_cpu_reg`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.cpp#L658)

```c++
// init CPU feature detection
--> ggml_cpu_init();

static struct ggml_backend_reg ggml_backend_cpu_reg = {
    /* .api_version = */ GGML_BACKEND_API_VERSION,
    /* .iface       = */ ggml_backend_cpu_reg_i,
    /* .context     = */ NULL,
};

return &ggml_backend_cpu_reg;
```

#### [`ggml_cpu_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L3240)

<details>
<summary>FP32_TO_FP16/FP16_TO_FP32</summary>

```c++
// On ARM NEON, it's quicker to directly convert x -> x instead of calling into ggml_lookup_fp16_to_fp32,
// so we define GGML_FP16_TO_FP32 and GGML_FP32_TO_FP16 elsewhere for NEON.
// This is also true for POWER9.
#if !defined(GGML_FP16_TO_FP32)
inline static float ggml_lookup_fp16_to_fp32(ggml_fp16_t f) {
    uint16_t s;
    memcpy(&s, &f, sizeof(uint16_t));
    return ggml_table_f32_f16[s];
}

#define GGML_FP16_TO_FP32(x) ggml_lookup_fp16_to_fp32(x)
#endif

#if !defined(GGML_FP32_TO_FP16)
#define GGML_FP32_TO_FP16(x) GGML_COMPUTE_FP32_TO_FP16(x)
#endif
```

</details>


```c++
// needed to initialize f16 tables
{
    struct ggml_init_params params = { 0, NULL, false };
    struct ggml_context * ctx = ggml_init(params);
    ---> {
        static bool is_first_call = true;
        ggml_critical_section_start();
        if (is_first_call) {
            // initialize time system (required on Windows)
            ggml_time_init();

            for (int i = 0; i < (1 << 16); ++i) {
                union {
                    uint16_t u16;
                    ggml_fp16_t fp16;
                } u = {i};
                ggml_table_f32_f16[i] = GGML_COMPUTE_FP16_TO_FP32(u.fp16);
            }

            is_first_call = false;
        }
        ggml_critical_section_end();
    }
    ggml_free(ctx);
}
ggml_critical_section_start();
static bool is_first_call = true;
if (is_first_call) {
    // initialize GELU, Quick GELU, SILU and EXP F32 tables
    {
        const uint64_t t_start = ggml_time_us(); UNUSED(t_start);
        for (int i = 0; i < (1 << 16); ++i) {
            union {
                uint16_t u16;
                ggml_fp16_t fp16;
            } u = {i};
            float f = GGML_FP16_TO_FP32(u.fp16);
            ggml_table_gelu_f16[i] = GGML_FP32_TO_FP16(ggml_gelu_f32(f));
            ggml_table_gelu_quick_f16[i] = GGML_FP32_TO_FP16(ggml_gelu_quick_f32(f));
        }
        const uint64_t t_end = ggml_time_us(); UNUSED(t_end);
        GGML_PRINT_DEBUG("%s: GELU, Quick GELU, SILU and EXP tables initialized in %f ms\n", __func__, (t_end - t_start)/1000.0);
#ifdef GGML_USE_OPENMP
        //if (!getenv("OMP_WAIT_POLICY")) {
        //    // set the wait policy to active, so that OpenMP threads don't sleep
        //    putenv("OMP_WAIT_POLICY=active");
        //}
        // 有意思
        if (!getenv("KMP_BLOCKTIME")) {
            // set the time to wait before sleeping a thread
            // this is less aggressive than setting the wait policy to active, but should achieve similar results in most cases
            putenv("KMP_BLOCKTIME=200"); // 200ms
        }
#endif
    }

#if defined(__ARM_ARCH)
    ggml_init_arm_arch_features();
#endif

    is_first_call = false;
}
ggml_critical_section_end();
```

[`ggml_backend_cpu_reg_get_device`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.cpp#L488)

```c++
static ggml_backend_dev_t ggml_backend_cpu_reg_get_device(ggml_backend_reg_t reg, size_t index) {
    GGML_ASSERT(index == 0);

    static ggml_backend_cpu_device_context ctx;
    static ggml_backend_device ggml_backend_cpu_device = {
        /* .iface   = */ ggml_backend_cpu_device_i,
        /* .reg     = */ reg,
        /* .context = */ &ctx,
    };

    return &ggml_backend_cpu_device;
}
```

[`ggml_backend_registry::register_backend`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend-reg.cpp#L210)

```c++
 void register_backend(ggml_backend_reg_t reg, dl_handle_ptr handle = nullptr) {
    if (!reg) {
        return;
    }

    backends.push_back({ reg, std::move(handle) });
    for (size_t i = 0; i < ggml_backend_reg_dev_count(reg); i++) {
        --> register_device(ggml_backend_reg_dev_get(reg, i));
    }
}
```

[`ggml_backend_registry::register_device`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend-reg.cpp#L210)

```c++
 void register_device(ggml_backend_dev_t device) {
    devices.push_back(device);
}
```

## [`common_params_parse_ex`](https://github.com/ggml-org/llama.cpp/blob/master/common/arg.cpp#L852)

这个函数是为了处理具体的example用的

```c++
common_params_parse_ex(argc, argv, ctx_arg);
----------
static bool common_params_parse_ex(int argc, char ** argv, common_params_context & ctx_arg) {
    // handle environment variables
    // handle command line arguments
    // handle model and download
    --> common_chat_verify_template;
}
```

# [`common_init_from_params`](https://github.com/ggml-org/llama.cpp/blob/master/common/common.cpp#L902)

```c++
common_init_result llama_init = common_init_from_params(params);
----------
struct common_init_result common_init_from_params(common_params & params) {}
```

<details>
<summary>struct common_init_result</summary>

```c++
// note: defines object's lifetime
struct common_init_result {
    llama_model_ptr   model;
    llama_context_ptr context;

    std::vector<llama_adapter_lora_ptr> lora;
};
```

</details>

```c++
common_init_result iparams;

auto mparams = common_model_params_to_llama(params);
--> llama_model_load_from_file --> llama_model_load_from_file_impl --> llama_model_load --> llama_model * model;
auto cparams = common_context_params_to_llama(params);
--> llama_init_from_model --> llama_context::llama_context --> llama_context * lctx;
llama_set_warmup(lctx, warmup=true); --> cparams.warmup = warmup;
std::vector<llama_token> tmp;
llama_token bos = llama_vocab_bos(vocab);
llama_token eos = llama_vocab_eos(vocab);
tmp.push_back(bos);
tmp.push_back(eos);
--> llama_decode --> llama_context::decode;
llama_memory_clear(llama_get_memory(lctx), true);
    --> lctx->memory->clear(data);
        --> cells.reset();
        --> ggml_backend_buffer_clear(buf.get(), 0);
// add the evaluation to the stats
llama_synchronize(lctx); --> lctx->synchronize; --> llama_context::synchronize; --> ggml_backend_sched_synchronize;
llama_perf_context_reset(lctx);
llama_set_warmup(lctx, false);

iparams.model.reset(model);
iparams.context.reset(lctx);
return iparams;
```

## [`llama_model_load`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama.cpp#L87)

```c++
llama_model * model = llama_model_load_from_file(params.model.path.c_str(), mparams);
----------
struct llama_model * llama_model_load_from_file(
    const char * path_model,
    struct llama_model_params params) {
    std::vector<std::string> splits = {};
    --> return llama_model_load_from_file_impl(path_model, splits, params);
}

    enum ggml_backend_dev_type {
        // CPU device using system memory
        GGML_BACKEND_DEVICE_TYPE_CPU,
        // GPU device using dedicated memory ？？？好奇统一内存的GPU用哪个
        GGML_BACKEND_DEVICE_TYPE_GPU,
        // accelerator devices intended to be used together with the CPU backend (e.g. BLAS or AMX)
        GGML_BACKEND_DEVICE_TYPE_ACCEL
    };

static struct llama_model * llama_model_load_from_file_impl(
        const std::string & path_model,
        std::vector<std::string> & splits,
        struct llama_model_params params) {
    llama_model * model = new llama_model(params);
    ggml_backend_dev_t dev = ggml_backend_dev_get(i);
    switch (ggml_backend_dev_type(dev)) {
        case GGML_BACKEND_DEVICE_TYPE_GPU:
            model->devices.push_back(dev); // 这一步很隐蔽啊！
            size_t free, total; // NOLINT
            ggml_backend_dev_memory(dev, &free, &total);
    }
    --> const int status = llama_model_load(path_model, splits, *model, params);
    return model;
}

// Returns 0 on success, -1 on error, and -2 on cancellation via llama_progress_callback
static int llama_model_load(const std::string & fname, std::vector<std::string> & splits, llama_model & model, llama_model_params & params) {}
```

<details>
<summary>struct llama_model</summary>

```c++
struct llama_model {
    llm_type type = LLM_TYPE_UNKNOWN;
    llm_arch arch = LLM_ARCH_UNKNOWN;

    std::string name = "n/a";

    llama_hparams hparams = {};
    llama_vocab   vocab;

    struct ggml_tensor * tok_embd   = nullptr;
    struct ggml_tensor * type_embd  = nullptr;
    struct ggml_tensor * pos_embd   = nullptr;
    struct ggml_tensor * tok_norm   = nullptr;
    struct ggml_tensor * tok_norm_b = nullptr;

    struct ggml_tensor * output_norm     = nullptr;
    struct ggml_tensor * output_norm_b   = nullptr;
    struct ggml_tensor * output          = nullptr;
    struct ggml_tensor * output_b        = nullptr;
    struct ggml_tensor * output_norm_enc = nullptr;

    std::vector<llama_layer> layers;

    llama_model_params params;

    // gguf metadata
    std::unordered_map<std::string, std::string> gguf_kv;

    // list of devices used in this model
    std::vector<ggml_backend_dev_t> devices;

    // for quantize-stats only
    std::vector<std::pair<std::string, struct ggml_tensor *>> tensors_by_name;

    // note: can mutate `cparams`
    // TODO: move this to new llm_arch_model_i interface
    llama_memory_i * create_memory(const llama_memory_params & params, llama_cparams & cparams) const;

    // TODO: move this to new llm_arch_model_i interface
    llm_graph_result_ptr build_graph(
            const llm_graph_params & params,
                       ggml_cgraph * gf,
                    llm_graph_type   type) const;

private:
    struct impl;
    std::unique_ptr<impl> pimpl;
};

```
    
</details>

```c++
--> llama_model_loader::llama_model_loader --> llama_model_loader ml;
ml.print_info();
model.load_arch(ml);
--> llama_model::load_hparams;
--> llama_model::load_vocab --> llama_vocab::load --> llama_vocab::impl::load;
model.load_stats(ml);
model.print_info();
--> llama_model::load_tensors;
return 0;
```

### [`llama_model_loader::llama_model_loader`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L468)

> 这个函数的功效在于读取gguf文件里面的各种信息，并且构建`tensor_meta`, 类型同样是`ggml_tensor *`，但是不被用于运算，用来在后面的`llama_model::load_tensors`函数中，构建起llm_model的类中的属性，比如`tok_embd`, `layers`, `output_norm` 和 `output`

> 后续的用法是这样的：`ggml_tensor * t_meta = ml.get_tensor_meta(tn.str().c_str());` 然后权重的tensor类似于是长这样的

```log
>>> p *cur
$189 = {
  type = GGML_TYPE_Q2_K,
  buffer = 0x0, # 注意到此时buffer地址还是0的未初始化值
  ne = {[0] = 4096, [1] = 128256, [2] = 1, [3] = 1},
  nb = {[0] = 84, [1] = 1344, [2] = 172376064, [3] = 172376064},
  op = GGML_OP_NONE,
  op_params = {[0] = 0 <repeats 16 times>},
  flags = 0,
  src = {[0] = 0x0, [1] = 0x0, [2] = 0x0, [3] = 0x0, [4] = 0x0, [5] = 0x0, [6] = 0x0, [7] = 0x0, [8] = 0x0, [9] = 0x0},
  view_src = 0x0,
  view_offs = 0,
  data = 0x0,
  name = '\000' <repeats 63 times>,
  extra = 0x0,
  padding = "\000\000\000\000\000\000\000"
}
```

```c++
llama_model_loader ml(fname, splits, params.use_mmap, params.check_tensors, params.kv_overrides, params.tensor_buft_overrides);
----------
llama_model_loader::llama_model_loader(
        const std::string & fname,
        std::vector<std::string> & splits,
        bool use_mmap,
        bool check_tensors,
        const llama_model_kv_override * param_overrides_p,
        const llama_model_tensor_buft_override * param_tensor_buft_overrides_p) {}
```

```c++
// Load the main GGUF
struct ggml_context * ctx = NULL;
struct gguf_init_params params = {
    /*.no_alloc = */ true,
    /*.ctx      = */ &ctx,
};

llama_model_loader:: gguf_context_ptr meta;
--> gguf_init_from_file --> gguf_init_from_file_impl --> meta;

files.emplace_back(new llama_file(fname.c_str(), "rb"));
contexts.emplace_back(ctx);

for (ggml_tensor * cur = ggml_get_first_tensor(ctx); cur; cur = ggml_get_next_tensor(ctx, cur)) {
    std::string tensor_name = std::string(cur->name);
    n_elements += ggml_nelements(cur);
    n_bytes    += ggml_nbytes(cur);
    ---> weights_map.emplace(tensor_name, llama_tensor_weight(files.back().get(), 0, meta.get(), cur));
}

n_kv      = gguf_get_n_kv(meta.get());
n_tensors = weights_map.size();

fver = (enum llama_fver) gguf_get_version(meta.get());

this->use_mmap = use_mmap;
this->check_tensors = check_tensors;
```

#### [`gguf_init_from_file_impl`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/gguf.cpp#L319)

```c++
meta.reset(gguf_init_from_file(fname.c_str(), params));

struct gguf_context * gguf_init_from_file(const char * fname, struct gguf_init_params params) {
    FILE * file = ggml_fopen(fname, "rb");
    --> struct gguf_context * result = gguf_init_from_file_impl(file, params);
    fclose(file);
    return result;
}

struct gguf_context * gguf_init_from_file_impl(FILE * file, struct gguf_init_params params) {}
```

<details>
<summary>struct gguf_context</summary>

```c++
struct gguf_context {
    uint32_t version = GGUF_VERSION;

    std::vector<struct gguf_kv> kv;
    std::vector<struct gguf_tensor_info> info;

    size_t alignment = GGUF_DEFAULT_ALIGNMENT;
    size_t offset    = 0; // offset of `data` from beginning of file
    size_t size      = 0; // size of `data` in bytes

    void * data = nullptr;
};
```
    
</details>

```c++
const struct gguf_reader gr(file);
struct gguf_context * ctx = new gguf_context;
// file magic
gr.read(magic, 4);
// header
gr.read(ctx->version);
gr.read(n_tensors);
gr.read(n_kv);
// KV pairs
for (int64_t i = 0; ok && i < n_kv; ++i)
    gr.read(key);
    gr.read(type);
    is_array = true; gr.read(type); gr.read(n);
    gguf_read_emplace_helper<xxx>    (gr, ctx->kv, key, is_array, n);
const int alignment_idx = gguf_find_key(ctx, GGUF_KEY_GENERAL_ALIGNMENT);
ctx->alignment = alignment_idx == -1 ? GGUF_DEFAULT_ALIGNMENT : gguf_get_val_u32(ctx, alignment_idx);

// read the tensor info
for (int64_t i = 0; ok && i < n_tensors; ++i)
    struct gguf_tensor_info info;
    std::string name; gr.read(name); ggml_set_name(&info.t, name.c_str()); // tensor name
    uint32_t n_dims = -1; gr.read(n_dims); gr.read(info.t.ne); // tensor shape
    gr.read(info.t.type); // tensor type
    // calculate byte offsets given the tensor shape and type
    const size_t  type_size = ggml_type_size(info.t.type);
    const int64_t blck_size = ggml_blck_size(info.t.type);
    info.t.nb;
    gr.read(info.offset); // tensor data offset within buffer
    ctx->info.push_back(info);

// store the current file offset - this is where the data section starts
ctx->offset = ftell(file);

// compute the total size of the data section, taking into account the alignment
ctx->size = 0;
for (size_t i = 0; i < ctx->info.size(); ++i)
    const gguf_tensor_info & ti = ctx->info[i];
    ctx->size += GGML_PAD(ggml_nbytes(&ti.t), ctx->alignment);


// load the tensor data only if requested
// compute the exact size needed for the new ggml_context
const size_t mem_size = n_tensors * ggml_tensor_overhead();

struct ggml_init_params pdata = {
    /*mem_size   =*/ mem_size,
    /*mem_buffer =*/ nullptr,
    /*no_alloc   =*/ params.no_alloc,
};
--> ggml_init --> *params.ctx;
struct ggml_context * ctx_data = *params.ctx;

// create the tensors
for (size_t i = 0; i < ctx->info.size(); ++i) {
    const struct gguf_tensor_info & info = ctx->info[i];
    --> ggml_new_tensor --> ggml_new_tensor_impl --> struct ggml_tensor * cur;
    ggml_set_name(cur, info.t.name);
}
return ctx;
```

##### [`ggml_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1414)

```c++
*params.ctx = ggml_init(pdata);
----------
struct ggml_context * ggml_init(struct ggml_init_params params) {}
```

<details>
<summary>struct ggml_context</summary>

```c++
struct ggml_context {
    size_t mem_size;
    void * mem_buffer;
    bool   mem_buffer_owned;
    bool   no_alloc;

    int    n_objects;

    struct ggml_object * objects_begin;
    struct ggml_object * objects_end;
};

size_t ggml_tensor_overhead(void) {
    return GGML_OBJECT_SIZE + GGML_TENSOR_SIZE;
}
```

</details>


```c++
struct ggml_context * ctx = GGML_MALLOC(sizeof(struct ggml_context));

// allow to call ggml_init with 0 size
if (params.mem_size == 0) {
    params.mem_size = GGML_MEM_ALIGN;
}

const size_t mem_size = params.mem_buffer ? params.mem_size : GGML_PAD(params.mem_size, GGML_MEM_ALIGN);

*ctx = (struct ggml_context) {
    /*.mem_size           =*/ mem_size,
    /*.mem_buffer         =*/ params.mem_buffer ? params.mem_buffer : ggml_aligned_malloc(mem_size),
    /*.mem_buffer_owned   =*/ params.mem_buffer ? false : true,
    /*.no_alloc           =*/ params.no_alloc,
    /*.n_objects          =*/ 0,
    /*.objects_begin      =*/ NULL,
    /*.objects_end        =*/ NULL,
};

return ctx;
```

##### [`ggml_new_tensor_impl`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1565)

```c++
struct ggml_tensor * cur = ggml_new_tensor(ctx_data, info.t.type, GGML_MAX_DIMS, info.t.ne);

struct ggml_tensor * ggml_new_tensor(
        struct ggml_context * ctx,
        enum   ggml_type      type,
        int                   n_dims,
        const int64_t       * ne) {
    --> return ggml_new_tensor_impl(ctx, type, n_dims, ne, NULL, 0);
}

static struct ggml_tensor * ggml_new_tensor_impl(
        struct ggml_context * ctx,
        enum   ggml_type      type,
        int                   n_dims,
        const int64_t       * ne,
        struct ggml_tensor  * view_src,
        size_t                view_offs) {}
```

<details>
<summary>struct ggml_tensor</summary>

```c++
// n-dimensional tensor
struct ggml_tensor {
    enum ggml_type type;

    struct ggml_backend_buffer * buffer;

    int64_t ne[GGML_MAX_DIMS]; // number of elements
    size_t  nb[GGML_MAX_DIMS]; // stride in bytes:
                               // nb[0] = ggml_type_size(type)
                               // nb[1] = nb[0]   * (ne[0] / ggml_blck_size(type)) + padding
                               // nb[i] = nb[i-1] * ne[i-1]

    // compute data
    enum ggml_op op;

    // op params - allocated as int32_t for alignment
    int32_t op_params[GGML_MAX_OP_PARAMS / sizeof(int32_t)];

    int32_t flags;

    struct ggml_tensor * src[GGML_MAX_SRC];

    // source tensor and offset for views
    struct ggml_tensor * view_src;
    size_t               view_offs;

    void * data;

    char name[GGML_MAX_NAME];

    void * extra; // extra things e.g. for ggml-cuda.cu

    char padding[8];
};

static const size_t GGML_TENSOR_SIZE = sizeof(struct ggml_tensor);
```
</details>


```c++
--> ggml_new_object --> obj_new;
struct ggml_tensor * const result = (struct ggml_tensor *)((char *)ctx->mem_buffer + obj_new->offs);
*result = (struct ggml_tensor) {
    /*.type         =*/ type,
    /*.buffer       =*/ NULL,
    /*.ne           =*/ { 1, 1, 1, 1 },
    /*.nb           =*/ { 0, 0, 0, 0 },
    /*.op           =*/ GGML_OP_NONE,
    /*.op_params    =*/ { 0 },
    /*.flags        =*/ 0,
    /*.src          =*/ { NULL },
    /*.view_src     =*/ view_src,
    /*.view_offs    =*/ view_offs,
    /*.data         =*/ view_src != NULL ? view_src->data + view_offs : NULL,
    /*.name         =*/ { 0 },
    /*.extra        =*/ NULL,
    /*.padding      =*/ { 0 },
};

result->ne;
result->nb;
ctx->n_objects++;
return result;
```

[`ggml_new_object`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1519)

```c++
struct ggml_object * const obj_new = ggml_new_object(ctx, GGML_OBJECT_TYPE_TENSOR, GGML_TENSOR_SIZE);

static struct ggml_object * ggml_new_object(struct ggml_context * ctx, enum ggml_object_type type, size_t size) {}
```

<details>
<summary>struct ggml_object</summary>

```c++
struct ggml_object {
    size_t offs;
    size_t size;

    struct ggml_object * next;

    enum ggml_object_type type;

    char padding[4];
};

static const size_t GGML_OBJECT_SIZE = sizeof(struct ggml_object);
```
</details>


```c++
// always insert objects at the end of the context's memory pool
struct ggml_object * obj_cur = ctx->objects_end;

const size_t cur_offs = obj_cur == NULL ? 0 : obj_cur->offs;
const size_t cur_size = obj_cur == NULL ? 0 : obj_cur->size;
const size_t cur_end  = cur_offs + cur_size;

// align to GGML_MEM_ALIGN
size_t size_needed = GGML_PAD(size, GGML_MEM_ALIGN);

char * const mem_buffer = ctx->mem_buffer;
struct ggml_object * const obj_new = (struct ggml_object *)(mem_buffer + cur_end);

*obj_new = (struct ggml_object) {
    .offs = cur_end + GGML_OBJECT_SIZE,
    .size = size_needed,
    .next = NULL,
    .type = type,
};

GGML_ASSERT_ALIGNED(mem_buffer + obj_new->offs);

if (obj_cur != NULL) {
    obj_cur->next = obj_new;
} else {
    // this is the first object in this context
    ctx->objects_begin = obj_new;
}

ctx->objects_end = obj_new;
return obj_new;
```

### [`llama_model::load_hparams`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L425)

```c++
model.load_hparams(ml);
----------
void llama_model::load_hparams(llama_model_loader & ml) {}
```

```c++
const gguf_context * ctx = ml.meta.get();
// get metadata as string
// gguf metadata
llama_model:: std::unordered_map<std::string, std::string> gguf_kv;
gguf_kv.emplace(name, value);
hparams.xxx = xxx; // via ml.get_key
pimpl->n_bytes = ml.n_bytes;
pimpl->desc_str = arch_name() + " " + type_name() + " " + ml.ftype_name();
```

### [`llama_vocab::impl::load`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-vocab.cpp#L1372)

```c++
model.load_vocab(ml);
----------
void llama_model::load_vocab(llama_model_loader & ml) {
    const auto kv = LLM_KV(arch);
    --> vocab.load(ml, kv);
}

void llama_vocab::load(llama_model_loader & ml, const LLM_KV & kv) {
    --> pimpl->load(ml, kv);
}

void llama_vocab::impl::load(llama_model_loader & ml, const LLM_KV & kv) {}
```

```c++
struct gguf_context * ctx = ml.meta.get();
// determine vocab type
ml.get_key(LLM_KV_TOKENIZER_MODEL, tokenizer_model);
ml.get_key(LLM_KV_TOKENIZER_PRE,   tokenizer_pre, false);
ml.get_key(LLM_KV_TOKENIZER_TOKEN_TYPE_COUNT, n_token_types, false);
// for now, only BPE models have pre-tokenizers
llama_vocab::impl::
    std::unordered_map<std::string, llama_token> token_to_id;
    std::vector<token_data> id_to_token;
--> init_tokenizer(type); --> tokenizer = std::make_unique<llm_tokenizer_bpe>(vocab);
// determine the newline token: LLaMA "<0x0A>" == 10 == '\n', Falcon 193 == '\n
// special tokens
    std::set<llama_token> special_eog_ids; // set of all tokens that cause "end of generation"
// build special tokens cache
    std::vector<llama_token> cache_special_tokens;
// build token to piece cache
    std::vector<std::string> cache_token_to_piece; // llama_token_to_piece(special = true);
```

### [`llama_model::load_tensors`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L1507)

> 这个函数主要是进行后端的分配，然后把 `llama_model_loader` ctx 里面的tensor copy到 `llama_model` 类中；
- 如果是CUDA后端，在 `alloc_tensor_range` 的时候给tensor分配buffer和data，然后在 `llama_model_loader::load_all_data` 上将权重传到GPU上；
- 如果是CPU后端，在 `llama_model_loader::load_all_data` 的时候给tensor分配buffer和data；

```c++
model.load_tensors(ml);
----------
bool llama_model::load_tensors(llama_model_loader & ml) {}
```

```c++
const auto & split_mode   = params.split_mode;
const auto & n_gpu_layers = params.n_gpu_layers;
const auto & use_mlock    = params.use_mlock;
const auto & tensor_split = params.tensor_split;
// build a list of buffer types for the CPU and GPU devices
--> make_cpu_buft_list --> pimpl->cpu_buft_list;
for (auto * dev : devices)
    make_gpu_buft_list --> ggml_backend_cuda_buffer_type --> buft_list_t buft_list;
    // add CPU buffer types as a fallback
    buft_list.insert(buft_list.end(), pimpl->cpu_buft_list.begin(), pimpl->cpu_buft_list.end());
    pimpl->gpu_buft_list.emplace(dev, std::move(buft_list));

// calculate the split points
std::vector<float> splits(n_devices());
// sum and normalize the splits to get the split points
ggml_backend_dev_t cpu_dev = ggml_backend_dev_by_type(GGML_BACKEND_DEVICE_TYPE_CPU);
const int i_gpu_start = std::max((int) hparams.n_layer - n_gpu_layers, (int) 0);
const int act_gpu_layers = devices.empty() ? 0 : std::min(n_gpu_layers, (int)n_layer + 1);
auto get_layer_buft_list = [&](int il) -> llama_model::impl::layer_dev {
    if (il < i_gpu_start || (il - i_gpu_start) >= act_gpu_layers) return {cpu_dev, &pimpl->cpu_buft_list};
        const int layer_gpu = std::upper_bound(splits.begin(), splits.begin() + n_devices(), float(il - i_gpu_start)/act_gpu_layers) - splits.begin(); // 0
        auto * dev = devices.at(layer_gpu);
        return {dev, &pimpl->gpu_buft_list.at(dev)};
}
// assign the input layer, there is very little benefit to offloading the input layer, so always keep it on the CPU
pimpl->dev_input = { cpu_dev, &pimpl->cpu_buft_list };
// assign the repeating layers to the devices according to the splits
pimpl->dev_layer.resize(n_layer);
for (int il = 0; il < n_layer; ++il) pimpl->dev_layer[il] = get_layer_buft_list(il);
// assign the output layer
pimpl->dev_output = get_layer_buft_list(n_layer);
// one ggml context per buffer type
int max_n_tensors = ml.n_tensors;
max_n_tensors += 1;         // duplicated output tensor
max_n_tensors += n_layer*2; // duplicated rope freq tensors
const size_t ctx_size = ggml_tensor_overhead()*max_n_tensors;
// one ggml context per buffer type
std::map<ggml_backend_buffer_type_t, ggml_context *> ctx_map;
auto ctx_for_buft = [&](ggml_backend_buffer_type_t buft) -> ggml_context * {
    auto it = ctx_map.find(buft);
    if (it == ctx_map.end()) {
        ggml_init_params params = {
            /*.mem_size   =*/ ctx_size,
            /*.mem_buffer =*/ NULL,
            /*.no_alloc   =*/ true,
        };
        ggml_context * ctx = ggml_init(params);
        ctx_map[buft] = ctx;
        pimpl->ctxs.emplace_back(ctx);
        return ctx;
    }
    return it->second;
};
// create tensors for the weights
auto create_tensor = [&](const LLM_TN_IMPL & tn, const std::initializer_list<int64_t> & ne, int flags) -> ggml_tensor * {
    ggml_tensor * t_meta = ml.get_tensor_meta(tn.str().c_str());
    llm_tensor_info info = llm_tensor_info_for(tn_tensor);
    // select the buffer type for this tensor
    switch (info.layer) {}
    --> select_weight_buft --> weight_buft_supported --> buft;
    // avoid using a host buffer when using mmap 啊这，如果使用mmap就没法利用host buffer了！！！
    auto * buft_dev = ggml_backend_buft_get_device(buft);
    if (ml.use_mmap && buft_dev && buft == ggml_backend_dev_host_buffer_type(buft_dev))
        // CPU 不符合 buft_dev 跳过了，GPU host buffer 会进入，GPU 因为host buffer == buft 所以也会跳过
        auto * cpu_dev = ggml_backend_dev_by_type(GGML_BACKEND_DEVICE_TYPE_CPU);
        buft = ggml_backend_dev_buffer_type(cpu_dev);
    ggml_context * ctx = ctx_for_buft(buft);
    --> return llama_model_loader::create_tensor;
}
```

```c++
layers.resize(n_layer);
// TODO: move to a separate function
const auto tn = LLM_TN(arch);

tok_embd = create_tensor(tn(LLM_TENSOR_TOKEN_EMBD, "weight"), {n_embd, n_vocab}, 0);

// output
output_norm = create_tensor(tn(LLM_TENSOR_OUTPUT_NORM, "weight"), {n_embd}, 0);
output      = create_tensor(tn(LLM_TENSOR_OUTPUT,      "weight"), {n_embd, n_vocab}, TENSOR_NOT_REQUIRED);

// if output is NULL, init from the input tok embed
if (output == NULL)
    output = create_tensor(tn(LLM_TENSOR_TOKEN_EMBD, "weight"), {n_embd, n_vocab}, TENSOR_DUPLICATED);

for (int i = 0; i < n_layer; ++i)
    auto & layer = layers[i];

    layer.attn_norm = create_tensor(tn(LLM_TENSOR_ATTN_NORM, "weight", i), {n_embd}, 0);

    layer.wq = create_tensor(tn(LLM_TENSOR_ATTN_Q,   "weight", i), {n_embd, n_embd_head_k * n_head}, 0);
    layer.wk = create_tensor(tn(LLM_TENSOR_ATTN_K,   "weight", i), {n_embd, n_embd_k_gqa}, 0);
    layer.wv = create_tensor(tn(LLM_TENSOR_ATTN_V,   "weight", i), {n_embd, n_embd_v_gqa}, 0);
    layer.wo = create_tensor(tn(LLM_TENSOR_ATTN_OUT, "weight", i), {n_embd_head_k * n_head, n_embd}, 0);

    // optional bias tensors
    layer.bq = create_tensor(tn(LLM_TENSOR_ATTN_Q,   "bias", i), {n_embd},     TENSOR_NOT_REQUIRED);
    layer.bk = create_tensor(tn(LLM_TENSOR_ATTN_K,   "bias", i), {n_embd_gqa}, TENSOR_NOT_REQUIRED);
    layer.bv = create_tensor(tn(LLM_TENSOR_ATTN_V,   "bias", i), {n_embd_gqa}, TENSOR_NOT_REQUIRED);
    layer.bo = create_tensor(tn(LLM_TENSOR_ATTN_OUT, "bias", i), {n_embd},     TENSOR_NOT_REQUIRED);

    layer.ffn_norm = create_tensor(tn(LLM_TENSOR_FFN_NORM, "weight", i), {n_embd}, 0);

    layer.ffn_gate = create_tensor(tn(LLM_TENSOR_FFN_GATE, "weight", i), {n_embd,   n_ff}, 0);
    layer.ffn_down = create_tensor(tn(LLM_TENSOR_FFN_DOWN, "weight", i), {  n_ff, n_embd}, 0);
    layer.ffn_up   = create_tensor(tn(LLM_TENSOR_FFN_UP,   "weight", i), {n_embd,   n_ff}, 0);

    // optional MLP bias
    layer.ffn_gate_b = create_tensor(tn(LLM_TENSOR_FFN_GATE, "bias", i), {n_ff}, TENSOR_NOT_REQUIRED);
    layer.ffn_down_b = create_tensor(tn(LLM_TENSOR_FFN_DOWN, "bias", i), {n_embd}, TENSOR_NOT_REQUIRED);
    layer.ffn_up_b   = create_tensor(tn(LLM_TENSOR_FFN_UP,   "bias", i), {n_ff}, TENSOR_NOT_REQUIRED);
```

```c++
ml.done_getting_tensors();
--> llama_model_loader::init_mappings --> ml.mappings;
pimpl->mappings.reserve(ml.mappings.size());
// create the backend buffers
using llama_buf_map = std::unordered_map<uint32_t=0, ggml_backend_buffer_t>;
std::vector<std::pair<ggml_context *, llama_buf_map>> ctx_bufs;
ctx_bufs.reserve(ctx_map.size());
for (auto & it : ctx_map)  ctx_bufs.emplace_back(it.second, buf_map);
// Ensure we have enough capacity for the maximum backend buffer we will potentially create
pimpl->bufs.reserve(ctx_map.size()); // * ml.files.size();

for (auto & it : ctx_map)
    ggml_context * ctx = it.second;
    llama_buf_map buf_map;
    buf_map.reserve(ctx_map.size()); // * ml.files.size()
    ggml_backend_dev_props props;
    --> ggml_backend_dev_get_props(dev, &props);
        static void ggml_backend_cpu_device_get_props(ggml_backend_dev_t dev, struct ggml_backend_dev_props * props) {
            props->name        = ggml_backend_cpu_device_get_name(dev);
            props->description = ggml_backend_cpu_device_get_description(dev);
            props->type        = ggml_backend_cpu_device_get_type(dev);
            ggml_backend_cpu_device_get_memory(dev, &props->memory_free, &props->memory_total);
            props->caps = {
                /* .async                 = */ false,
                /* .host_buffer           = */ false,
                /* .buffer_from_host_ptr  = */ true,
                /* .events                = */ false,
            };
        }
        static void ggml_backend_cuda_device_get_props(ggml_backend_dev_t dev, ggml_backend_dev_props * props) {
            props->name        = ggml_backend_cuda_device_get_name(dev);
            props->description = ggml_backend_cuda_device_get_description(dev);
            props->type        = ggml_backend_cuda_device_get_type(dev);
            ggml_backend_cuda_device_get_memory(dev, &props->memory_free, &props->memory_total);
            props->caps = {
                /* .async                 = */ true,
                /* .host_buffer           = */ getenv("GGML_CUDA_NO_PINNED") == nullptr,
                /* .buffer_from_host_ptr  = */ false,
                /* .events                = */ true, //ifndef GGML_CUDA_NO_PEER_COPY
            };
        }
    if (ml.use_mmap && use_mmap_buffer && buffer_from_host_ptr_supported && is_default_buft)
        //for (uint32_t idx = 0; idx < ml.files.size(); idx++)
        // only the mmap region containing the tensors in the model is mapped to the backend buffer
        // this is important for metal with apple silicon: if the entire model could be mapped to a metal buffer, then we could just use metal for all layers
        // this allows using partial offloading when the model size exceeds the metal buffer size, but not the RAM size
        --> llama_model_loader::get_mapping_range --> void * addr, size_t first, last;
        --> ggml_backend_dev_buffer_from_host_ptr --> ggml_backend_cpu_buffer_from_ptr --> ggml_backend_buffer_t buf;
    else
        --> ggml_backend_alloc_ctx_tensors_from_buft --> alloc_tensor_range --> ggml_backend_buffer_t buf;
    
    // indicate that this buffer contains weights, this is used by ggml_backend_sched to improve op scheduling: ops that use a weight are preferably scheduled to the backend that contains the weight
    ggml_backend_buffer_set_usage(buf.second, GGML_BACKEND_BUFFER_USAGE_WEIGHTS);
    pimpl->bufs.emplace_back(buf);
    // if (use_mlock && ggml_backend_buffer_is_host(buf)) use_mlock 我们这里好像默认是false
    // for (uint32_t idx = 0; idx < ml.files.size(); idx++)
    buf_map.emplace(0, buf);

// populate tensors_by_name
for (auto & ctx : pimpl->ctxs) for (auto * cur = ggml_get_first_tensor(ctx.get()); cur != NULL; cur = ggml_get_next_tensor(ctx.get(), cur)) tensors_by_name.emplace_back(ggml_get_name(cur), cur);

// load tensor data
for (auto & it : ctx_bufs)
    ggml_context * ctx = it.first;
    auto & bufs = it.second;
    --> llama_model_loader::load_all_data;

for (auto & mapping : ml.mappings) pimpl->mappings.emplace_back(std::move(mapping));
return true;
```

#### [`make_cpu_buft_list`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L279)

```c++
pimpl->cpu_buft_list = make_cpu_buft_list(devices);
----------
static buft_list_t make_cpu_buft_list(const std::vector<ggml_backend_dev_t> & devices) {}
```

<details>
<summary>struct ggml_backend_buffer_type/ggml_backend_buffer_type_t</summary>

```c++
struct ggml_backend_buffer_type {
    struct ggml_backend_buffer_type_i  iface;
    ggml_backend_dev_t device;
    void * context;
};
typedef struct ggml_backend_buffer_type * ggml_backend_buffer_type_t;
// lists of buffer types used for each layer
using buft_list_t = std::vector<std::pair<ggml_backend_dev_t, ggml_backend_buffer_type_t>>;
```

</details>

```c++
// CPU: ACCEL -> GPU host -> CPU extra -> CPU
// add ACCEL buffer types

// add a host buffer type
// storing the tensors in a host buffer is useful when the processing of large batches
// is offloaded to a GPU device, since it reduces the time spent on data transfers
// generally, this will be done using the first device in the list
// a better approach would be to handle this on a weight-by-weight basis using the offload_op
// function of the device to determine if it would benefit from being stored in a host buffer
for (auto * dev : devices) // 这里的devices指的是GPU
    ggml_backend_buffer_type_t buft = ggml_backend_dev_host_buffer_type(dev);
        --> return device->iface.get_host_buffer_type(device);
        ----------
        static ggml_backend_buffer_type_t ggml_backend_cuda_device_get_host_buffer_type(ggml_backend_dev_t dev) {
            GGML_UNUSED(dev);
            --> return ggml_backend_cuda_host_buffer_type();
        }

    buft_list.emplace_back(dev, buft);

// add extra buffer types, only if no GPU device is present
// ref: https://github.com/ggml-org/llama.cpp/issues/12481#issuecomment-2743136094
auto * cpu_reg = ggml_backend_dev_backend_reg(cpu_dev);
auto ggml_backend_dev_get_extra_bufts_fn = (ggml_backend_dev_get_extra_bufts_t)
    ggml_backend_reg_get_proc_address(cpu_reg, "ggml_backend_dev_get_extra_bufts");
    ---> return reg->iface.get_proc_address(reg, name);
        --> return (void *) ggml_backend_dev_get_extra_bufts_t fct = ggml_backend_cpu_device_get_extra_buffers_type;
ggml_backend_buffer_type_t * extra_bufts = ggml_backend_dev_get_extra_bufts_fn(cpu_dev);
buft_list.emplace_back(cpu_dev, *extra_bufts);

// add the CPU buffer type
ggml_backend_dev_t dev = ggml_backend_dev_get(i);
buft_list.emplace_back(dev, ggml_backend_dev_buffer_type(dev)); //if (ggml_backend_dev_type(dev) == GGML_BACKEND_DEVICE_TYPE_CPU)
    --> return device->iface.get_buffer_type(device);
        static ggml_backend_buffer_type_t ggml_backend_cpu_device_get_buffer_type(ggml_backend_dev_t dev) {}
        --> return ggml_backend_cpu_buffer_type();
```

```log
>>> p buft_list
$94 = std::vector of length 3, capacity 4 = {[0] = {
    first = 0x555555a53760,
    second = 0x7ffff771adc0 <ggml_backend_cuda_host_buffer_type::ggml_backend_cuda_buffer_type_host>
  }, [1] = {
    first = 0x7ffff784a080 <ggml_backend_cpu_reg_get_device(ggml_backend_reg*, unsigned long)::ggml_backend_cpu_device>,
    second = 0x7ffff784a180 <ggml_backend_cpu_repack_buffer_type()::ggml_backend_cpu_buffer_type_repack>
  }, [2] = {
    first = 0x7ffff784a080 <ggml_backend_cpu_reg_get_device(ggml_backend_reg*, unsigned long)::ggml_backend_cpu_device>,
    second = 0x7fffe6c03b60 <ggml_backend_cpu_buffer_type::ggml_backend_cpu_buffer_type>
  }}
```

##### [`ggml_backend_cuda_host_buffer_type`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L1138)

```c++
ggml_backend_buffer_type_t ggml_backend_cuda_host_buffer_type() {
    static struct ggml_backend_buffer_type ggml_backend_cuda_buffer_type_host = {
        /* .iface    = */ {
            /* .get_name         = */ ggml_backend_cuda_host_buffer_type_name,
            /* .alloc_buffer     = */ ggml_backend_cuda_host_buffer_type_alloc_buffer,
            /* .get_alignment    = */ ggml_backend_cpu_buffer_type()->iface.get_alignment,
            /* .get_max_size     = */ NULL, // defaults to SIZE_MAX
            /* .get_alloc_size   = */ ggml_backend_cpu_buffer_type()->iface.get_alloc_size,
            /* .is_host          = */ ggml_backend_cpu_buffer_type()->iface.is_host,
        },
        /* .device   = */ ggml_backend_reg_dev_get(ggml_backend_cuda_reg(), 0),
        /* .context  = */ nullptr,
    };

    return &ggml_backend_cuda_buffer_type_host;
}
```

##### [`ggml_backend_cpu_repack_buffer_type`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/repack.cpp#L1555)

```c++
static ggml_backend_buffer_type_t * ggml_backend_cpu_device_get_extra_buffers_type(ggml_backend_dev_t device) {
    --> return ggml_backend_cpu_get_extra_buffers_type().data();

    GGML_UNUSED(device);
}

std::vector<ggml_backend_buffer_type_t>& ggml_backend_cpu_get_extra_buffers_type() {
    static std::vector<ggml_backend_buffer_type_t> bufts = []() {
        std::vector<ggml_backend_buffer_type_t> bufts;

#if defined(__AMX_INT8__) && defined(__AVX512VNNI__)
        if (ggml_backend_amx_buffer_type()) {
            bufts.push_back(ggml_backend_amx_buffer_type());
        }
#endif

#ifdef GGML_USE_CPU_KLEIDIAI
        if (ggml_backend_cpu_kleidiai_buffer_type()) {
            bufts.push_back(ggml_backend_cpu_kleidiai_buffer_type());
        }
#endif

#ifdef GGML_USE_CPU_REPACK
        if (ggml_backend_cpu_repack_buffer_type()) {
            --> bufts.push_back(ggml_backend_cpu_repack_buffer_type());
        }
#endif

        bufts.push_back(NULL);

        return bufts;
    }();

    return bufts;
}

ggml_backend_buffer_type_t ggml_backend_cpu_repack_buffer_type(void) {
    static struct ggml_backend_buffer_type ggml_backend_cpu_buffer_type_repack = {
        /* .iface    = */ {
                           /* .get_name         = */ ggml_backend_cpu_repack_buffer_type_get_name,
                           /* .alloc_buffer     = */ ggml_backend_cpu_repack_buffer_type_alloc_buffer,
                           /* .get_alignment    = */ ggml_backend_cpu_repack_buffer_type_get_alignment,
                           /* .get_max_size     = */ nullptr,  // defaults to SIZE_MAX
                           /* .get_alloc_size   = */ nullptr,  // defaults to ggml_nbytes
                           /* .is_host          = */ nullptr,
                           },
        /* .device  = */ ggml_backend_reg_dev_get(ggml_backend_cpu_reg(), 0),
        /* .context = */ new ggml::cpu::repack::extra_buffer_type(),
    };

    return &ggml_backend_cpu_buffer_type_repack;
}
```



##### [`ggml_backend_cpu_buffer_type`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1973)

```c++
ggml_backend_buffer_type_t ggml_backend_cpu_buffer_type(void) {
    static struct ggml_backend_buffer_type ggml_backend_cpu_buffer_type = {
        /* .iface   = */ {
            /* .get_name         = */ ggml_backend_cpu_buffer_type_get_name,
            /* .alloc_buffer     = */ ggml_backend_cpu_buffer_type_alloc_buffer,
            /* .get_alignment    = */ ggml_backend_cpu_buffer_type_get_alignment,
            /* .get_max_size     = */ NULL, // defaults to SIZE_MAX
            /* .get_alloc_size   = */ NULL, // defaults to ggml_nbytes
            /* .is_host          = */ ggml_backend_cpu_buffer_type_is_host,
        },
        /* .device  = */ NULL, // FIXME ggml_backend_reg_dev_get(ggml_backend_cpu_reg(), 0),
        /* .context = */ NULL,
    };

    return &ggml_backend_cpu_buffer_type;
}
```

#### [`ggml_backend_cuda_buffer_type`](https://github.com/ggml-org/llama.cpp/ggml/src/ggml-cuda/ggml-cuda.cu#L730)

```c++
buft_list_t buft_list = make_gpu_buft_list(dev, split_mode, tensor_split);
----------
// GPU: split if LLAMA_SPLIT_MODE_ROW -> GPU
static buft_list_t make_gpu_buft_list(ggml_backend_dev_t dev, llama_split_mode split_mode, const float * tensor_split) {
    // split_mode == LLAMA_SPLIT_MODE_LAYER
    // add the device default buffer type
    --> buft_list.emplace_back(dev, ggml_backend_dev_buffer_type(dev));
        --> return device->iface.get_buffer_type(device);
        static ggml_backend_buffer_type_t ggml_backend_cuda_device_get_buffer_type(ggml_backend_dev_t dev) {}
            ggml_backend_cuda_device_context * ctx = (ggml_backend_cuda_device_context *)dev->context;
            --> return ggml_backend_cuda_buffer_type(ctx->device);
}

ggml_backend_buffer_type_t ggml_backend_cuda_buffer_type(int device) {}
```

<details>
<summary>struct ggml_backend_cuda_buffer_type_context</summary>

```c++
struct ggml_backend_cuda_buffer_type_context {
    int device;
    std::string name;
};
```

</details>

```c++
static std::mutex mutex;
std::lock_guard<std::mutex> lock(mutex);

static ggml_backend_buffer_type ggml_backend_cuda_buffer_types[GGML_CUDA_MAX_DEVICES];
static bool ggml_backend_cuda_buffer_type_initialized = false;
if (!ggml_backend_cuda_buffer_type_initialized) {
    for (int i = 0; i < ggml_backend_cuda_get_device_count(); i++) {
        ggml_backend_cuda_buffer_types[i] = {
            /* .iface    = */ ggml_backend_cuda_buffer_type_interface,
            /* .device   = */ ggml_backend_reg_dev_get(ggml_backend_cuda_reg(), i),
            /* .context  = */ new ggml_backend_cuda_buffer_type_context{i, GGML_CUDA_NAME + std::to_string(i)},
        };
    }
    ggml_backend_cuda_buffer_type_initialized = true;
}
return &ggml_backend_cuda_buffer_types[device];
```

#### [`weight_buft_supported`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L140)

```c++
ggml_backend_buffer_type_t buft = select_weight_buft(hparams, t_meta, op, *buft_list);
----------
// find the first buffer type in the list that can use the tensor
static ggml_backend_buffer_type_t select_weight_buft(const llama_hparams & hparams, ggml_tensor * tensor, ggml_op op, const buft_list_t & buft_list) {
    for (const auto & cur : buft_list)
        ggml_backend_dev_t cur_dev = cur.first; // 目前只有CPU和GPU两个选项
        ggml_backend_buffer_type_t cur_buft = cur.second; // 有CUDA, CUDA_HOST, CPU_REPACK, CPU四个选项
        --> if (weight_buft_supported(hparams, tensor, op, cur_buft = cur.second, cur_dev = cur.first)) return cur_buft;
}

// checks if the weight tensor can be used with the specified buffer type and device
static bool weight_buft_supported(const llama_hparams & hparams, ggml_tensor * w, ggml_op op, ggml_backend_buffer_type_t buft, ggml_backend_dev_t dev) {}
```

```c++
ggml_init_params params = {
    /*.mem_size   =*/ ggml_tensor_overhead()*8,
    /*.mem_buffer =*/ NULL,
    /*.no_alloc   =*/ true,
};
ggml_context_ptr ctx_ptr { ggml_init(params) };

ggml_context * ctx = ctx_ptr.get();
ggml_tensor * op_tensor = nullptr;
```

```c++
switch (op) {
    case GGML_OP_GET_ROWS:
        {
            ggml_tensor * b = ggml_new_tensor_1d(ctx, GGML_TYPE_I32, 512);
            op_tensor = ggml_get_rows(ctx, w, b);
        } break;
    case GGML_OP_MUL_MAT:
        {
            ggml_tensor * b = ggml_new_tensor_4d(ctx, GGML_TYPE_F32, w->ne[0], 512, w->ne[2], w->ne[3]);
            op_tensor = ggml_mul_mat(ctx, w, b);
        } break;
    case GGML_OP_ADD:
        {
            ggml_tensor * a = ggml_new_tensor_4d(ctx, GGML_TYPE_F32, w->ne[0], w->ne[1], w->ne[2], w->ne[3]);
            op_tensor = ggml_add(ctx, a, w);
        } break;
    case GGML_OP_MUL:
        {
            ggml_tensor * a = ggml_new_tensor_4d(ctx, GGML_TYPE_F32, w->ne[0], w->ne[1], w->ne[2], w->ne[3]);
            op_tensor = ggml_mul(ctx, a, w);
        } break;
    case GGML_OP_DIV:
        {
            ggml_tensor * a = ggml_new_tensor_1d(ctx, GGML_TYPE_F32, w->ne[0]);
            op_tensor = ggml_div(ctx, a, w);
        } break;
    case GGML_OP_ROPE:
        {
            int n_embd_head = hparams.n_embd_head_v;
            int n_head = hparams.n_head();
            ggml_tensor * a = ggml_new_tensor_3d(ctx, GGML_TYPE_F32, n_embd_head, n_head, 512);
            ggml_tensor * b = ggml_new_tensor_1d(ctx, GGML_TYPE_I32, 512);
            op_tensor = ggml_rope_ext(
                ctx, a, b, w,
                0, 0, 0, 0, 0,
                0, 0, 0, 0
            );
        } break;
}
```


<details>
<summary>ggml_new_tensor</summary>

```c++
struct ggml_tensor * ggml_new_tensor_1d()
    return ggml_new_tensor(ctx, type, 1, &ne0);

struct ggml_tensor * ggml_new_tensor_2d()
    const int64_t ne[2] = { ne0, ne1};
    return ggml_new_tensor(ctx, type, 2, ne);

struct ggml_tensor * ggml_new_tensor_3d()
    const int64_t ne[3] = { ne0, ne1, ne2 };
    return ggml_new_tensor(ctx, type, 3, ne);

struct ggml_tensor * ggml_new_tensor_4d()
    const int64_t ne[4] = { ne0, ne1, ne2, ne3 };
    return ggml_new_tensor(ctx, type, 4, ne);

struct ggml_tensor * ggml_get_rows()
    struct ggml_tensor * result = ggml_new_tensor_4d(ctx, type, a->ne[0], b->ne[0], b->ne[1], b->ne[2]);
    result->op     = GGML_OP_GET_ROWS;
    result->src[0] = a;
    result->src[1] = b;
    return result;

static struct ggml_tensor * ggml_rope_impl()
    int sections[4] = {0, 0, 0, 0};

    struct ggml_tensor * result = inplace ? ggml_view_tensor(ctx, a) : ggml_dup_tensor(ctx, a);

    int32_t params[15] = { /*n_past*/ 0, n_dims, mode, /*n_ctx*/ 0, n_ctx_orig };
    memcpy(params +  5, &freq_base,    sizeof(float));
    memcpy(params +  6, &freq_scale,   sizeof(float));
    memcpy(params +  7, &ext_factor,   sizeof(float));
    memcpy(params +  8, &attn_factor,  sizeof(float));
    memcpy(params +  9, &beta_fast,    sizeof(float));
    memcpy(params + 10, &beta_slow,    sizeof(float));
    memcpy(params + 11, &sections,     sizeof(int)*4);
    ggml_set_op_params(result, params, sizeof(params));

    result->op     = GGML_OP_ROPE;
    result->src[0] = a;
    result->src[1] = b;
    result->src[2] = c;

    return result;
```
    
</details>

```c++
w->buffer = ggml_backend_buft_alloc_buffer(buft, 0);
    --> // return a dummy buffer for zero-sized allocations
        return ggml_backend_buffer_init(buft, {}, NULL, 0);
--> ggml_backend_dev_supports_op(dev, op_tensor) --> bool op_supported;
ggml_backend_buffer_free(w->buffer);
return op_supported;
```

[`ggml_backend_buffer_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L82)

```c++
ggml_backend_buffer_t ggml_backend_buffer_init(
               ggml_backend_buffer_type_t buft,
        struct ggml_backend_buffer_i      iface,
               void *                     context,
               size_t                     size) {
    ggml_backend_buffer_t buffer = new ggml_backend_buffer {
        /* .interface = */ iface,
        /* .buft      = */ buft,
        /* .context   = */ context,
        /* .size      = */ size,
        /* .usage     = */ GGML_BACKEND_BUFFER_USAGE_ANY
    };

    return buffer;
}
```

##### [`ggml_backend_cuda_device_supports_op`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L3002)

```c++
static bool ggml_backend_cuda_device_supports_op(ggml_backend_dev_t dev, const ggml_tensor * op) {
    ggml_backend_cuda_device_context * dev_ctx = (ggml_backend_cuda_device_context *) dev->context;
    // split buffers can only be used with GGML_OP_MUL_MAT
    if (op->op != GGML_OP_MUL_MAT) for (int i = 0; i < GGML_MAX_SRC; i++)
        if (op->src[i] && op->src[i]->buffer && ggml_backend_buft_is_cuda_split(op->src[i]->buffer->buft)) return false;
    // check if all the sources are allocated on this device
    for (int i = 0; i < GGML_MAX_SRC; i++)
    if (op->src[i] && op->src[i]->buffer && ggml_backend_buft_is_cuda(op->src[i]->buffer->buft))
        auto * buft_ctx = (ggml_backend_cuda_buffer_type_context *)op->src[i]->buffer->buft->context;
            if (buft_ctx->device != dev_ctx->device) return false;
    switch (op->op) {
        case GGML_OP_UNARY:
            switch (ggml_get_unary_op(op)) {
                case GGML_UNARY_OP_ABS:
                case GGML_UNARY_OP_SGN:
                case GGML_UNARY_OP_NEG:
                case GGML_UNARY_OP_STEP:
                case GGML_UNARY_OP_GELU:
                case GGML_UNARY_OP_SILU:
                case GGML_UNARY_OP_RELU:
                case GGML_UNARY_OP_SIGMOID:
                case GGML_UNARY_OP_HARDSIGMOID:
                case GGML_UNARY_OP_HARDSWISH:
                case GGML_UNARY_OP_GELU_ERF:
                case GGML_UNARY_OP_GELU_QUICK:
                case GGML_UNARY_OP_TANH:
                case GGML_UNARY_OP_EXP:
                    return ggml_is_contiguous(op->src[0]);
                default:
                    return false;
            }
            break;
        case GGML_OP_MUL_MAT:
        case GGML_OP_MUL_MAT_ID:
            {
                struct ggml_tensor * a = op->src[0];
                struct ggml_tensor * b = op->src[1];
                if (a->buffer && ggml_backend_buft_is_cuda_split(a->buffer->buft)) {
                    if (a->ne[2] > 1 || a->ne[3] > 1) {
                        return false;
                    }
                    // for small weight matrices the active device can end up without any rows, don't use row split in those cases
                    // this avoids some edge cases (and the performance would not be good anyways)
                    ggml_backend_cuda_split_buffer_type_context * buft_ctx = (ggml_backend_cuda_split_buffer_type_context *) a->buffer->buft->context;
                    int64_t row_low;
                    int64_t row_high;
                    get_row_split(&row_low, &row_high, a, buft_ctx->tensor_split, dev_ctx->device);
                    if (row_low == row_high) {
                        return false;
                    }
                }
                if (b->type == GGML_TYPE_F16 && a->type != GGML_TYPE_F16) {
                    return false;
                }
#ifdef GGML_USE_MUSA
                if (b->type == GGML_TYPE_F16 && b->ne[2]*b->ne[3] > 1 &&
                    !ggml_is_transposed(a) && !ggml_is_transposed(b)) {
                    return false;
                }
#endif // GGML_USE_MUSA
                switch (a->type) {
                    case GGML_TYPE_F32:
                    case GGML_TYPE_F16:
                    case GGML_TYPE_Q4_0:
                    case GGML_TYPE_Q4_1:
                    case GGML_TYPE_Q5_0:
                    case GGML_TYPE_Q5_1:
                    case GGML_TYPE_Q8_0:
                    case GGML_TYPE_Q2_K:
                    case GGML_TYPE_Q3_K:
                    case GGML_TYPE_Q4_K:
                    case GGML_TYPE_Q5_K:
                    case GGML_TYPE_Q6_K:
                    case GGML_TYPE_Q8_K:
                    case GGML_TYPE_IQ1_M:
                    case GGML_TYPE_IQ1_S:
                    case GGML_TYPE_IQ2_S:
                    case GGML_TYPE_IQ2_XS:
                    case GGML_TYPE_IQ2_XXS:
                    case GGML_TYPE_IQ3_S:
                    case GGML_TYPE_IQ3_XXS:
                    case GGML_TYPE_IQ4_NL:
                    case GGML_TYPE_IQ4_XS:
                    case GGML_TYPE_BF16:
#ifdef GGML_USE_MUSA
                        if (a->type == GGML_TYPE_Q3_K) {
                            return false;
                        }
#endif // GGML_USE_MUSA
                        return true;
                    default:
                        return false;
                }
            } break;
        case GGML_OP_OUT_PROD:
            return op->type == GGML_TYPE_F32 && op->src[0]->type == GGML_TYPE_F32 && op->src[1]->type == GGML_TYPE_F32;
        case GGML_OP_GET_ROWS:
            {
                switch (op->src[0]->type) {
                    case GGML_TYPE_F16:
                    case GGML_TYPE_F32:
                    case GGML_TYPE_Q4_0:
                    case GGML_TYPE_Q4_1:
                    case GGML_TYPE_Q5_0:
                    case GGML_TYPE_Q5_1:
                    case GGML_TYPE_Q8_0:
                        return true;
                    default:
                        return false;
                }
            } break;
        case GGML_OP_CPY:
            {
                ggml_type src0_type = op->src[0]->type;
                ggml_type src1_type = op->src[1]->type;
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_F32)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_BF16)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_F16)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_Q8_0)
                if (src0_type == GGML_TYPE_Q8_0 && src1_type == GGML_TYPE_F32)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_Q4_0)
                if (src0_type == GGML_TYPE_Q4_0 && src1_type == GGML_TYPE_F32)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_Q4_1)
                if (src0_type == GGML_TYPE_Q4_1 && src1_type == GGML_TYPE_F32)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_Q5_0)
                if (src0_type == GGML_TYPE_Q5_0 && src1_type == GGML_TYPE_F32)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_Q5_1)
                if (src0_type == GGML_TYPE_Q5_1 && src1_type == GGML_TYPE_F32)
                if (src0_type == GGML_TYPE_F32 && src1_type == GGML_TYPE_IQ4_NL)
                if (src0_type == GGML_TYPE_F16 && src1_type == GGML_TYPE_F16)
                if (src0_type == GGML_TYPE_F16 && src1_type == GGML_TYPE_F32)
                if (src0_type == src1_type && ggml_is_contiguous(op->src[0]) && ggml_is_contiguous(op->src[1]))
                    return true;
                return false;
            } break;
        case GGML_OP_DUP:
            {
                ggml_type src0_type = op->src[0]->type;
                return src0_type != GGML_TYPE_I32 && src0_type != GGML_TYPE_I16;
            } break;
        case GGML_OP_NORM:
        case GGML_OP_RMS_NORM:
        case GGML_OP_L2_NORM:
            return true;
        case GGML_OP_NONE:
        case GGML_OP_RESHAPE:
        case GGML_OP_VIEW:
        case GGML_OP_PERMUTE:
        case GGML_OP_TRANSPOSE:
        case GGML_OP_ADD:
        case GGML_OP_ADD1:
        case GGML_OP_SUB:
        case GGML_OP_MUL:
        case GGML_OP_DIV:
        case GGML_OP_SCALE:
        case GGML_OP_SQR:
        case GGML_OP_SQRT:
        case GGML_OP_SIN:
        case GGML_OP_COS:
        case GGML_OP_CLAMP:
        case GGML_OP_LOG:
        case GGML_OP_SSM_SCAN:
        case GGML_OP_SSM_CONV:
            return true;
        case GGML_OP_CONT:
            return op->src[0]->type != GGML_TYPE_BF16;
        case GGML_OP_DIAG_MASK_INF:
        case GGML_OP_SOFT_MAX:
            return true;
        case GGML_OP_ROPE:
        case GGML_OP_ROPE_BACK: {
            return op->src[0]->nb[0] == ggml_type_size(op->src[0]->type) && ggml_is_contiguous_2(op->src[0]);
        }
        default:
            return false;
    }
}
```

##### [`ggml_backend_cpu_device_supports_op`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.cpp:392)

```c++
static bool ggml_backend_cpu_device_supports_op(ggml_backend_dev_t dev, const struct ggml_tensor * op) {
    const struct ggml_tensor * src0 = op->src[0];
    const struct ggml_tensor * src1 = op->src[1];

    if (op->op == GGML_OP_NONE or GGML_OP_RESHAPE or GGML_OP_VIEW or GGML_OP_PERMUTE or GGML_OP_TRANSPOSE) {
        return true;
    }

    // extra_buffer_op?
    for (auto extra : ggml_backend_cpu_get_extra_buffers_type()) {
        // if (extra)
        auto buf_extra = (ggml::cpu::extra_buffer_type*) extra->context;
        --> if (buf_extra && buf_extra->supports_op(dev, op)) return true;
    }

    // the other case need host buffer.
    for (int i = 0; i < GGML_MAX_SRC; i++)
        if (op->src[i] && op->src[i]->buffer && !ggml_backend_buft_is_host(op->src[i]->buffer->buft)) return false;
        --> return buft->iface.is_host(buft);
            static bool ggml_backend_cpu_buffer_type_is_host(ggml_backend_buffer_type_t buft)
            --> return true;

    switch (op->op) {
        case GGML_OP_CPY:
            return
                op->type != GGML_TYPE_IQ3_XXS &&
                op->type != GGML_TYPE_IQ3_S   &&
                op->type != GGML_TYPE_IQ2_XXS &&
                op->type != GGML_TYPE_IQ2_XS  &&
                op->type != GGML_TYPE_IQ2_S   &&
                op->type != GGML_TYPE_IQ1_S   &&
                op->type != GGML_TYPE_IQ1_M; // missing type_traits.from_float
        case GGML_OP_MUL_MAT:
            return src1->type == GGML_TYPE_F32 || src1->type == ggml_get_type_traits_cpu(src0->type)->vec_dot_type;
        case GGML_OP_OUT_PROD:
            return (src0->type == GGML_TYPE_F32 || (ggml_is_quantized(src0->type) && src0->ne[2] == src1->ne[2] && src0->ne[3] == src1->ne[3])) &&
                src1->type == GGML_TYPE_F32 && op->type == GGML_TYPE_F32;
        default:
            return true;
    }
}
```

[`ggml::cpu::repack::extra_buffer_type::supports_op`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/repack.cpp#L1508)

```c++
namespace ggml::cpu::repack {
class extra_buffer_type : ggml::cpu::extra_buffer_type {
    bool supports_op(ggml_backend_dev_t, const struct ggml_tensor * op) override {
        if (op->op == GGML_OP_MUL_MAT && op->src[0]->buffer && (ggml_n_dims(op->src[0]) == 2) && 
            op->src[0]->buffer->buft == ggml_backend_cpu_repack_buffer_type() && 
            --> ggml_repack_get_optimal_repack_type(op->src[0])) {
            if (op->src[1]->buffer && !ggml_backend_buft_is_host(op->src[1]->buffer->buft)) {
                return false;
            }
            if (op->src[1]->type == GGML_TYPE_F32) {
                return true;
            }
            //if (op->src[1]->type == GGML_TYPE_Q8_0) {
            //    return true;
            //}
            // may be possible if Q8_0 packed...
        }
        return false;
    }

    ggml::cpu::tensor_traits * get_tensor_traits(const struct ggml_tensor * op) override {
        if (op->op == GGML_OP_MUL_MAT || op->op == GGML_OP_MUL_MAT_ID) {
            if (op->src[0]->buffer && op->src[0]->buffer->buft == ggml_backend_cpu_repack_buffer_type()) {
                return (ggml::cpu::tensor_traits *) op->src[0]->extra;
            }
        }
        return nullptr;
    }
};
}  // namespace ggml::cpu::repack
```

[`ggml_repack_get_optimal_repack_type`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/repack.cpp#L1416)

```c++
static const ggml::cpu::tensor_traits * ggml_repack_get_optimal_repack_type(const struct ggml_tensor * cur) {

    // instance for Q4
    static const ggml::cpu::repack::tensor_traits<block_q4_0, 4, 4, GGML_TYPE_Q8_0> q4_0_4x4_q8_0;
    static const ggml::cpu::repack::tensor_traits<block_q4_0, 8, 4, GGML_TYPE_Q8_0> q4_0_4x8_q8_0;
    static const ggml::cpu::repack::tensor_traits<block_q4_0, 8, 8, GGML_TYPE_Q8_0> q4_0_8x8_q8_0;
    static const ggml::cpu::repack::tensor_traits<block_q4_K, 8, 8, GGML_TYPE_Q8_K> q4_K_8x8_q8_K;

    // instance for IQ4
    static const ggml::cpu::repack::tensor_traits<block_iq4_nl, 4, 4, GGML_TYPE_Q8_0> iq4_nl_4x4_q8_0;

    if (cur->type == GGML_TYPE_Q4_0) {
        if (ggml_cpu_has_avx2() || (ggml_cpu_has_sve() && ggml_cpu_has_matmul_int8() && ggml_cpu_get_sve_cnt() == QK8_0)) {
            if (cur->ne[1] % 8 == 0) {
                return &q4_0_8x8_q8_0;
            }
        }
        if (ggml_cpu_has_neon() && ggml_cpu_has_matmul_int8()) {
            if (cur->ne[1] % 4 == 0) {
                return &q4_0_4x8_q8_0;
            }
        }
        if (ggml_cpu_has_neon() && ggml_cpu_has_dotprod()) {
            if (cur->ne[1] % 4 == 0) {
                return &q4_0_4x4_q8_0;
            }
        }
    } else if (cur->type == GGML_TYPE_Q4_K) {
        if (ggml_cpu_has_avx2()) {
            if (cur->ne[1] % 8 == 0) {
                return &q4_K_8x8_q8_K;
            }
        }
    } else if (cur->type == GGML_TYPE_IQ4_NL) {
        if (ggml_cpu_has_neon() && ggml_cpu_has_dotprod()) {
            if (cur->ne[1] % 4 == 0) {
                return &iq4_nl_4x4_q8_0;
            }
        }
    }

    return nullptr;
}
```

#### [`llama_model_loader::create_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L789)

```c++
return ml.create_tensor(ctx, tn, ne, flags);
----------
struct ggml_tensor * llama_model_loader::create_tensor(struct ggml_context * ctx, const std::string & name, const std::initializer_list<int64_t> & ne, int flags) {}
```

```c++
--> check_tensor_dims --> cur;
bool duplicated = flags & TENSOR_DUPLICATED;
--> ggml_dup_tensor --> tensor;
ggml_set_name(tensor, ggml_get_name(cur));
if (duplicated) {
    size_data += ggml_nbytes(cur);
} else {
    n_created++;
}
return tensor;
```

[`llama_model_loader::check_tensor_dims`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L759)

> 这个函数能检测到转置矩阵的存在，可以加以利用！

```c++
const struct ggml_tensor * cur = check_tensor_dims(name, ne, !(flags & TENSOR_NOT_REQUIRED));
----------
const struct ggml_tensor * llama_model_loader::check_tensor_dims(const std::string & name, const std::vector<int64_t> & ne, bool required) const {
    const struct ggml_tensor * cur = get_tensor_meta(name.c_str());
    return cur;
}
```

[`ggml_dup_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1692)

```c++
struct ggml_tensor * tensor = ggml_dup_tensor(ctx, cur);
----------
struct ggml_tensor * ggml_dup_tensor(struct ggml_context * ctx, const struct ggml_tensor * src) {
    return ggml_new_tensor(ctx, src->type, GGML_MAX_DIMS, src->ne);
}
```

#### [`llama_model_loader::init_mappings`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L845)

```c++
ml.init_mappings(true, nullptr);
----------
void llama_model_loader::init_mappings(bool prefetch, llama_mlocks * mlock_mmaps) {}
```

<details>
<summary>struct llama_mmap/llama_mmaps</summary>

```c++
struct llama_mmap {
    llama_mmap(const llama_mmap &) = delete;
    llama_mmap(struct llama_file * file, size_t prefetch = (size_t) -1, bool numa = false);
    ~llama_mmap();

    size_t size() const;
    void * addr() const;

    void unmap_fragment(size_t first, size_t last);

    static const bool SUPPORTED;

private:
    struct impl;
    std::unique_ptr<impl> pimpl;
};
using llama_mmaps  = std::vector<std::unique_ptr<llama_mmap>>;

size_t llama_mmap::size() const { return pimpl->size; }
void * llama_mmap::addr() const { return pimpl->addr; }

void llama_mmap::unmap_fragment(size_t first, size_t last) { pimpl->unmap_fragment(first, last); }
```

</details>

```c++
// for (const auto & file : files)
--> llama_mmap::llama_mmap --> llama_mmap::impl::impl --> mapping;
llama_model_loader:: std::vector<std::pair<size_t, size_t>>  mmaps_used.emplace_back(mapping->size(), 0);
llama_model_loader:: llama_mmaps mappings.emplace_back(std::move(mapping));
```

[`llama_mmap::llama_mmap`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-mmap.cpp#L441)

```c++
std::unique_ptr<llama_mmap> mapping = std::make_unique<llama_mmap>(file.get(), prefetch ? -1 : 0, is_numa);
----------
llama_mmap::llama_mmap(struct llama_file * file, size_t prefetch, bool numa) : pimpl(std::make_unique<impl>(file, prefetch, numa)) {}
struct llama_mmap::impl {
    void * addr;
    size_t size;
    std::vector<std::pair<size_t, size_t>> mapped_fragments;
    static void align_range(size_t * first, size_t * last, size_t page_size) {
        size_t offset_in_page = *first & (page_size - 1);
        size_t offset_to_page = offset_in_page == 0 ? 0 : page_size - offset_in_page;
        *first += offset_to_page;

        *last = *last & ~(page_size - 1);

        if (*last <= *first) {
            *last = *first;
        }
    }

    impl(struct llama_file * file, size_t prefetch, bool numa) {
        ---> size = file->size();
        int fd = file->file_id();
        int flags = MAP_SHARED;
        if (prefetch) { flags |= MAP_POPULATE; }
        ---> addr = mmap(NULL, file->size(), PROT_READ, flags, fd, 0);
        if (prefetch > 0) {
            if (posix_madvise(addr, std::min(file->size(), prefetch), POSIX_MADV_WILLNEED)) {
                LLAMA_LOG_WARN("warning: posix_madvise(.., POSIX_MADV_WILLNEED) failed: %s\n",
                        strerror(errno));
            }
        }
        ---> mapped_fragments.emplace_back(0, file->size());
    }
}
```

#### [`llama_model_loader::get_mapping_range`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L878)

```c++
ml.get_mapping_range(&first, &last, &addr, idx=0, ctx);
----------
void llama_model_loader::get_mapping_range(size_t * first, size_t * last, void ** addr, int idx, ggml_context * ctx) const {}
```

```c++
const auto & mapping = mappings.at(idx);

*first = mapping->size();
*last  = 0;
*addr = mapping->addr();
for (ggml_tensor * tensor = ggml_get_first_tensor(ctx); tensor; tensor = ggml_get_next_tensor(ctx, tensor)) {
    const auto * weight = get_weight(ggml_get_name(tensor));
    if (!weight || weight->idx != idx) {
        continue;
    }
    *first = std::min(*first, weight->offs);
    *last  = std::max(*last,  weight->offs + ggml_nbytes(tensor));
}
```

<details>
<summary>struct llama_model_loader::llama_tensor_weight</summary>

```c++
// Holds information on a model weight
struct llama_tensor_weight {
    uint16_t  idx; // source file index
    size_t   offs; // tensor data offset in the original file
    ggml_tensor * tensor;
}

const llama_model_loader::llama_tensor_weight * llama_model_loader::get_weight(const char * name) const {
    auto pos = weights_map.find(name);
    if (pos != weights_map.end()) {
        return &pos->second;
    }

    return nullptr;
}
```

</details>


#### [`ggml_backend_cpu_buffer_from_ptr`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L2013)

```c++
ggml_backend_buffer_t buf = ggml_backend_dev_buffer_from_host_ptr(dev, (char *) addr + first, last - first, ggml_get_max_tensor_size(ctx));
----------
ggml_backend_buffer_t ggml_backend_dev_buffer_from_host_ptr(ggml_backend_dev_t device, void * ptr, size_t size, size_t max_tensor_size) {
    --> return device->iface.buffer_from_host_ptr(device, ptr, size, max_tensor_size);
}

static ggml_backend_buffer_t ggml_backend_cpu_device_buffer_from_host_ptr(ggml_backend_dev_t dev, void * ptr, size_t size, size_t max_tensor_size) {
    --> return ggml_backend_cpu_buffer_from_ptr(ptr, size);
}

ggml_backend_buffer_t ggml_backend_cpu_buffer_from_ptr(void * ptr, size_t size) {
    --> return ggml_backend_buffer_init(ggml_backend_cpu_buffer_from_ptr_type(), ggml_backend_cpu_buffer_from_ptr_i, ptr, size);
}
```

[`ggml_backend_buffer_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L82)

<details>
<summary>struct ggml_backend_buffer/ggml_backend_buffer_t</summary>

```c++
struct ggml_backend_buffer {
    struct ggml_backend_buffer_i  iface;
    ggml_backend_buffer_type_t    buft;
    void * context;
    size_t size;
    enum ggml_backend_buffer_usage usage;
};

typedef struct ggml_backend_buffer * ggml_backend_buffer_t;
```

</details>

```c++
// backend buffer

ggml_backend_buffer_t ggml_backend_buffer_init(
               ggml_backend_buffer_type_t buft,
        struct ggml_backend_buffer_i      iface,
               void *                     context,
               size_t                     size) {
    ggml_backend_buffer_t buffer = new ggml_backend_buffer {
        /* .interface = */ iface,
        /* .buft      = */ buft,
        /* .context   = */ context,
        /* .size      = */ size,
        /* .usage     = */ GGML_BACKEND_BUFFER_USAGE_ANY
    };

    return buffer;
}
```

[`ggml_backend_cpu_buffer_from_ptr_type`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1996)

```c++

static ggml_backend_buffer_type_t ggml_backend_cpu_buffer_from_ptr_type(void) {
    static struct ggml_backend_buffer_type ggml_backend_cpu_buffer_type = {
        /* .iface   = */ {
            /* .get_name         = */ ggml_backend_cpu_buffer_from_ptr_type_get_name,
            /* .alloc_buffer     = */ ggml_backend_cpu_buffer_type_alloc_buffer,
            /* .get_alignment    = */ ggml_backend_cpu_buffer_type_get_alignment,
            /* .get_max_size     = */ NULL, // defaults to SIZE_MAX
            /* .get_alloc_size   = */ NULL, // defaults to ggml_nbytes
            /* .is_host          = */ ggml_backend_cpu_buffer_type_is_host,
        },
        /* .device  = */ NULL, // FIXME ggml_backend_reg_dev_get(ggml_backend_cpu_reg(), 0),
        /* .context = */ NULL,
    };

    return &ggml_backend_cpu_buffer_type;
}
```

#### [`alloc_tensor_range`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L946)

[`ggml_backend_alloc_ctx_tensors_from_buft`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L987)

```c++
ggml_backend_buffer_t buf = ggml_backend_alloc_ctx_tensors_from_buft(ctx, buft);
----------
ggml_backend_buffer_t ggml_backend_alloc_ctx_tensors_from_buft(struct ggml_context * ctx, ggml_backend_buffer_type_t buft) {
    size_t alignment = ggml_backend_buft_get_alignment(buft);
    size_t max_size = ggml_backend_buft_get_max_size(buft);
    size_t cur_buf_size = 0;
    struct ggml_tensor * first = ggml_get_first_tensor(ctx);
    for (struct ggml_tensor * t = first; t != NULL; t = ggml_get_next_tensor(ctx, t))
        size_t this_size = GGML_PAD(ggml_backend_buft_get_alloc_size(buft, t), alignment); --> ggml_nbytes(tensor)
        cur_buf_size += this_size;
    // allocate remaining tensors
    --> alloc_tensor_range(ctx, first, NULL, buft, cur_buf_size, &ggml_backend_buffer_t * buffers, &size_t n_buffers);

    ggml_backend_buffer_t buffer = buffers[0];
    free(buffers);
    return buffer;
}

static bool alloc_tensor_range(struct ggml_context * ctx,
        struct ggml_tensor * first, struct ggml_tensor * last,
        ggml_backend_buffer_type_t buft, size_t size,
        ggml_backend_buffer_t ** buffers, size_t * n_buffers) {}
```


```c++
// ggml_backend_buffer_t buffer = ggml_backend_buft_alloc_buffer(buft, size);
--> ggml_backend_buft_alloc_buffer --> ggml_backend_buffer_t buffer;
    ggml_backend_buffer_t ggml_backend_buft_alloc_buffer(ggml_backend_buffer_type_t buft, size_t size)
    --> return buft->iface.alloc_buffer(buft, size);
        static ggml_backend_buffer_t ggml_backend_cuda_buffer_type_alloc_buffer(ggml_backend_buffer_type_t buft, size_t size) {}
*buffers = realloc(*buffers, sizeof(ggml_backend_buffer_t) * (*n_buffers + 1));
(*buffers)[(*n_buffers)++] = buffer;
--> ggml_tallocr_new --> struct ggml_tallocr tallocr;
for (struct ggml_tensor * t = first; t != last; t = ggml_get_next_tensor(ctx, t))
    --> ggml_tallocr_alloc;
```

##### [`ggml_backend_cuda_buffer_type_alloc_buffer`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L680)

<details>
<summary>struct ggml_backend_cuda_buffer_context</summary>

```c++
struct ggml_backend_cuda_buffer_context {
    int device;
    void * dev_ptr = nullptr;
    std::string name;
}
```

</details>

```c++
ggml_backend_cuda_buffer_type_context * buft_ctx = (ggml_backend_cuda_buffer_type_context *)buft->context;
ggml_cuda_set_device(buft_ctx->device);
--> cudaError_t err = ggml_cuda_device_malloc(&void *dev_ptr, size, buft_ctx->device);
ggml_backend_cuda_buffer_context * ctx = new ggml_backend_cuda_buffer_context(buft_ctx->device, dev_ptr);
return ggml_backend_buffer_init(buft, ggml_backend_cuda_buffer_interface, ctx, size);
```

[`ggml_cuda_device_malloc`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L100)
```c++
static cudaError_t ggml_cuda_device_malloc(void ** ptr, size_t size, int device) {
    ggml_cuda_set_device(device);
    cudaError_t err;
    if (getenv("GGML_CUDA_ENABLE_UNIFIED_MEMORY") != nullptr) err = cudaMallocManaged(ptr, size);
    else err = cudaMalloc(ptr, size);
    return err;
}
```

```log
>>> p size
$335 = 2463531264
```

##### [`ggml_tallocr_new`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L77)

> tallocr 看起来就是buffer的管理器而已，因为base, align和offset都是靠buffer拿到的。应该tallocr只会管理着offset

<details>
<summary>struct ggml_tallocr</summary>

```c++
// Tensor allocator
struct ggml_tallocr {
    ggml_backend_buffer_t buffer;
    void * base;
    size_t alignment;
    size_t offset;
};
```

</details>

```c++
struct ggml_tallocr tallocr = ggml_tallocr_new(buffer);
----------
struct ggml_tallocr ggml_tallocr_new(ggml_backend_buffer_t buffer) {
    void * base = ggml_backend_buffer_get_base(buffer);
    size_t align = ggml_backend_buffer_get_alignment(buffer);

    assert(align && !(align & (align - 1))); // power of 2

    struct ggml_tallocr talloc = (struct ggml_tallocr) {
        /*.buffer    = */ buffer,
        /*.base      = */ base,
        /*.alignment = */ align,
        /*.offset    = */ aligned_offset(base, 0, align),
    };
    return talloc;
}
```

##### [`ggml_tallocr_alloc`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L92)

```c++
status = ggml_tallocr_alloc(&tallocr, t);
----------
enum ggml_status ggml_tallocr_alloc(struct ggml_tallocr * talloc, struct ggml_tensor * tensor) {
    size_t size = ggml_backend_buffer_get_alloc_size(talloc->buffer, tensor);
    --> return ggml_backend_buft_get_alloc_size(ggml_backend_buffer_get_type(buffer), tensor); --> ggml_nbytes(tensor);

    size = GGML_PAD(size, talloc->alignment);

    void * addr = (char *)ggml_backend_buffer_get_base(talloc->buffer) + talloc->offset;
    talloc->offset += size;

    assert(((uintptr_t)addr % talloc->alignment) == 0);

    return ggml_backend_tensor_alloc(talloc->buffer, tensor, addr);
    ---> // 非常重要，是更进一步的标志！ *key*
        tensor->buffer = buffer;
        tensor->data = addr;
        --> return ggml_backend_buffer_init_tensor(buffer, tensor);
            static enum ggml_status ggml_backend_cuda_buffer_init_tensor(ggml_backend_buffer_t buffer, ggml_tensor * tensor) {}
}
```

[`ggml_backend_cuda_buffer_init_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L578)

```c++
ggml_backend_cuda_buffer_context * ctx = (ggml_backend_cuda_buffer_context *)buffer->context;
if (tensor->view_src != NULL) {
    assert(tensor->view_src->buffer->buft == buffer->buft);
    return GGML_STATUS_SUCCESS;
}
if (ggml_is_quantized(tensor->type) && tensor->view_src == nullptr && ggml_backend_buffer_get_usage(buffer) != GGML_BACKEND_BUFFER_USAGE_COMPUTE) {
    // initialize padding to 0 to avoid possible NaN values
    const size_t original_size = ggml_nbytes(tensor);
    const size_t padded_size = ggml_backend_buft_get_alloc_size(buffer->buft, tensor);
    if (padded_size > original_size) {
        ggml_cuda_set_device(ctx->device);
        cudaMemset((char *)tensor->data + original_size, 0, padded_size - original_size);
    }
}
return GGML_STATUS_SUCCESS;
```

#### [`llama_model_loader::load_all_data`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L918)

```c++
ml.load_all_data(ctx, bufs, use_mlock ? &pimpl->mlock_mmaps : NULL, params.progress_callback, params.progress_callback_user_data);
----------
bool llama_model_loader::load_all_data(
        struct ggml_context * ctx,
        llama_buf_map & bufs,
        llama_mlocks * lmlocks,
        llama_progress_callback progress_callback,
        void * progress_callback_user_data) {}
```

```c++
for (struct ggml_tensor * cur = ggml_get_first_tensor(ctx); cur != NULL; cur = ggml_get_next_tensor(ctx, cur))
    const auto * weight = get_weight(ggml_get_name(cur));
    size_t n_size = ggml_nbytes(cur);
    const auto & mapping = mappings.at(weight->idx);
    ggml_backend_buffer_t buf_mmap = bufs.at(weight->idx);
    uint8_t * data = (uint8_t *) mapping->addr() + weight->offs;
    // either we have a buffer to allocate the tensor in, or it is already allocated
    GGML_ASSERT(buf_mmap || cur->data);
    if (buf_mmap && cur->data == nullptr)
        // CPU 侧的 tensor 的 buffer 和 data 在这儿赋值的
        --> ggml_backend_tensor_alloc(buffer=buf_mmap, tensor=cur, addr=data);
            tensor->buffer = buffer;
            tensor->data = addr;
            // 下面的CPU没有提供函数指针，所以不起作用
            // return ggml_backend_buffer_init_tensor(buffer, tensor);
        auto & mmap_used = mmaps_used[weight->idx];
        mmap_used.first  = std::min(mmap_used.first,  weight->offs);
        mmap_used.second = std::max(mmap_used.second, weight->offs + n_size);
    else // *key* 非常关键的一步，将权重拷贝到GPU显存中
        --> ggml_backend_tensor_set(cur, data, 0, n_size);
            ggml_backend_buffer_t buf = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
            --> buf->iface.set_tensor(buf, tensor, data, offset, size) --> ggml_backend_cuda_buffer_set_tensor;
    size_done += n_size;
// check if this is the last call and do final cleanup
for (uint32_t idx = 0; idx < mappings.size(); idx++)
    const auto & mmap_used = mmaps_used.at(idx);
    auto & mapping = mappings.at(idx);
    --> mapping->unmap_fragment(0, mmap_used.first);
    --> if (mmap_used.second != 0) mapping->unmap_fragment(mmap_used.second, mapping->size());
return true;
```

[`llama_mmap::impl::unmap_fragment`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-mmap.cpp#L321)

```c++
void llama_mmap::unmap_fragment(size_t first, size_t last) { pimpl->unmap_fragment(first, last); }

struct llama_mmap::impl {
    void unmap_fragment(size_t first, size_t last) {
        int page_size = sysconf(_SC_PAGESIZE);
        align_range(&first, &last, page_size);
        size_t len = last - first;
        void * next_page_start = (uint8_t *) addr + first;
        munmap(next_page_start, len);
        for (const auto & frag : mapped_fragments) new_mapped_fragments.emplace_back(last, frag.second);
        mapped_fragments = std::move(new_mapped_fragments);
    }
}
```

## [`llama_context::llama_context`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L19)

```c++
llama_context * lctx = llama_init_from_model(model, cparams);
----------
llama_context * llama_init_from_model(
                 llama_model * model,
        llama_context_params   params) {
    --> auto * ctx = new llama_context(*model, params);
    return ctx;
}

llama_context::llama_context(
        const llama_model & model,
              llama_context_params params) :
    model(model),
    batch_allocr(std::make_unique<llama_batch_allocr>()) {}
```

<details>
<summary>struct llama_context</summary>

```c++
struct llama_context {
private:
    const llama_model & model;

    llama_cparams       cparams;

    std::unique_ptr<llama_memory_i> memory;

    // TODO: temporary, until the llama_kv_self_defrag() API is removed
    bool memory_force_optimize = false;

    // decode output (2-dimensional array: [n_outputs][n_vocab])
    size_t  logits_size = 0; // capacity (of floats) for logits
    float * logits      = nullptr;

    // reuse the batch_allocr to avoid unnecessary memory allocations
    std::unique_ptr<llama_batch_allocr> batch_allocr;

    int32_t n_outputs     = 0; // number of actually-used outputs in the current ubatch or last logical batch
    int32_t n_outputs_max = 0; // capacity (of tokens positions) for the output buffers

    std::vector<int32_t> output_ids; // map batch token positions to ids of the logits and embd buffers

    ggml_backend_sched_ptr sched;

    ggml_backend_t backend_cpu = nullptr;
    std::vector<ggml_backend_ptr> backends;

    ggml_context_ptr ctx_compute;

    // training
    ggml_opt_context_t opt_ctx = nullptr;

    ggml_threadpool_t threadpool       = nullptr;
    ggml_threadpool_t threadpool_batch = nullptr;

    ggml_abort_callback abort_callback      = nullptr;
    void *              abort_callback_data = nullptr;

    std::vector<std::pair<ggml_backend_t, ggml_backend_set_n_threads_t>> set_n_threads_fns;

    // buffer types used for the compute buffer of each backend
    std::vector<ggml_backend_t>             backend_ptrs;
    std::vector<ggml_backend_buffer_type_t> backend_buft;

    // memory buffers used to evaluate the model
    std::vector<uint8_t> buf_compute_meta;

    // host buffer for the model output (logits and embeddings)
    ggml_backend_buffer_ptr buf_output;

    bool has_evaluated_once = false;

    // perf
    mutable int64_t t_start_us  = 0;
    mutable int64_t t_load_us   = 0;
    mutable int64_t t_p_eval_us = 0;
    mutable int64_t t_eval_us   = 0;

    mutable int64_t t_compute_start_us = 0;
    mutable int64_t n_queued_tokens    = 0;

    mutable int32_t n_p_eval = 0; // number of tokens in eval calls for the prompt (with batch size > 1)
    mutable int32_t n_eval   = 0; // number of eval calls
};
```

</details>

<details>
<summary>struct ggml_backend/ggml_backend_t</summary>

```c++
struct ggml_backend {
    ggml_guid_t guid;
    struct ggml_backend_i iface;
    ggml_backend_dev_t device;
    void * context;
};
typedef struct ggml_backend * ggml_backend_t;
```

</details>

```c++
// GPU backends
for (auto * dev : model.devices) llama_context:: std::vector<ggml_backend_ptr> backends.emplace_back(
    --> ggml_backend_dev_init(dev, nullptr));
    static ggml_backend_t ggml_backend_cuda_device_init_backend(ggml_backend_dev_t dev, const char * params)
        --> ggml_backend_cuda_init;
// add ACCEL backends (such as BLAS)
// add CPU backend
--> ggml_backend_init_by_type --> ggml_backend_dev_init --> ggml_backend_cpu_device_init_backend --> ggml_backend_cpu_init --> llama_context:: ggml_backend_t backend_cpu;
backends.emplace_back(backend_cpu);
// create a list of the set_n_threads functions in the backends
//for (auto & backend : backends)
auto ggml_backend_set_n_threads_fn = (ggml_backend_set_n_threads_t) ggml_backend_reg_get_proc_address(ggml_backend_dev_backend_reg(ggml_backend_get_device(backend.get())), "ggml_backend_set_n_threads");
llama_context:: std::vector<std::pair<ggml_backend_t, ggml_backend_set_n_threads_t>> set_n_threads_fns.emplace_back(backend.get(), ggml_backend_set_n_threads_fn);
// graph outputs buffer
// resized during inference when a batch uses more outputs
--> llama_context::output_reserve --> llama_context:: logits_size, output_ids, buf_output, logits, n_outputs=0;
// init the memory module
llama_memory_params params_mem = {
    /*.type_k   =*/ params.type_k,
    /*.type_v   =*/ params.type_v,
    /*.swa_full =*/ params.swa_full,
};
--> llama_model::create_memory --> llama_kv_cache_unified::llama_kv_cache_unified --> llama_context:: std::unique_ptr<llama_memory_i> memory;
// init backends
llama_context:: std::vector<ggml_backend_t> backend_buft.clear();
llama_context:: std::vector<ggml_backend_buffer_type_t> backend_ptrs.clear();
for (auto & backend : backends)
    auto * buft = ggml_backend_get_default_buffer_type(backend.get());
    auto backend_type = ggml_backend_dev_type(ggml_backend_get_device(backend.get()));
    if (backend_type == GGML_BACKEND_DEVICE_TYPE_CPU && !model.devices.empty()) {
        // use the host buffer of the first device CPU for faster transfer of the intermediate state
        auto * dev = model.devices[0];
        auto * host_buft = ggml_backend_dev_host_buffer_type(dev);
        if (host_buft) buft = host_buft;
    }
    // buffer types used for the compute buffer of each backend
    backend_buft.push_back(buft);
    backend_ptrs.push_back(backend.get());
// memory buffers used to evaluate the model
// buffer used to store the computation graph and the tensor meta data
const size_t max_nodes = this->graph_max_nodes();
--> ggml_graph_nbytes --> llama_context:: std::vector<uint8_t> buf_compute_meta.resize(ggml_tensor_overhead()*max_nodes + ggml_graph_overhead_custom(max_nodes, false));
--> ggml_backend_sched_new --> llama_context:: ggml_backend_sched_ptr sched.reset(ggml_backend_sched_new(backend_ptrs.data(), backend_buft.data(), backend_ptrs.size(), max_nodes, pipeline_parallel, cparams.op_offload));
// reserve worst-case graph
// simulate full KV cache
// const auto mctx = memory->init_full();
--> llama_kv_cache_unified_state::llama_kv_cache_unified_state --> std::make_unique<llama_kv_cache_unified_context> mctx;
const uint32_t n_seqs = cparams.n_seq_max;
const uint32_t n_tokens = std::min(cparams.n_ctx, cparams.n_ubatch);
// reserve pp graph first so that buffers are only allocated once
auto * gf   = graph_reserve(n_tokens, n_seqs, n_tokens, mctx.get());
n_splits_pp = ggml_backend_sched_get_n_splits(sched.get());
n_nodes_pp  = ggml_graph_n_nodes(gf);
// reserve with tg graph to get the number of splits and nodes
auto * gf   = graph_reserve(1, 1, 1, mctx.get());
n_splits_tg = ggml_backend_sched_get_n_splits(sched.get());
n_nodes_tg  = ggml_graph_n_nodes(gf);
// reserve again with pp graph to avoid ggml-alloc reallocations during inference
auto * gf   = graph_reserve(n_tokens, n_seqs, n_tokens, mctx.get());
```

### [`ggml_backend_cuda_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L3545)

```c++
return ggml_backend_cuda_init(ctx->device);
----------
ggml_backend_t ggml_backend_cuda_init(int device) {}
```

<details>
<summary>struct ggml_backend_cuda_context</summary>

```c++
struct ggml_backend_cuda_context {
    int device;
    std::string name;
    cudaEvent_t copy_event = nullptr;

    cudaStream_t streams[GGML_CUDA_MAX_DEVICES][GGML_CUDA_MAX_STREAMS] = { { nullptr } };
    cublasHandle_t cublas_handles[GGML_CUDA_MAX_DEVICES] = {nullptr};

    std::unique_ptr<ggml_cuda_graph> cuda_graph;

    explicit ggml_backend_cuda_context(int device) :
        device(device),
        name(GGML_CUDA_NAME + std::to_string(device)) {
    }

    ~ggml_backend_cuda_context();

    cudaStream_t stream(int device, int stream) {
        if (streams[device][stream] == nullptr) {
            ggml_cuda_set_device(device);
            CUDA_CHECK(cudaStreamCreateWithFlags(&streams[device][stream], cudaStreamNonBlocking));
        }
        return streams[device][stream];
    }

    cudaStream_t stream() {
        return stream(device, 0);
    }

    cublasHandle_t cublas_handle(int device) {
        if (cublas_handles[device] == nullptr) {
            ggml_cuda_set_device(device);
            CUBLAS_CHECK(cublasCreate(&cublas_handles[device]));
            CUBLAS_CHECK(cublasSetMathMode(cublas_handles[device], CUBLAS_TF32_TENSOR_OP_MATH));
        }
        return cublas_handles[device];
    }

    cublasHandle_t cublas_handle() {
        return cublas_handle(device);
    }

    // pool
    std::unique_ptr<ggml_cuda_pool> pools[GGML_CUDA_MAX_DEVICES];

    static std::unique_ptr<ggml_cuda_pool> new_pool_for_device(int device);

    ggml_cuda_pool & pool(int device) {
        if (pools[device] == nullptr) {
            pools[device] = new_pool_for_device(device);
        }
        return *pools[device];
    }

    ggml_cuda_pool & pool() {
        return pool(device);
    }
};
```

</details>

```c++
ggml_backend_cuda_context * ctx = new ggml_backend_cuda_context(device);
ggml_backend_t cuda_backend = new ggml_backend {
    /* .guid      = */ ggml_backend_cuda_guid(),
    /* .interface = */ ggml_backend_cuda_interface,
    /* .device    = */ ggml_backend_reg_dev_get(ggml_backend_cuda_reg(), device),
    /* .context   = */ ctx,
};
return cuda_backend;
```

### [`ggml_backend_cpu_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.cpp#L196)

```c++
backend_cpu = ggml_backend_init_by_type(GGML_BACKEND_DEVICE_TYPE_CPU, nullptr);
----------
ggml_backend_t ggml_backend_init_by_type(enum ggml_backend_dev_type type, const char * params) {
    ggml_backend_dev_t dev = ggml_backend_dev_by_type(type);
    if (!dev) {
        return nullptr;
    }
    --> return ggml_backend_dev_init(dev, params);
}


ggml_backend_dev_t ggml_backend_dev_by_type(enum ggml_backend_dev_type type)
    for (size_t i = 0; i < ggml_backend_dev_count(); i++) {
        ggml_backend_dev_t dev = ggml_backend_dev_get(i);
        if (ggml_backend_dev_type(dev) == type) {
            return dev;
        }
    }

ggml_backend_t ggml_backend_dev_init(ggml_backend_dev_t device, const char * params)
    --> return device->iface.init_backend(device, params);

static ggml_backend_t ggml_backend_cpu_device_init_backend(ggml_backend_dev_t dev, const char * params)
    --> return ggml_backend_cpu_init();

ggml_backend_t ggml_backend_cpu_init(void) {}
```

<details>
<summary>struct ggml_backend_cpu_context</summary>

```c++
struct ggml_backend_cpu_context {
    int                 n_threads;
    ggml_threadpool_t   threadpool;

    uint8_t *           work_data;
    size_t              work_size;

    ggml_abort_callback abort_callback;
    void *              abort_callback_data;
};
```
    
</details>

```c++
// initialize CPU backend now to avoid slowing the first graph computation
ggml_cpu_init();
struct ggml_backend_cpu_context * ctx = new ggml_backend_cpu_context;
ctx->n_threads           = GGML_DEFAULT_N_THREADS;
ctx->threadpool          = NULL;
ctx->work_data           = NULL;
ctx->work_size           = 0;
ctx->abort_callback      = NULL;
ctx->abort_callback_data = NULL;
ggml_backend_t cpu_backend = new ggml_backend {
    /* .guid      = */ ggml_backend_cpu_guid(),
    /* .interface = */ ggml_backend_cpu_i,
    /* .device    = */ ggml_backend_reg_dev_get(ggml_backend_cpu_reg(), 0),
    /* .context   = */ ctx,
};
return cpu_backend;
```

### [`llama_context::output_reserve`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1239)

```c++
output_reserve(params.n_seq_max);
----------
int32_t llama_context::output_reserve(int32_t n_outputs) {}
```

```c++
const int64_t n_outputs_max = std::max<int64_t>(n_outputs, n_seq_max());
logits_size = n_vocab*n_outputs_max;
// map batch token positions to ids of the logits and embd buffers
// init, never resized afterwards
llama_context:: std::vector<int32_t> output_ids.resize(n_batch);
const size_t new_size  = (logits_size + embd_size) * sizeof(float);
// try to use the host buffer of the device where the output tensor is allocated for faster transfer to system memory
auto * buft = ggml_backend_dev_host_buffer_type(model.dev_output());
// host buffer for the model output (logits and embeddings)
//buf_output.reset(ggml_backend_buft_alloc_buffer(buft, new_size));
--> ggml_backend_buft_alloc_buffer
    --> ggml_backend_cpu_buffer_type_alloc_buffer
    --> ggml_backend_cuda_host_buffer_type_alloc_buffer // only if ngl > n_layers
        --> llama_context:: ggml_backend_buffer_ptr buf_output;
--> ggml_backend_buffer_get_base --> ggml_backend_cpu_buffer_get_base --> output_base;
llama_context:: float * logits = output_base;
// set all ids as invalid (negative)
std::fill(output_ids.begin(), output_ids.end(), -1);
this->n_outputs = 0;
return n_outputs_max;
```

#### [`ggml_backend_cpu_buffer_type_alloc_buffer`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1950)

```c++
buf_output.reset(ggml_backend_buft_alloc_buffer(buft, new_size));
----------
static ggml_backend_buffer_t ggml_backend_cpu_buffer_type_alloc_buffer(ggml_backend_buffer_type_t buft, size_t size) {
    void * data = ggml_aligned_malloc(size);
        const int alignment = 64;
        void * aligned_memory = NULL;
        int result = posix_memalign(&aligned_memory, alignment, size);
        return aligned_memory;
    return ggml_backend_buffer_init(buft, ggml_backend_cpu_buffer_i, data, size);
}
```

#### [`ggml_backend_cuda_host_buffer_type_alloc_buffer`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L1123)

```c++
buf_output.reset(ggml_backend_buft_alloc_buffer(buft, new_size));
----------
static ggml_backend_buffer_t ggml_backend_cuda_host_buffer_type_alloc_buffer(ggml_backend_buffer_type_t buft, size_t size) {
    --> void * ptr = ggml_cuda_host_malloc(size);
        void * ptr = nullptr;
        cudaError_t err = cudaMallocHost((void **) &ptr, size);
        return ptr;
    ggml_backend_buffer_t buffer = ggml_backend_cpu_buffer_from_ptr(ptr, size);
    buffer->buft = buft;
    buffer->iface.free_buffer = ggml_backend_cuda_host_buffer_free_buffer;
    return buffer;
}
```

#### [`ggml_backend_cpu_buffer_get_base`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1869)

```c++
float * output_base = (float *) ggml_backend_buffer_get_base(buf_output.get());
----------
void * ggml_backend_buffer_get_base(ggml_backend_buffer_t buffer) {
    --> return buffer->iface.get_base(buffer);
}

static void * ggml_backend_cpu_buffer_get_base(ggml_backend_buffer_t buffer) {
    uintptr_t data = (uintptr_t)buffer->context;
    // align the buffer
    if (data % TENSOR_ALIGNMENT != 0) data = GGML_PAD(data, TENSOR_ALIGNMENT);
    return (void *)data;
}
```

### [`llama_kv_cache_unified::llama_kv_cache_unified`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L19)

[`llama_model::create_memory`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L13751)

```c++
memory.reset(model.create_memory(params_mem, cparams));
----------
llama_memory_i * llama_model::create_memory(const llama_memory_params & params, llama_cparams & cparams) const {
    llama_memory_i * res;
    switch (arch) {
        const auto padding = llama_kv_cache_unified::get_padding(cparams);
            --> // the FA kernels require padding to avoid extra runtime boundary checks
            return cparams.flash_attn ? 256u : 32u;
        cparams.n_ctx = GGML_PAD(cparams.n_ctx, padding);
        --> res = new llama_kv_cache_unified(
                            *this,
                            nullptr,
                            params.type_k,
                            params.type_v,
                            !cparams.flash_attn,
                            cparams.offload_kqv,
                            kv_size=cparams.n_ctx,
                            cparams.n_seq_max,
                            padding,
                            hparams.n_swa,
                            hparams.swa_type);
    }
}

llama_kv_cache_unified::llama_kv_cache_unified(
        const llama_model &  model,
          layer_filter_cb && filter,
                ggml_type    type_k,
                ggml_type    type_v,
                     bool    v_trans,
                     bool    offload,
                 uint32_t    kv_size,
                 uint32_t    n_seq_max,
                 uint32_t    n_pad,
                 uint32_t    n_swa,
           llama_swa_type    swa_type) :
    model(model), hparams(model.hparams), v_trans(v_trans),
    n_seq_max(n_seq_max), n_pad(n_pad), n_swa(n_swa), swa_type(swa_type) {}
```

<details>
<summary>class llama_kv_cache_unified</summary>

```c++
class llama_kv_cache_unified : public llama_memory_i {
    using ubatch_heads = std::vector<uint32_t>;
private:
    const llama_model & model;
    const llama_hparams & hparams;

    struct kv_layer {
        // layer index in the model
        // note: can be different from the layer index in the KV cache
        uint32_t il;

        ggml_tensor * k;
        ggml_tensor * v;
    };

    bool v_trans = true;  // the value tensor is transposed

    // the current index from where we start searching for a free slot in the ring buffer of KV cells (see find_slot())
    // note: this is not part of the KV state and it's only used to speed-up the find_slot() method
    uint32_t head = 0;

    const uint32_t n_seq_max = 1;

    // required padding

    const llama_swa_type swa_type = LLAMA_SWA_TYPE_NONE;

    std::vector<ggml_context_ptr>        ctxs;
    std::vector<ggml_backend_buffer_ptr> bufs;

    llama_kv_cells_unified cells;

    std::vector<kv_layer> layers;

    // model layer id -> KV cache layer id
    std::unordered_map<int32_t, int32_t> map_layer_ids;
};
```

</details>


<details>
<summary>class llama_kv_cells_unified</summary>

```c++
class llama_kv_cells_unified {
private:
    bool has_shift = false;

    // set of indices of used cells (i.e. pos[i] != -1, allowed to not have any seq_id)
    std::set<uint32_t> used;

    std::vector<llama_pos> pos;

    // this array accumulates any applied shifts to the pos array since the last reset_shift() call
    // this is used to queue multiple updates to the pos array, which in the end can be applied in one go:
    //
    //   cells.pos_add(x, shift_x);
    //   cells.pos_div(y, shift_y);
    //   ...
    //
    //   if (cells.has_shift()) {
    //      for (int i = 0; i < n; ++i) {
    //          auto shift_i = cells.get_shift(i);
    //          ...
    //      }
    //      cells.reset_shift();
    //   }
    //
    std::vector<llama_pos> shift;

    using bits_t = std::bitset<LLAMA_MAX_SEQ>;

    // the bitset seq[i] tells us which sequences are currently occupying the i-th cell
    std::vector<bits_t> seq;

    // the set seq_pos[s] tells us which positions are currently present for sequence s
    // this way seq_pos[s].begin() and seq_pos[s].rbegin() give us the min/max positions currently in the cache
    std::set<llama_pos> seq_pos[LLAMA_MAX_SEQ];
};

    void reset() {
        for (uint32_t i = 0; i < pos.size(); ++i) {
            pos[i]   = -1;
            shift[i] =  0;
            seq[i].reset();
        }

        has_shift = false;

        used.clear();

        for (uint32_t s = 0; s < LLAMA_MAX_SEQ; ++s) {
            seq_pos[s].clear();
        }
    }

    void resize(uint32_t n) {
        pos.resize(n);
        shift.resize(n);
        seq.resize(n);

        reset();
    }

// copy the state of cells [i, i + n) (used for save/restore the state of the cells)
llama_kv_cells_unified cp(uint32_t i, uint32_t n) const {
    llama_kv_cells_unified res;
    res.resize(n);
    for (uint32_t j = 0; j < n; ++j) {
        res.pos[j] = pos[i + j];
        res.seq[j] = seq[i + j];
    }
    return res;
}

// set the state of cells [i, i + other.pos.size()) (used for save/restore the state of the cells)
void set(uint32_t i, const llama_kv_cells_unified & other) {
    for (uint32_t j = 0; j < other.pos.size(); ++j) {
        if (pos[i + j] == -1 && other.pos[j] != -1) {
            used.insert(i + j);
        }

        if (pos[i + j] != -1 && other.pos[j] == -1) {
            used.erase(i + j);
        }

        if (pos[i + j] != -1) {
            seq_pos_rm(i + j);
        }

        pos[i + j] = other.pos[j];
        seq[i + j] = other.seq[j];

        if (pos[i + j] != -1) {
            seq_pos_add(i + j);
        }
    }
}

    // remove cell i
    void seq_pos_rm(uint32_t i) {
        for (int s = 0; s < LLAMA_MAX_SEQ; ++s) {
            if (seq[i].test(s)) {
                seq_pos[s].erase(pos[i]);
            }
        }
    }

    // add cell i
    void seq_pos_add(uint32_t i) {
        for (int s = 0; s < LLAMA_MAX_SEQ; ++s) {
            if (seq[i].test(s)) {
                seq_pos[s].insert(pos[i]);
            }
        }
    }

    // set the position of an empty cell
    // does not modify "has_shift"
    // note: call only if the cell is empty
    void pos_set(uint32_t i, llama_pos p) {
        pos[i] = p;
        used.insert(i);
    }

    // note: call only if the cell is not empty and the seq_id is not in the cell
    void seq_add(uint32_t i, llama_seq_id seq_id) {
        seq[i].set(seq_id);
        seq_pos[seq_id].insert(pos[i]);
    }

```

</details>

```c++
// create a context for each buffer type
std::map<ggml_backend_buffer_type_t, ggml_context *> ctx_map;
auto ctx_for_buft = [&](ggml_backend_buffer_type_t buft) -> ggml_context * {
    auto it = ctx_map.find(buft);
    if (it == ctx_map.end()) {
        ggml_init_params params = {
            /*.mem_size   =*/ size_t(2u*hparams.n_layer*ggml_tensor_overhead()),
            /*.mem_buffer =*/ NULL,
            /*.no_alloc   =*/ true,
        };
        ggml_context * ctx = ggml_init(params);
        ctx_map[buft] = ctx;
        ctxs.emplace_back(ctx); //上下文的信息就写在这里了啊 *key*
        return ctx;
    }
    return it->second;
};

head = 0;
// llama_kv_cache_unified:: llama_kv_cells_unified cells;
cells.resize(kv_size);
for (uint32_t il = 0; il < hparams.n_layer; il++)
    const uint32_t n_embd_k_gqa = hparams.n_embd_k_gqa(il) + hparams.n_embd_k_s();
    const uint32_t n_embd_v_gqa = hparams.n_embd_v_gqa(il) + hparams.n_embd_v_s();
    auto * dev = model.dev_layer(il);
    buft = ggml_backend_dev_buffer_type(dev);
    dev_name = ggml_backend_dev_name(dev);
    ggml_context * ctx = ctx_for_buft(buft);
    ggml_tensor * k = ggml_new_tensor_2d(ctx, type_k, n_embd_k_gqa, kv_size);
    ggml_tensor * v = ggml_new_tensor_2d(ctx, type_v, n_embd_v_gqa, kv_size);
    ggml_format_name(k, "cache_k_l%d", il);
    ggml_format_name(v, "cache_v_l%d", il);
    // model layer id -> KV cache layer id
    // llama_kv_cache_unified:: std::unordered_map<int32_t, int32_t> map_layer_ids;
    map_layer_ids[il] = layers.size();
    // llama_kv_cache_unified:: std::vector<kv_layer> layers;
    layers.push_back({ il, k, v });

// allocate tensors and initialize the buffers to avoid NaNs in the padding
for (auto it : ctx_map) {
    auto * buft = ctx_map[0].first;
    auto * ctx  = ctx_map[0].second;
    --> ggml_backend_alloc_ctx_tensors_from_buft -> alloc_tensor_range --> ggml_backend_buffer_t buf;
    --> ggml_backend_buffer_clear(buf, 0);
        --> static void ggml_backend_cpu_buffer_clear(ggml_backend_buffer_t buffer, uint8_t value)
            memset(buffer->context, value, buffer->size);
        --> static void ggml_backend_cuda_buffer_clear(ggml_backend_buffer_t buffer, uint8_t value)
            ggml_backend_cuda_buffer_context * ctx = (ggml_backend_cuda_buffer_context *)buffer->context;
            ggml_cuda_set_device(ctx->device);
            cudaMemsetAsync(ctx->dev_ptr, value, buffer->size, cudaStreamPerThread);
            cudaStreamSynchronize(cudaStreamPerThread);

    llama_kv_cache_unified:: std::vector<ggml_backend_buffer_ptr> bufs.emplace_back(buf);
}
```

### [`ggml_graph_nbytes`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L5961)

```c++
buf_compute_meta.resize(ggml_tensor_overhead()*max_nodes + ggml_graph_overhead_custom(max_nodes, false));
----------
size_t ggml_graph_overhead_custom(size_t size, bool grads) {
    --> return GGML_OBJECT_SIZE + GGML_PAD(ggml_graph_nbytes(size, grads), GGML_MEM_ALIGN);
}

static size_t ggml_graph_nbytes(size_t size, bool grads) {}
```

```c++
size_t hash_size = ggml_hash_size(size * 2);
void * p = 0;
incr_ptr_aligned(&p, sizeof(struct ggml_cgraph), 1);
incr_ptr_aligned(&p, size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)); // nodes
incr_ptr_aligned(&p, size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)); // leafs
incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)); // hash keys
incr_ptr_aligned(&p, ggml_bitset_size(hash_size) * sizeof(ggml_bitset_t), sizeof(ggml_bitset_t));

size_t nbytes = (size_t) p;
return nbytes;
```

### [`ggml_backend_sched_new`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1455)

```c++
sched.reset(ggml_backend_sched_new(backend_ptrs.data(), backend_buft.data(), backend_ptrs.size(), max_nodes, pipeline_parallel, cparams.op_offload));
----------
ggml_backend_sched_t ggml_backend_sched_new(
        ggml_backend_t * backends,
        ggml_backend_buffer_type_t * bufts,
        int n_backends,
        size_t graph_size,
        bool parallel,
        bool op_offload) {}
```

<details>
<summary>struct ggml_hash_set</summary>

```c++
struct ggml_hash_set {
    size_t size;
    ggml_bitset_t * used;       // whether or not the keys are in use i.e. set
    struct ggml_tensor ** keys; // actual tensors in the set, keys[i] is only defined if ggml_bitset_get(used, i)
};

static size_t ggml_hash_insert(struct ggml_hash_set * hash_set, struct ggml_tensor * key) {
    size_t h = ggml_hash(key) % hash_set->size;

    // linear probing
    size_t i = h;
    do {
        if (!ggml_bitset_get(hash_set->used, i)) {
            ggml_bitset_set(hash_set->used, i);
            hash_set->keys[i] = key;
            return i;
        }
        if (hash_set->keys[i] == key) {
            return GGML_HASHSET_ALREADY_EXISTS;
        }
        i = (i + 1) % hash_set->size;
    } while (i != h);

    // visited all hash table entries -> not found
    GGML_ABORT("fatal error");
}

static size_t ggml_hash_find_or_insert(struct ggml_hash_set * hash_set, struct ggml_tensor * key) {
    ...
        if (hash_set->keys[i] == key) {
            return i;
        }
    ...
}

struct ggml_hash_set ggml_hash_set_new(size_t size) {
    size = ggml_hash_size(size);
    struct ggml_hash_set result;
    result.size = size;
    result.keys = GGML_MALLOC(sizeof(struct ggml_tensor *) * size);
    result.used = GGML_CALLOC(ggml_bitset_size(size), sizeof(ggml_bitset_t));
    return result;
}

void ggml_hash_set_free(struct ggml_hash_set * hash_set) {
    GGML_FREE(hash_set->used);
    GGML_FREE(hash_set->keys);
}

void ggml_hash_set_reset(struct ggml_hash_set * hash_set) {
    memset(hash_set->used, 0, sizeof(ggml_bitset_t) * ggml_bitset_size(hash_set->size));
}

static struct hash_node * ggml_gallocr_hash_get(ggml_gallocr_t galloc, struct ggml_tensor * t) {
    size_t i = ggml_hash_find_or_insert(&galloc->hash_set, t);
    return &galloc->hash_values[i];
}
```

</details>

<details>
<summary>struct ggml_backend_sched/ggml_backend_sched_t</summary>

```c++
struct ggml_backend_sched {
    bool is_reset; // true if the scheduler has been reset since the last graph split
    bool is_alloc;

    int n_backends;

    ggml_backend_t backends[GGML_SCHED_MAX_BACKENDS];
    ggml_backend_buffer_type_t bufts[GGML_SCHED_MAX_BACKENDS];
    ggml_gallocr_t galloc;

    // hash map of the nodes in the graph
    struct ggml_hash_set  hash_set;
    int                 * hv_tensor_backend_ids; // [hash_set.size]
    struct ggml_tensor ** hv_tensor_copies;      // [hash_set.size][n_backends][n_copies]

    int * node_backend_ids; // [graph_size]
    int * leaf_backend_ids; // [graph_size]

    int * prev_node_backend_ids; // [graph_size]
    int * prev_leaf_backend_ids; // [graph_size]

    // copy of the graph with modified inputs
    struct ggml_cgraph graph;

    // graph splits
    struct ggml_backend_sched_split * splits;
    int n_splits;
    int splits_capacity;

    // pipeline parallelism support
    int n_copies;
    int cur_copy;
    ggml_backend_event_t events[GGML_SCHED_MAX_BACKENDS][GGML_SCHED_MAX_COPIES];
    struct ggml_tensor * graph_inputs[GGML_SCHED_MAX_SPLIT_INPUTS];
    int n_graph_inputs;

    struct ggml_context * ctx;

    ggml_backend_sched_eval_callback callback_eval;
    void * callback_eval_user_data;

    char * context_buffer;
    size_t context_buffer_size;

    bool op_offload;

    int debug;
};

typedef struct ggml_backend_sched * ggml_backend_sched_t;
```

</details>

```c++
struct ggml_backend_sched * sched = (ggml_backend_sched *) calloc(1, sizeof(struct ggml_backend_sched));
sched->n_backends = n_backends;
sched->n_copies = parallel ? GGML_SCHED_MAX_COPIES : 1;
// initialize hash table
// FIXME: needs to be size*2 to account for leafs (do it in graph_split instead)
sched->hash_set    = ggml_hash_set_new(graph_size);
sched->hv_tensor_backend_ids = (int *) malloc(sched->hash_set.size * sizeof(sched->hv_tensor_backend_ids[0]));
sched->hv_tensor_copies      = (ggml_tensor **) malloc(sched->hash_set.size * sched->n_backends * sched->n_copies * sizeof(struct ggml_tensor *));
const size_t ggml_sched_max_splits = graph_size; // at most there is one split for each node in the graph
const size_t nodes_size = graph_size + ggml_sched_max_splits*GGML_SCHED_MAX_SPLIT_INPUTS*2;
sched->node_backend_ids = (int *) calloc(nodes_size, sizeof(sched->node_backend_ids[0]));
sched->leaf_backend_ids = (int *) calloc(nodes_size, sizeof(sched->leaf_backend_ids[0]));
sched->prev_node_backend_ids = (int *) calloc(nodes_size, sizeof(sched->prev_node_backend_ids[0]));
sched->prev_leaf_backend_ids = (int *) calloc(nodes_size, sizeof(sched->prev_leaf_backend_ids[0]));

sched->context_buffer_size = ggml_sched_max_splits*GGML_SCHED_MAX_SPLIT_INPUTS*2*sizeof(struct ggml_tensor) + ggml_graph_overhead_custom(graph_size, false);
sched->context_buffer = (char *) malloc(sched->context_buffer_size);

const int initial_splits_capacity = 16;
sched->splits = (ggml_backend_sched_split *) calloc(initial_splits_capacity, sizeof(sched->splits[0]));
sched->splits_capacity = initial_splits_capacity;
for (int b = 0; b < n_backends; b++)
    sched->backends[b] = backends[b];
    sched->bufts[b] = bufts[b];
--> ggml_gallocr_new_n --> sched->galloc;
sched->op_offload = op_offload;
ggml_backend_sched_reset(sched);
    --> // reset state for the next run
    //if (!sched->is_reset)
    ggml_hash_set_reset(&sched->hash_set);
    memset(sched->hv_tensor_backend_ids, -1, sched->hash_set.size * sizeof(sched->hv_tensor_backend_ids[0]));
    memset(sched->hv_tensor_copies,       0, sched->hash_set.size * sched->n_backends * sched->n_copies * sizeof(struct ggml_tensor *));
    sched->is_reset = true;
    sched->is_alloc = false;
return sched;
```

#### [`ggml_gallocr_new_n`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L380)

```c++
// Graph allocator
/*
  Example usage:
    ggml_gallocr_t galloc = ggml_gallocr_new(ggml_backend_cpu_buffer_type());

    // optional: create a worst-case graph and reserve the buffers to avoid reallocations
    ggml_gallocr_reserve(galloc, build_graph(max_batch));

    // allocate the graph
    struct ggml_cgraph * graph = build_graph(batch);
    ggml_gallocr_alloc_graph(galloc, graph);

    printf("compute buffer size: %zu bytes\n", ggml_gallocr_get_buffer_size(galloc, 0));

    // evaluate the graph
    ggml_backend_graph_compute(backend, graph);
*/

// special tensor flags for use with the graph allocator:
//   ggml_set_input(): all input tensors are allocated at the beginning of the graph in non-overlapping addresses
//   ggml_set_output(): output tensors are never freed and never overwritten
```

<details>
<summary>struct ggml_gallocr/ggml_gallocr_t</summary>

```c++
struct ggml_gallocr {
    ggml_backend_buffer_type_t * bufts; // [n_buffers]
    ggml_backend_buffer_t * buffers; // [n_buffers]
    struct ggml_dyn_tallocr ** buf_tallocs; // [n_buffers]
    int n_buffers;

    struct ggml_hash_set hash_set;
    struct hash_node * hash_values; // [hash_set.size]

    struct node_alloc * node_allocs; // [n_nodes]
    int n_nodes;

    struct leaf_alloc * leaf_allocs; // [n_leafs]
    int n_leafs;
};


typedef struct ggml_gallocr * ggml_gallocr_t;
```

</details>


```c++
sched->galloc = ggml_gallocr_new_n(sched->bufts, n_backends);
----------
ggml_gallocr_t ggml_gallocr_new_n(ggml_backend_buffer_type_t * bufts, int n_bufs) {
    galloc->bufts = calloc(n_bufs, sizeof(ggml_backend_buffer_type_t));
    galloc->buffers = calloc(n_bufs, sizeof(ggml_backend_buffer_t));
    galloc->buf_tallocs = calloc(n_bufs, sizeof(struct ggml_dyn_tallocr *));
    for (int i = 0; i < n_bufs; i++)
        galloc->bufts[i] = bufts[i];
        galloc->buffers[i] = NULL;
        --> ggml_dyn_tallocr_new(ggml_backend_buft_get_alignment(bufts[i])) --> galloc->buf_tallocs[i];
    galloc->n_buffers = n_bufs;
    return galloc;
}
```

[`ggml_dyn_tallocr_new`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L310)

<details>
<summary>struct ggml_dyn_tallocr</summary>

```c++
struct ggml_dyn_tallocr {
    size_t alignment;
    int n_free_blocks;
    struct free_block free_blocks[MAX_FREE_BLOCKS];
    size_t max_size;
};
```

</details>

```c++
galloc->buf_tallocs[i] = ggml_dyn_tallocr_new(alignment);
----------
static struct ggml_dyn_tallocr * ggml_dyn_tallocr_new(size_t alignment) {
    struct ggml_dyn_tallocr * alloc = (struct ggml_dyn_tallocr *)malloc(sizeof(struct ggml_dyn_tallocr));

    *alloc = (struct ggml_dyn_tallocr) {
        /*.alignment     = */ alignment,
        /*.n_free_blocks = */ 0,
        /*.free_blocks   = */ {{0}},
        /*.max_size      = */ 0,
    };

    ggml_dyn_tallocr_reset(alloc);
        alloc->n_free_blocks = 1;
        alloc->free_blocks[0].offset = 0;
        alloc->free_blocks[0].size = SIZE_MAX/2; // restrict maximum size of a measure allocator to half size_t max to avoid overflows
        alloc->max_size = 0;
    return alloc;
}
```

### [`llama_kv_cache_unified_context::llama_kv_cache_unified_context`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L1718)

<details>
<summary>class llama_kv_cache_unified_context</summary>

```c++
class llama_kv_cache_unified_context : public llama_memory_context_i {
public:
    // some shorthands
    using ubatch_heads = llama_kv_cache_unified::ubatch_heads;
    using defrag_info  = llama_kv_cache_unified::defrag_info;

    // used for errors
    llama_kv_cache_unified_context(llama_memory_status status);

    // used to create a full-cache context
    llama_kv_cache_unified_context(
            llama_kv_cache_unified * kv);

    // used to create an update context
    llama_kv_cache_unified_context(
            llama_kv_cache_unified * kv,
            llama_context * lctx,
            bool do_shift,
            defrag_info dinfo);

    // used to create a batch procesing context from a batch
    llama_kv_cache_unified_context(
            llama_kv_cache_unified * kv,
            ubatch_heads heads,
            std::vector<llama_ubatch> ubatches);

    virtual ~llama_kv_cache_unified_context();

    //
    // llama_memory_context_i
    //

    bool next()  override;
    bool apply() override;

    llama_memory_status  get_status() const override;
    const llama_ubatch & get_ubatch() const override;

    //
    // llama_kv_cache_unified_context specific API
    //

    uint32_t get_n_kv() const;

    // get views of the current state of the cache
    ggml_tensor * get_k(ggml_context * ctx, int32_t il) const;
    ggml_tensor * get_v(ggml_context * ctx, int32_t il) const;

    // store k_cur and v_cur in the cache based on the provided head location
    ggml_tensor * cpy_k(ggml_context * ctx, ggml_tensor * k_cur, int32_t il) const;
    ggml_tensor * cpy_v(ggml_context * ctx, ggml_tensor * v_cur, int32_t il) const;

    void set_input_k_shift(ggml_tensor * dst) const;

    void set_input_kq_mask   (ggml_tensor * dst, const llama_ubatch * ubatch, bool causal_attn) const;
    void set_input_pos_bucket(ggml_tensor * dst, const llama_ubatch * ubatch) const;

private:
    llama_memory_status status;

    llama_kv_cache_unified * kv;
    llama_context * lctx;

    //
    // update context
    //

    bool do_shift = false;

    defrag_info dinfo;

    //
    // batch processing context
    //

    // the index of the next ubatch to process
    size_t i_next = 0;

    ubatch_heads heads;

    std::vector<llama_ubatch> ubatches;

    //
    // data needed for building the compute graph for the current ubatch:
    //

    // a heuristic, to avoid attending the full cache if it is not yet utilized
    // as the cache gets filled, the benefit from this heuristic disappears
    int32_t n_kv;

    // the beginning of the current slot in which the ubatch will be inserted
    int32_t head;
};
```

</details>

```c++
const auto mstate = memory->init_full();
----------
llama_memory_state_ptr llama_kv_cache_unified::init_full() {
    --> return std::make_unique<llama_kv_cache_unified_context>(this);
}

llama_kv_cache_unified_context::llama_kv_cache_unified_context(
        llama_kv_cache_unified * kv) : status(LLAMA_MEMORY_STATUS_SUCCESS), kv(kv) {
    n_kv = kv->get_size();
    head = 0;
}      
```

### [`llama_context::graph_reserve`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1295)

```c++
// reserve pp graph first so that buffers are only allocated once
auto * gf = graph_reserve(n_tokens, n_seqs, n_tokens, mctx.get());
// reserve with tg graph to get the number of splits and nodes
auto * gf = graph_reserve(1, 1, 1, mctx.get());
// reserve again with pp graph to avoid ggml-alloc reallocations during inference
auto * gf = graph_reserve(n_tokens, n_seqs, n_tokens, mctx.get());
----------
ggml_cgraph * llama_context::graph_reserve(uint32_t n_tokens, uint32_t n_seqs, uint32_t n_outputs, const llama_memory_context_i * mctx) {}
```

```c++
this->n_outputs = n_outputs;
--> llama_batch_allocr::llama_batch_allocr --> llama_batch_allocr balloc;
--> llama_batch_allocr::ubatch_reserve --> llama_ubatch ubatch;
--> llama_context::graph_init --> ggml_cgraph * gf;
--> llama_context::graph_build --> llama_model::build_graph --> llm_build_llama::llm_build_llama --> std::unique_ptr<llm_graph_result> res;
ggml_backend_sched_reset(sched.get());
// initialize scheduler with the specified graph
--> ggml_backend_sched_reserve;
return gf;
```

#### [`llama_batch_allocr::llama_batch_allocr`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L12)

<details>
<summary>class llama_batch_allocr</summary>

```c++
// a helper for sanitizing, fulfilling and splitting a batch
class llama_batch_allocr {
    llama_batch batch;

    // only for debugging purposes
    const llama_vocab * vocab;

    // TODO: this is more of a temporary solution until we have a better way to handle multiple positions per token/embd
    //       ref: https://github.com/ggml-org/llama.cpp/issues/13694#issuecomment-2983871762
    const uint32_t n_pos_per_embd;

    uint32_t n_embd;
    uint32_t n_outputs;

    std::array<llama_seq_id, 1> seq_id_0 = { 0 }; // default sequence id

    std::vector<llama_pos>      pos;
    std::vector<int32_t>        n_seq_id;
    std::vector<llama_seq_id *> seq_id;
    std::vector<llama_seq_id>   seq_id_unq;
    std::vector<int32_t>        seq_idx;
    std::vector<int8_t>         output;

    using pos_set_t = std::set<llama_pos>;
    using seq_cpl_t = std::vector<bool>;

    std::vector<pos_set_t> seq_pos; // seq_pos[s]: the set of positions in sequence s
    std::vector<seq_cpl_t> seq_cpl; // seq_cpl[s0][s1]: if sequence s0 is coupled to sequence s1

    using idx_vec_t = std::vector<int32_t>;
    using seq_set_t = std::bitset<LLAMA_MAX_SEQ>;

    std::vector<seq_set_t> seq_set; // seq_set[i]: the sequence set of token i

    std::unordered_map<seq_set_t, idx_vec_t> seq_set_map; // the indices at which the sequence set appears

    // batch indices of the output
    std::vector<int32_t> out_ids;

    // used[i] indicates if token i has already been used in a previous ubatch
    std::vector<bool> used;

    // llama_ubatch points to this data:
    struct ubatch {
        std::vector<llama_token>    token;
        std::vector<float>          embd;
        std::vector<llama_pos>      pos;
        std::vector<int32_t>        n_seq_id;
        std::vector<llama_seq_id *> seq_id;
        std::vector<llama_seq_id>   seq_id_unq;
        std::vector<int32_t>        seq_idx;
        std::vector<int8_t>         output;
    };

    // current splitting state:
    std::vector<ubatch> ubatches;

    int debug;
};
```

</details>

```c++
llama_batch_allocr balloc(model.hparams.n_pos_per_embd());
----------
llama_batch_allocr::llama_batch_allocr(uint32_t n_pos_per_embd) : n_pos_per_embd(n_pos_per_embd) {
    const char * LLAMA_BATCH_DEBUG = getenv("LLAMA_BATCH_DEBUG");
    debug = LLAMA_BATCH_DEBUG ? atoi(LLAMA_BATCH_DEBUG) : 0;
    seq_pos.resize(LLAMA_MAX_SEQ);
    seq_cpl.resize(LLAMA_MAX_SEQ);
    for (auto & cur : seq_cpl) cur.resize(LLAMA_MAX_SEQ);
    seq_idx.resize(LLAMA_MAX_SEQ, -1);
}
```

#### [`lama_batch_allocr::ubatch_reserve`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L339)

```c++
llama_ubatch ubatch = balloc.ubatch_reserve(n_tokens/n_seqs, n_seqs);
----------
llama_ubatch llama_batch_allocr::ubatch_reserve(uint32_t n_seq_tokens, uint32_t n_seqs) {}
```

<details>
<summary>struct llama_ubatch</summary>

```c++
// keep this struct lightweight
// it points to data in `llama_batch_allocr`
struct llama_ubatch {
    bool equal_seqs;
    // TODO: whole_seqs for embeddings?
    uint32_t n_tokens;     // total tokens (n_seq_tokens * n_seqs)
    uint32_t n_seq_tokens; // tokens per sequence set
    uint32_t n_seqs;       // sequence sets in the ubatch
    uint32_t n_seqs_unq;   // unique sequence ids in the ubatch
    // seq_id_unq: unique sequence ids in the ubatch
    // seq_idx:    indices of the unique sequence ids in the ubatch in [0, n_seqs_unq)
    //             used for extracting sequence pooled embeddings
    //                          // size               | idx | val
    llama_token  *  token;      // [n_tokens]         | i   | id, token
    float        *  embd;       // [n_embd, n_tokens] | i   | embd
    llama_pos    *  pos;        // [n_tokens]         | i   | pos
    int32_t      *  n_seq_id;   // [n_tokens]         | i   | -
    llama_seq_id ** seq_id;     // [n_tokens]         | s   | s0, s1, seq_id
    llama_seq_id *  seq_id_unq; // [n_seqs_unq]       | s   | seq_id
    int32_t      *  seq_idx;    // [LLAMA_MAX_SEQ]    | -   | seq_idx
    int8_t       *  output;     // [n_tokens]         | i   | -
};
```

</details>

```c++
const uint32_t n_tokens = n_seq_tokens*n_seqs;
--> clear();
--> split_reset();
ubatches.emplace_back();
auto & ubatch = ubatches.back();
ubatch.token     .resize(n_tokens);
ubatch.embd      .clear();
ubatch.pos       .resize(n_tokens);
ubatch.n_seq_id  .resize(n_tokens);
ubatch.seq_id    .resize(n_tokens);
ubatch.seq_id_unq.resize(0);
ubatch.seq_idx   .resize(LLAMA_MAX_SEQ, -1);
ubatch.output    .resize(n_tokens);
// for (uint32_t s = 0; s < n_seqs; ++s)
ubatch.seq_idx[0] = 0;
ubatch.seq_id_unq.push_back(0);
llama_ubatch res {
    /*.equal_seqs   =*/ true,
    /*.n_tokens     =*/ n_tokens,
    /*.n_seq_tokens =*/ n_seq_tokens,
    /*.n_seqs       =*/ n_seqs,
    /*.n_seqs_unq   =*/ n_seqs,
    /*.token        =*/ ubatch.token.data(),
    /*.embd         =*/ nullptr,
    /*.pos          =*/ ubatch.pos.data(),
    /*.n_seq_id     =*/ ubatch.n_seq_id.data(),
    /*.seq_id       =*/ ubatch.seq_id.data(),
    /*.seq_id_unq   =*/ ubatch.seq_id_unq.data(),
    /*.seq_idx      =*/ ubatch.seq_idx.data(),
    /*.output       =*/ ubatch.output.data(),
};
return res;
```

[`llama_batch_allocr::clear`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L579)

```c++
void llama_batch_allocr::clear() {
    n_outputs = 0;
    batch = {};
    pos       .clear();
    n_seq_id  .clear();
    seq_id    .clear();
    seq_id_unq.clear();
    output    .clear();
    for (auto & cur : seq_pos) cur.clear();
    for (auto & cur : seq_cpl) std::fill(cur.begin(), cur.end(), false);
    seq_set.clear();
    seq_set_map.clear();
    std::fill(seq_idx.begin(), seq_idx.end(), -1);
}
```

#### [`llama_context::graph_init`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1319)

> ctx_compute 存放的是原始的静态计算图

```c++
auto * gf = graph_init();
----------
ggml_cgraph * llama_context::graph_init() {}
```

<details>
<summary>struct ggml_cgraph</summary>

```c++
struct ggml_cgraph {
    int size;    // maximum number of nodes/leafs/grads/grad_accs
    int n_nodes; // number of nodes currently in use
    int n_leafs; // number of leafs currently in use

    struct ggml_tensor ** nodes;     // tensors with data that can change if the graph is evaluated
    struct ggml_tensor ** grads;     // the outputs of these tensors are the gradients of the nodes
    struct ggml_tensor ** grad_accs; // accumulators for node gradients
    struct ggml_tensor ** leafs;     // tensors with constant data

    struct ggml_hash_set visited_hash_set;

    enum ggml_cgraph_eval_order order;
};
```

</details>

```c++
ggml_init_params params = {
    /*.mem_size   =*/ buf_compute_meta.size(),
    /*.mem_buffer =*/ buf_compute_meta.data(),
    /*.no_alloc   =*/ true,
};
llama_context:: ggml_context_ptr ctx_compute.reset(ggml_init(params));
--> return ggml_new_graph_custom;
```

[`ggml_new_graph_custom`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L5986)


```c++
return ggml_new_graph_custom(ctx_compute.get(), graph_max_nodes(), false);
----------
struct ggml_cgraph * ggml_new_graph_custom(struct ggml_context * ctx, size_t size, bool grads) {
    const size_t obj_size = ggml_graph_nbytes(size, grads);
    struct ggml_object * obj = ggml_new_object(ctx, GGML_OBJECT_TYPE_GRAPH, obj_size);
    struct ggml_cgraph * cgraph = (struct ggml_cgraph *) ((char *) ctx->mem_buffer + obj->offs);

    // the size of the hash table is doubled since it needs to hold both nodes and leafs
    size_t hash_size = ggml_hash_size(size * 2);

    void * p = cgraph + 1;

    struct ggml_tensor ** nodes_ptr     =         incr_ptr_aligned(&p, size      * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *));
    struct ggml_tensor ** leafs_ptr     =         incr_ptr_aligned(&p, size      * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *));
    struct ggml_tensor ** hash_keys_ptr =         incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *));
    struct ggml_tensor ** grads_ptr     = grads ? incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)) : NULL;
    struct ggml_tensor ** grad_accs_ptr = grads ? incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)) : NULL;

    ggml_bitset_t * hash_used = incr_ptr_aligned(&p, ggml_bitset_size(hash_size) * sizeof(ggml_bitset_t), sizeof(ggml_bitset_t));

    *cgraph = (struct ggml_cgraph) {
        /*.size         =*/ size,
        /*.n_nodes      =*/ 0,
        /*.n_leafs      =*/ 0,
        /*.nodes        =*/ nodes_ptr,
        /*.grads        =*/ grads_ptr,
        /*.grad_accs    =*/ grad_accs_ptr,
        /*.leafs        =*/ leafs_ptr,
        /*.hash_table   =*/ { hash_size, hash_used, hash_keys_ptr },
        /*.order        =*/ GGML_CGRAPH_EVAL_ORDER_LEFT_TO_RIGHT,
    };

    ggml_hash_set_reset(&cgraph->visited_hash_set);
    return cgraph;
}

```

#### [`llm_build_llama::llm_build_llama`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L4686)

> 这里开始真正构建模型的拓扑关系了，但是尚未给中间的激活tensor分配buffer和data域，所以是静态的计算图

> 注意到全程都没有用到sched的信息，就只用了 model(负责权重tensor的提供)，mctx(负责kvcache tensor的提供)， ctx_compute(记录最后生成的带有拓扑结构的静态图的tensors 信息) 

<details>
<summary>class llm_graph_input_i/llm_graph_input_ptr</summary>

```c++
class llm_graph_input_i {
public:
    virtual ~llm_graph_input_i() = default;
    virtual void set_input(const llama_ubatch * ubatch) = 0;
};
using llm_graph_input_ptr = std::unique_ptr<llm_graph_input_i>;
```

</details>

<details>
<summary>class llm_graph_result</summary>

```c++
//
// llm_graph_result
//

// these objects deliver the result from the graph build process back to the llama_context
// note that the input tensors created for the graph are referenced here - the goal is to be able to populate their
//   specific data, by calling the set_inputs() method
// along with the input tensors, the object also provides commonly used outputs tensors, such as logits, embeddings, etc.
//   these are used by the llama_context to extact the relevant data, based on the compute parameters

class llm_graph_result_i {
public:
    virtual ~llm_graph_result_i() = default;

    virtual ggml_tensor * get_tokens()      = 0;
    virtual ggml_tensor * get_logits()      = 0;
    virtual ggml_tensor * get_embd()        = 0;
    virtual ggml_tensor * get_embd_pooled() = 0;

    virtual void set_inputs(const llama_ubatch * ubatch) = 0;
};

using llm_graph_result_ptr = std::unique_ptr<llm_graph_result_i>;

class llm_graph_result : public llm_graph_result_i {
public:
    // important graph nodes
    ggml_tensor * t_tokens      = nullptr;
    ggml_tensor * t_logits      = nullptr;
    ggml_tensor * t_embd        = nullptr;
    ggml_tensor * t_embd_pooled = nullptr;

    std::vector<llm_graph_input_ptr> inputs;
}

    void set_inputs(const llama_ubatch * ubatch) override {
        for (auto & input : inputs) {
            input->set_input(ubatch);
        }
    }

    llm_graph_input_i * add_input(llm_graph_input_ptr input) {
        inputs.emplace_back(std::move(input));
        return inputs.back().get();
    }
```

</details>

<details>
<summary>struct llm_graph_params</summary>

```c++
struct llm_graph_params {
    ggml_context * ctx;

    const llm_arch arch;

    const llama_hparams & hparams;
    const llama_cparams & cparams;
    const llama_ubatch  & ubatch;

    ggml_backend_sched_t sched;
    ggml_backend_t backend_cpu;

    const llama_adapter_cvec     * cvec;
    const llama_adapter_loras    * loras;
    const llama_memory_context_i * mctx;
    const llama_cross            * cross;

    uint32_t n_outputs;

    const llm_graph_cb & cb;
};
```

</details>

<details>
<summary>struct llm_graph_context</summary>

```c++
struct llm_graph_context {
    const llm_arch arch;

    const llama_hparams & hparams;
    const llama_cparams & cparams;
    const llama_ubatch  & ubatch;

    const int64_t n_embd;
    const int64_t n_layer;
    const int64_t n_rot;
    const int64_t n_ctx;       // user-specified context size (can be different from n_ctx_train)
    const int64_t n_head;
    const int64_t n_head_kv;
    const int64_t n_embd_head_k;
    const int64_t n_embd_k_gqa;
    const int64_t n_embd_head_v;
    const int64_t n_embd_v_gqa;
    const int64_t n_expert;
    const int64_t n_expert_used;

    const float freq_base;
    const float freq_scale;
    const float ext_factor;
    const float attn_factor;
    const float beta_fast;
    const float beta_slow;
    const float norm_eps;
    const float norm_rms_eps;

    const int32_t n_tokens;
    const int32_t n_outputs;
    const int32_t n_ctx_orig; // yarn

    const enum llama_pooling_type pooling_type;
    const enum llama_rope_type    rope_type;

    ggml_context * ctx0 = nullptr;

    ggml_backend_sched_t sched;

    ggml_backend_t backend_cpu; // TODO: needed by build_attn_mha, figure out a way to remove?

    const llama_adapter_cvec   * cvec;
    const llama_adapter_loras  * loras;
    const llama_memory_state_i * mstate;
    const llama_cross          * cross;

    const llm_graph_cb & cb_func;

    std::unique_ptr<llm_graph_result> res;
};
```

</details>


```c++
auto res = graph_build(ctx_compute.get(), gf, ubatch, LLM_GRAPH_TYPE_DEFAULT, mctx);
----------
llm_graph_result_ptr llama_context::graph_build(
                    ggml_context * ctx,
                     ggml_cgraph * gf,
              const llama_ubatch & ubatch,
                  llm_graph_type   gtype,
      const llama_memory_context_i * mctx) {
      --> return model.build_graph(
            struct llm_graph_params = {
                /*.ctx         =*/ ctx,
                /*.arch        =*/ model.arch,
                /*.hparams     =*/ model.hparams,
                /*.cparams     =*/ cparams,
                /*.ubatch      =*/ ubatch,
                /*.sched       =*/ sched.get(),
                /*.backend_cpu =*/ backend_cpu,
                /*.cvec        =*/ &cvec,
                /*.loras       =*/ &loras,
                /*.mstate      =*/ mstate,
                /*.cross       =*/ &cross,
                /*.n_outputs   =*/ n_outputs,
                /*.cb          =*/ graph_get_cb(),
            }, gf, gtype);
}

llm_graph_result_ptr llama_model::build_graph(
        const llm_graph_params & params,
                   ggml_cgraph * gf,
                llm_graph_type   type) const {
    --> std::unique_ptr<llm_graph_context> llm = std::make_unique<llm_build_llama>(*this, params, gf);
    return std::move(llm->res);
}

struct llm_build_llama : public llm_graph_context {
    llm_build_llama(const llama_model & model, const llm_graph_params & params, ggml_cgraph * gf) :
    --> llm_graph_context(params) {}
        ctx0 (params.ctx),
        mctx (params.mctx),
};
```

```c++
const int64_t n_embd_head = hparams.n_embd_head_v;
ggml_tensor * cur;
ggml_tensor * inpL;
--> build_inp_embd --> res->add_input(inp), ggml_tensor * inpL = ggml_get_rows(ctx0, tok_embd, inp->tokens);;
// inp_pos - contains the positions
--> build_inp_pos --> res->add_input(inp), ggml_tensor * inp_pos = inp->pos;
--> build_attn_inp_kv_unified --> res->add_input(inp), auto * inp_attn = inp;
--> ggml_tensor * inp_out_ids = build_inp_out_ids();
const float kq_scale = hparams.f_attention_scale == 0.0f ? 1.0f/sqrtf(float(n_embd_head)) : hparams.f_attention_scale;
for (int il = 0; il < n_layer; ++il) {
    ggml_tensor * inpSA = inpL;
    // norm
    cur = build_norm(inpL,
            model.layers[il].attn_norm, NULL,
            LLM_NORM_RMS, il);
    cb(cur, "attn_norm", il);
    // self-attention
    {
        // rope freq factors for llama3; may return nullptr for llama2 and other models
        ggml_tensor * rope_factors = model.get_rope_factors(cparams, il);
        // compute Q and K and RoPE them
        ggml_tensor * Qcur = build_lora_mm(model.layers[il].wq, cur);
        cb(Qcur, "Qcur", il);
        ggml_tensor * Kcur = build_lora_mm(model.layers[il].wk, cur);
        cb(Kcur, "Kcur", il);
        ggml_tensor * Vcur = build_lora_mm(model.layers[il].wv, cur);
        cb(Vcur, "Vcur", il);
        Qcur = ggml_reshape_3d(ctx0, Qcur, n_embd_head, n_head,    n_tokens);
        Kcur = ggml_reshape_3d(ctx0, Kcur, n_embd_head, n_head_kv, n_tokens);
        Vcur = ggml_reshape_3d(ctx0, Vcur, n_embd_head, n_head_kv, n_tokens);
        Qcur = ggml_rope_ext(
                ctx0, Qcur, inp_pos, rope_factors,
                n_rot, rope_type, n_ctx_orig, freq_base, freq_scale,
                ext_factor, attn_factor, beta_fast, beta_slow
                );
        Kcur = ggml_rope_ext(
                ctx0, Kcur, inp_pos, rope_factors,
                n_rot, rope_type, n_ctx_orig, freq_base, freq_scale,
                ext_factor, attn_factor, beta_fast, beta_slow
                );
        cb(Qcur, "Qcur", il);
        cb(Kcur, "Kcur", il);
        cb(Vcur, "Vcur", il);
        --> cur = build_attn(inp_attn, gf,
                model.layers[il].wo, model.layers[il].bo,
                Qcur, Kcur, Vcur, nullptr, nullptr, kq_scale, il);
        cb(cur, "attn_out", il);
    }
    if (il == n_layer - 1) {
        // skip computing output for unused tokens
        cur   = ggml_get_rows(ctx0,   cur, inp_out_ids); // node_1144
        inpSA = ggml_get_rows(ctx0, inpSA, inp_out_ids); // node_1145
    }
    ggml_tensor * ffn_inp = ggml_add(ctx0, cur, inpSA);
    cb(ffn_inp, "ffn_inp", il);
    // feed-forward network (non-MoE)
    cur = build_norm(ffn_inp,
            model.layers[il].ffn_norm, NULL,
            LLM_NORM_RMS, il);
    cb(cur, "ffn_norm", il);
    cur = build_ffn(cur,
            model.layers[il].ffn_up,   model.layers[il].ffn_up_b,   NULL,
            model.layers[il].ffn_gate, model.layers[il].ffn_gate_b, NULL,
            model.layers[il].ffn_down, model.layers[il].ffn_down_b, NULL,
            NULL,
            LLM_FFN_SILU, LLM_FFN_PAR, il);
    cb(cur, "ffn_out", il);
    cur = ggml_add(ctx0, cur, ffn_inp);
    cb(cur, "ffn_out", il);
    cur = build_cvec(cur, il);
    cb(cur, "l_out", il);
    // input for next layer
    inpL = cur;
}
// cur = inpL;
cur = build_norm(cur,
        model.output_norm, NULL,
        LLM_NORM_RMS, -1);
cb(cur, "result_norm", -1);
res->t_embd = cur;
// lm_head
cur = build_lora_mm(model.output, cur);
cb(cur, "result_output", -1);
res->t_logits = cur;
--> ggml_build_forward_expand --> ggml_build_forward_impl --> ggml_visit_parents;
```

##### [`llm_graph_context::build_inp_embd`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L847)

```c++
inpL = build_inp_embd(model.tok_embd);
----------
// input embeddings with optional lora
ggml_tensor * llm_graph_context::build_inp_embd(ggml_tensor * tok_embd) const {}
```

<details>
<summary>class llm_graph_input_embd</summary>

```c++
class llm_graph_input_embd : public llm_graph_input_i {
public:
    ggml_tensor * tokens = nullptr; // I32 [n_batch]
    ggml_tensor * embd   = nullptr; // F32 [n_embd, n_batch]
};


void llm_graph_input_embd::set_input(const llama_ubatch * ubatch) {
    const int64_t n_tokens = ubatch->n_tokens;
    ggml_backend_tensor_set(tokens, ubatch->token, 0, n_tokens*ggml_element_size(tokens));
}
```

</details>

```c++
const int64_t n_embd = hparams.n_embd;
auto inp = std::make_unique<llm_graph_input_embd>();
inp->tokens = ggml_new_tensor_1d(ctx0, GGML_TYPE_I32, ubatch.n_tokens);
ggml_set_input(inp->tokens);
    tensor->flags |= GGML_TENSOR_FLAG_INPUT;
res->t_tokens = inp->tokens;
ggml_tensor * cur = ggml_get_rows(ctx0, tok_embd, inp->tokens);
cb(cur, "inp_embd", -1);
res->add_input(std::move(inp));
return cur;
```

##### [`llm_graph_context::build_inp_pos`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L898)

```c++
// inp_pos - contains the positions
ggml_tensor * inp_pos = build_inp_pos();
----------
ggml_tensor * llm_graph_context::build_inp_pos() const {}
```

<details>
<summary>class llm_graph_input_pos</summary>

```c++
class llm_graph_input_pos : public llm_graph_input_i {
public:
    ggml_tensor * pos = nullptr; // I32 [n_batch]
    const int64_t n_pos_per_embd = 1;
};

void llm_graph_input_pos::set_input(const llama_ubatch * ubatch) {
    // if (ubatch->pos && pos)
    const int64_t n_tokens = ubatch->n_tokens;
    ggml_backend_tensor_set(pos, ubatch->pos, 0, n_tokens*n_pos_per_embd*ggml_element_size(pos));
}
```

</details>

```c++
auto inp = std::make_unique<llm_graph_input_pos>(n_pos_per_embd());
auto & cur = inp->pos;
cur = ggml_new_tensor_1d(ctx0, GGML_TYPE_I32, n_tokens*n_pos_per_embd());
ggml_set_input(cur);
res->add_input(std::move(inp));
return cur;
```

##### [`llm_graph_context::build_attn_inp_kv_unified`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L1224)

```c++
auto * inp_attn = build_attn_inp_kv_unified();
----------
llm_graph_input_attn_kv_unified * llm_graph_context::build_attn_inp_kv_unified() const {}
```

<details>
<summary>llm_graph_input_attn_kv_unified</summary>

```c++
class llm_graph_input_attn_kv_unified : public llm_graph_input_i {
public:
    ggml_tensor * self_kq_mask     = nullptr; // F32 [n_kv, n_batch]
    ggml_tensor * self_kq_mask_cnv = nullptr; //     [n_kv, n_batch]

    const llama_hparams & hparams;
    const llama_cparams & cparams;

    const llama_kv_cache_unified_state * kv_state;
};

void llm_graph_input_attn_kv_unified::set_input(const llama_ubatch * ubatch) {
    if (self_kq_mask) {
        mctx->set_input_kq_mask(self_kq_mask, ubatch, cparams.causal_attn);
    }
}
```

</details>

```c++
const auto * mctx_cur = static_cast<const llama_kv_cache_unified_context *>(mctx);
auto inp = std::make_unique<llm_graph_input_attn_kv_unified>(hparams, cparams, mctx_cur);
const auto n_kv = mctx_cur->get_n_kv(); // 这里应该每一次都会不一样
inp->self_kq_mask = ggml_new_tensor_2d(ctx0, GGML_TYPE_F32, n_kv, GGML_PAD(n_tokens, GGML_KQ_MASK_PAD));
ggml_set_input(inp->self_kq_mask);
inp->self_kq_mask_cnv = cparams.flash_attn ? ggml_cast(ctx0, inp->self_kq_mask, GGML_TYPE_F16) : inp->self_kq_mask;
return (llm_graph_input_attn_kv_unified *) res->add_input(std::move(inp));
```

##### [`llm_graph_context::build_inp_out_ids`](src/llama-graph.cpp#L925)

```c++
ggml_tensor * inp_out_ids = build_inp_out_ids();
----------
ggml_tensor * llm_graph_context::build_inp_out_ids() const {}
```

<details>
<summary>class llm_graph_input_out_ids</summary>

```c++
class llm_graph_input_out_ids : public llm_graph_input_i {
public:
    ggml_tensor * out_ids; // I32 [n_outputs]

    const llama_hparams & hparams;
    const llama_cparams & cparams;

    const int32_t n_outputs;
};


void llm_graph_input_out_ids::set_input(const llama_ubatch * ubatch) {
    const int64_t n_tokens = ubatch->n_tokens;

    int32_t * data = (int32_t *) out_ids->data;

    if (n_outputs == n_tokens) {
        for (int i = 0; i < n_tokens; ++i) {
            data[i] = i;
        }
    } else if (ubatch->output) {
        int32_t n_outputs = 0;
        for (int i = 0; i < n_tokens; ++i) {
            if (ubatch->output[i]) {
                data[n_outputs++] = i;
            }
        }
    } else if (n_outputs == 1) {
        // only keep last output
        data[0] = n_tokens - 1;
    }
}
```

</details>

```c++
auto inp = std::make_unique<llm_graph_input_out_ids>(hparams, cparams, n_outputs);
auto & cur = inp->out_ids;
cur = ggml_new_tensor_1d(ctx0, GGML_TYPE_I32, n_outputs);
ggml_set_input(cur);
res->add_input(std::move(inp));
return cur;
```

##### [`llm_graph_context::build_attn`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L1255)


```c++
cur = build_attn(inp_attn, gf,
                model.layers[il].wo, model.layers[il].bo,
                Qcur, Kcur, Vcur, nullptr, nullptr, kq_scale, il);
----------
ggml_tensor * llm_graph_context::build_attn(
        llm_graph_input_attn_kv_unified * inp,
        ggml_cgraph * gf,
        ggml_tensor * wo,
        ggml_tensor * wo_b,
        ggml_tensor * q_cur,
        ggml_tensor * k_cur,
        ggml_tensor * v_cur,
        ggml_tensor * kq_b,
        ggml_tensor * v_mla,
            float     kq_scale,
            int       il) const {}
```

```c++
// these nodes are added to the graph together so that they are not reordered
// by doing so, the number of splits in the graph is reduced
--> ggml_build_forward_expand(gf, q_cur) --> ggml_build_forward_impl --> ggml_visit_parents;
--> ggml_build_forward_expand(gf, k_cur) --> ggml_build_forward_impl --> ggml_visit_parents;
--> ggml_build_forward_expand(gf, v_cur) --> ggml_build_forward_impl --> ggml_visit_parents;
const auto * mctx_cur = static_cast<const llama_kv_cache_unified_context *>(mctx);
// store to KV cache
--> ggml_build_forward_expand(gf, kv_state->cpy_k(ctx0, k_cur, il)) --> ggml_build_forward_impl --> ggml_visit_parents;
--> ggml_build_forward_expand(gf, kv_state->cpy_v(ctx0, v_cur, il)) --> ggml_build_forward_impl --> ggml_visit_parents;
const auto & kq_mask = inp->get_kq_mask();
ggml_tensor * q = q_cur;
ggml_tensor * k = kv_state->get_k(ctx0, il);
ggml_tensor * v = kv_state->get_v(ctx0, il);
--> lm_graph_context::build_attn_mha --> ggml_tensor * cur;
cb(cur, "kqv_out", il);
cur = build_lora_mm(wo, cur);
return cur;
```

- [`llama_kv_cache_unified_state::cpy_k`]()
- [`llama_kv_cache_unified_state::cpy_v`]()
- [`ggml_cpy_impl`]()

```c++
kv_state->cpy_k(ctx0, k_cur, il);
kv_state->cpy_v(ctx0, v_cur, il);
----------
ggml_tensor * llama_kv_cache_unified_state::cpy_k(ggml_context * ctx, ggml_tensor * k_cur, int32_t il) const
    return kv->cpy_k(ctx, k_cur, il, head);

ggml_tensor * llama_kv_cache_unified_state::cpy_v(ggml_context * ctx, ggml_tensor * v_cur, int32_t il) const
    return kv->cpy_v(ctx, v_cur, il, head);

ggml_tensor * llama_kv_cache_unified::cpy_k(ggml_context * ctx, ggml_tensor * k_cur, int32_t il, uint32_t head_cur) const {
    const int32_t ikv = map_layer_ids.at(il);

    auto * k = layers[ikv].k;

    const int64_t n_tokens = k_cur->ne[2];

    ggml_tensor * k_view = ggml_view_1d(ctx, k,
            n_tokens*hparams.n_embd_k_gqa(il),
            ggml_row_size(k->type, hparams.n_embd_k_gqa(il))*head_cur);

    --> return ggml_cpy(ctx, k_cur, k_view);
}

ggml_tensor * llama_kv_cache_unified::cpy_v(ggml_context * ctx, ggml_tensor * v_cur, int32_t il, uint32_t head_cur) const {
    const int32_t ikv = map_layer_ids.at(il);

    auto * v = layers[ikv].v;

    const int64_t n_tokens = v_cur->ne[2];

    v_cur = ggml_reshape_2d(ctx, v_cur, hparams.n_embd_v_gqa(il), n_tokens);

    ggml_tensor * v_view = nullptr;

    if (!v_trans) {
        v_view = ggml_view_1d(ctx, v,
                n_tokens*hparams.n_embd_v_gqa(il),
                ggml_row_size(v->type, hparams.n_embd_v_gqa(il))*head_cur);
    --> } else { //一般走这个路径
        // note: the V cache is transposed when not using flash attention
        v_view = ggml_view_2d(ctx, v, n_tokens, hparams.n_embd_v_gqa(il),
                (v->ne[1])*ggml_element_size(v),
                (head_cur)*ggml_element_size(v));

        v_cur = ggml_transpose(ctx, v_cur);
    }

    --> return ggml_cpy(ctx, v_cur, v_view);
}

struct ggml_tensor * ggml_cpy(
        struct ggml_context * ctx,
        struct ggml_tensor * a,
        struct ggml_tensor * b) {
    --> return ggml_cpy_impl(ctx, a, b);
}

static struct ggml_tensor * ggml_cpy_impl(
        struct ggml_context * ctx,
        struct ggml_tensor  * a,
        struct ggml_tensor  * b) {
    // make a view of the destination
    struct ggml_tensor * result = ggml_view_tensor(ctx, b);
    if (strlen(b->name) > 0) {
        ggml_format_name(result, "%s (copy of %s)", b->name, a->name);
    } else {
        ggml_format_name(result, "%s (copy)", a->name);
    }

    result->op     = GGML_OP_CPY;
    result->src[0] = a;
    result->src[1] = b;

    return result;
}
```

[`llm_graph_context::build_attn_mha`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L1061)

```c++
ggml_tensor * cur = build_attn_mha(gf, q, k, v, kq_b, kq_mask, v_mla, kq_scale);
----------
ggml_tensor * llm_graph_context::build_attn_mha(
         ggml_cgraph * gf,
         ggml_tensor * q,
         ggml_tensor * k,
         ggml_tensor * v,
         ggml_tensor * kq_b,
         ggml_tensor * kq_mask,
         ggml_tensor * v_mla,
             float     kq_scale) const {
    const bool v_trans = v->nb[1] > v->nb[2];

    q = ggml_permute(ctx0, q, 0, 2, 1, 3);
    k = ggml_permute(ctx0, k, 0, 2, 1, 3);
    v = ggml_permute(ctx0, v, 0, 2, 1, 3);

    const auto n_tokens = q->ne[1];
    const auto n_head   = q->ne[2];
    ggml_tensor * kq = ggml_mul_mat(ctx0, k, q);
    // note: this op tends to require high floating point range
    // while for some models F16 is enough, for others it is not, so we default to F32 here
    ggml_mul_mat_set_prec(kq, GGML_PREC_F32);
    kq = ggml_soft_max_ext(ctx0, kq, kq_mask, kq_scale, hparams.f_max_alibi_bias);
    ggml_tensor * kqv = ggml_mul_mat(ctx0, v, kq);
    ggml_tensor * cur = ggml_permute(ctx0, kqv, 0, 2, 1, 3);
    cur = ggml_cont_2d(ctx0, cur, cur->ne[0]*n_head, n_tokens);
    ggml_build_forward_expand(gf, cur);
    return cur;
}
```

##### [`ggml_visit_parents`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L5794)

> 这函数主要是为了构建 gf，也即 ggml_cgraph * cgraph;
> cgraph->leafs[cgraph->n_leafs] = node; cgraph->n_leafs++; cgraph->nodes[cgraph->n_nodes] = node; cgraph->n_nodes++;

```c++
ggml_build_forward_expand(gf, cur);
----------
void ggml_build_forward_expand(struct ggml_cgraph * cgraph, struct ggml_tensor * tensor) {
    --> ggml_build_forward_impl(cgraph, tensor, true);
}

static void ggml_build_forward_impl(struct ggml_cgraph * cgraph, struct ggml_tensor * tensor, bool expand) {
    const int n0 = cgraph->n_nodes;
    --> ggml_visit_parents(cgraph, tensor);
    const int n_new = cgraph->n_nodes - n0;
}

static void ggml_visit_parents(struct ggml_cgraph * cgraph, struct ggml_tensor * node) {}
```

```c++
// check if already visited
if (ggml_hash_insert(&cgraph->visited_hash_set, node) == GGML_HASHSET_ALREADY_EXISTS) {
    return;
}

for (int i = 0; i < GGML_MAX_SRC; ++i) {
    const int k =
        (cgraph->order == GGML_CGRAPH_EVAL_ORDER_LEFT_TO_RIGHT) ? i :
        (cgraph->order == GGML_CGRAPH_EVAL_ORDER_RIGHT_TO_LEFT) ? (GGML_MAX_SRC-1-i) :
        /* unknown order, just fall back to using i*/ i;
    if (node->src[k]) {
        ggml_visit_parents(cgraph, node->src[k]);
    }
}

if (node->op == GGML_OP_NONE && !(node->flags & GGML_TENSOR_FLAG_PARAM)) {
    // reached a leaf node, not part of the gradient graph (e.g. a constant)
    if (strlen(node->name) == 0) {
        ggml_format_name(node, "leaf_%d", cgraph->n_leafs);
    }
    cgraph->leafs[cgraph->n_leafs] = node;
    cgraph->n_leafs++;
} else {
    if (strlen(node->name) == 0) {
        ggml_format_name(node, "node_%d", cgraph->n_nodes);
    }
    cgraph->nodes[cgraph->n_nodes] = node;
    cgraph->n_nodes++;
}
```

#### [`ggml_backend_sched_reserve`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1549)

```c++
ggml_backend_sched_reserve(sched.get(), gf);
----------
bool ggml_backend_sched_reserve(ggml_backend_sched_t sched, struct ggml_cgraph * measure_graph) {}
```

```c++
--> ggml_backend_sched_split_graph --> sched->splits, sched->graph, sched->node_backend_ids, sched->leaf_backend_ids;
--> ggml_backend_sched_synchronize;
--> ggml_gallocr_reserve_n --> sched->galloc;
ggml_backend_sched_reset(sched);
return true;
```

##### [`ggml_backend_sched_split_graph`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L865)

> 分配每个node/leaf的后端类型，构建动态的部署计算图，并且归纳出需要split的node

> 还有就是初始化自己域内的ctx变量，并且把需要split的节点在ctx上开辟出来，而且看起来有很多余量

> 根据后端资源的分配构建动态图，存入sched->graph中

```c++
ggml_backend_sched_split_graph(sched, measure_graph);
----------
// assigns backends to ops and splits the graph into subgraphs that can be computed on the same backend
static void ggml_backend_sched_split_graph(ggml_backend_sched_t sched, struct ggml_cgraph * graph) {}
```

<details>
<summary>struct ggml_backend_sched_split</summary>

```c++
struct ggml_backend_sched_split {
    int backend_id;
    int i_start;
    int i_end;
    struct ggml_tensor * inputs[GGML_SCHED_MAX_SPLIT_INPUTS];
    int n_inputs;
    // graph view of this split
    struct ggml_cgraph graph;
};
```

</details>

```c++
// reset splits
sched->n_splits = 0;
sched->n_graph_inputs = 0;
sched->is_reset = false;

struct ggml_init_params params = {
    /* .mem_size =   */ sched->context_buffer_size,
    /* .mem_buffer = */ sched->context_buffer,
    /* .no_alloc =   */ true
};

ggml_free(sched->ctx);
sched->ctx = ggml_init(params);
// pass 1: assign backends to ops with pre-allocated inputs
for (int i = 0; i < graph->n_leafs; i++)
    int * leaf_backend_id = &sched->hv_tensor_backend_ids[hash_id(graph->leafs[i])];
    if (*leaf_backend_id == -1)
    --> *leaf_backend_id = ggml_backend_sched_backend_id_from_cur(sched, graph->leafs[i]);
for (int i = 0; i < graph->n_nodes; i++)
    int * node_backend_id = sched->hv_tensor_backend_ids[hash_id(graph->nodes[i])]
    if (*node_backend_id == -1)
    --> *node_backend_id = ggml_backend_sched_backend_id_from_cur(sched, graph->nodes[i]);
// pass 2: expand current backend assignments
// assign the same backend to adjacent nodes
// expand gpu backends (i.e. non last prio) up and down, ignoring cpu (the lowest priority backend)
// thus, cpu will never be used unless weights are on cpu, or there are no gpu ops between cpu ops
// ops unsupported by the backend being expanded will be left unassigned so that they can be assigned later when the locations of its inputs are known
// expand gpu down
for (int i = 0; i < graph->n_nodes; i++)
    if (*node_backend_id == sched->n_backends - 1)
        cur_backend_id = -1; // skip cpu (lowest prio backend)
    ggml_backend_sched_set_if_supported(sched, node, cur_backend_id, node_backend_id); //SET_CAUSE(node, "2.sup");
    //node_id = [5,8,28,29,32,34,36,37,41,44,58,59,60,62,64, so on...]
// expand gpu up
for (int i = graph->n_nodes - 1; i >= 0; i--)
    if (*node_backend_id == sched->n_backends - 1)
        cur_backend_id = -1; // skip cpu (lowest prio backend)
    ggml_backend_sched_set_if_supported(sched, node, cur_backend_id, node_backend_id); //SET_CAUSE(node, "2.sup");
    //node_id = [26,24,23,22,1]
// expand rest down 没有起作用
// expand rest up 没有起作用
////pass 3没有起作用
// pass 3: upgrade nodes to higher prio backends with compatible buffer types
// if the tensor is already in the same buffer type (*) as another higher priority backend, we should move it there
// however, we also need to verify that the sources are in compatible buffer types
// (*) the actual requirement is more relaxed, the buffer type of the backend should be supported by all the users of this tensor further down the graph
// however, this is slow to verify, so we have a more strict requirement that the buffer type is the same
// this is not uncommon since multiple backends can use host memory, with the same buffer type (eg. BLAS and CPU)
// additionally, set remaining unassigned nodes to the backend with the most supported inputs
// only nodes that could not be assigned during expansion due to the backend not supporting the op should be unassigned at this point
// pass 4: assign backends to remaining src from dst and view_src
////开始针对view_src了，后半段遍历SRC的没起作用
for (int i = 0; i < graph->n_nodes; i++)
    struct ggml_tensor * node = graph->nodes[i];
    int * cur_backend_id = &tensor_backend_id(node);
    if (node->view_src != NULL && *cur_backend_id == -1) *cur_backend_id = tensor_backend_id(node->view_src); //SET_CAUSE(node, "4.vsrc");
// pass 5: split graph, find tensors that need to be copied
int i_split = 0;
struct ggml_backend_sched_split * split = &sched->splits[0];
// find the backend of the first split, skipping view ops
int cur_backend_id = split->backend_id = tensor_backend_id(graph->nodes[0]);
split->i_start = 0;
split->n_inputs = 0;
for (int i = 0; i < graph->n_nodes; i++)
    // check if we should start a new split based on the sources of the current node
    if (node_backend_id == cur_backend_id && split->n_inputs > 0)
        for (int j = 0; j < GGML_MAX_SRC; j++) struct ggml_tensor * src = node->src[j]; size_t src_id = hash_id(src);
        // check if a weight is on a different and incompatible backend by starting a new split,
        // the memory of the previously offloaded weights can be reused
        if (src->buffer != NULL && src->buffer->usage == GGML_BACKEND_BUFFER_USAGE_WEIGHTS)
            //如果出现已经有split了，且weight不在当前的backend上，且当前的backend的不支持，需要新的split
            sched->hv_tensor_backend_ids[src_id] != cur_backend_id && \
            --> ggml_backend_sched_buffer_supported --> ggml_backend_supports_buft;
    if (node_backend_id != cur_backend_id || need_new_split)
        split->i_end = i;
        i_split++;
        split = &sched->splits[i_split];
        split->backend_id = node_backend_id;
        split->i_start = i;
        split->n_inputs = 0;
        cur_backend_id = node_backend_id;
    // find inputs that are not on the same backend
    for (int j = 0; j < GGML_MAX_SRC; j++)
        struct ggml_tensor * src = node->src[j];
        size_t src_id = hash_id(src);
        sched->hv_tensor_backend_ids[src_id] != cur_backend_id && \
        --> !ggml_backend_sched_buffer_supported --> ggml_backend_supports_buft; //这个条件必不可能为true啊
            // create a copy of the input in the split's backend
            if (sched->hv_tensor_copies[hash_set.size=src_id][n_backends=cur_backend_id][n_copies=0] == NULL)
                ggml_backend_t backend = sched->backends[cur_backend_id];
                // for (int c = 0; c < sched->n_copies; c++)
                --> ggml_dup_tensor_layout(sched->ctx, src) --> struct ggml_tensor * tensor_copy;
                    return struct ggml_tensor * dup = ggml_dup_tensor(ctx, tensor);
                    for (int i = 0; i < GGML_MAX_DIMS; i++) dup->nb[i] = tensor->nb[i];
                sched->hv_tensor_copies[hash_set.size=src_id][n_backends=cur_backend_id][n_copies=0] = tensor_copy;
                int n_inputs = split->n_inputs++;
                split->inputs[n_inputs] = src;
            //后续还承接了 ROPE/SOFT_MAX 的处理
            // node->src[j] = tensor_id_copy(src_id, cur_backend_id, sched->cur_copy=0);
            node->src[j] = tensor_copy
split->i_end = graph->n_nodes;
sched->n_splits = i_split + 1;
// swap node_backend_ids and leaf _backend_ids with prevs
int * tmp = sched->node_backend_ids;
sched->node_backend_ids = sched->prev_node_backend_ids;
sched->prev_node_backend_ids = tmp;
tmp = sched->leaf_backend_ids;
sched->leaf_backend_ids = sched->prev_leaf_backend_ids;
sched->prev_leaf_backend_ids = tmp;
// --> for `ggml_backend_sched_alloc_splits` func
int graph_size = std::max(graph->n_nodes, graph->n_leafs) + sched->n_splits*GGML_SCHED_MAX_SPLIT_INPUTS*2*sched->n_copies;
// 这里第二次有可能就不执行了
if (sched->graph.size < graph_size)
    sched->graph.size = graph_size;
    sched->graph.nodes = (ggml_tensor **) realloc(sched->graph.nodes, graph_size * sizeof(struct ggml_tensor *));
    sched->graph.leafs = (ggml_tensor **) realloc(sched->graph.leafs, graph_size * sizeof(struct ggml_tensor *));
sched->graph.n_nodes = 0;
sched->graph.n_leafs = 0;
struct ggml_cgraph * graph_copy = &sched->graph;
for (int i = 0; i < sched->n_splits; i++) {
    struct ggml_backend_sched_split * split = &sched->splits[i];
    // 这里也非常巧妙 *key*
    --> split->graph = ggml_graph_view(graph, split->i_start, split->i_end);
    // add inputs to the graph copy so that they are allocated by ggml-alloc at the start of the split
    for (int j = 0; j < split->n_inputs; j++) {
        struct ggml_tensor * input = split->inputs[j];
        const size_t input_id = hash_id(input);
        struct ggml_tensor * input_cpy = sched->hv_tensor_copies[input_id][split->backend_id][0];
        // add a dependency to the input source so that it is not freed before the copy is done
        // 就是占个位置，建立依赖关系，以免被后面的tensor给覆盖掉
        struct ggml_tensor * input_dep = ggml_view_tensor(sched->ctx, input);
        input_dep->src[0] = input;
        sched->node_backend_ids[graph_copy->n_nodes] = sched->hv_tensor_backend_ids[input_id];
        graph_copy->nodes[graph_copy->n_nodes++] = input_dep;
        // add a dependency to the input copy so that it is allocated at the start of the split
        sched->node_backend_ids[graph_copy->n_nodes] = split->backend_id;
        graph_copy->nodes[graph_copy->n_nodes++] = input_cpy;
    }
    for (int j = split->i_start; j < split->i_end; j++) {
        assert(graph_copy->size > graph_copy->n_nodes);
        sched->node_backend_ids[graph_copy->n_nodes] = tensor_backend_id(graph->nodes[j]);
        graph_copy->nodes[graph_copy->n_nodes++] = graph->nodes[j];
    }
}
// add leafs from the original graph
for (int i = 0; i < graph->n_leafs; i++) {
    struct ggml_tensor * leaf = graph->leafs[i];
    sched->leaf_backend_ids[graph_copy->n_leafs] = tensor_backend_id(leaf);
    assert(graph_copy->size > graph_copy->n_leafs);
    graph_copy->leafs[graph_copy->n_leafs++] = leaf;
}
```

```log
>>> p *leaf_buffer_ids@graph->n_leafs
$1094 = {[0] = 1 <repeats 11 times>, [11] = 0, [12] = 1, [13] = 1, [14] = 1, [15] = 1, [16] = 1, [17] = 0 <repeats 336 times>, [353] = 1, [354] = 0, [355] = 0, [356] = 0, [357] = 1, [358] = 0, [359] = 0, [360] = 1}
>>> p *node_buffer_ids@graph->n_nodes
$1095 = {[0] = 1, [1] = 1, [2] = 0, [3] = 0, [4] = 1, [5] = 0, [6] = 0, [7] = 1, [8] = 0, [9] = 0, [10] = 0, [11] = 1, [12] = 0, [13] = 1, [14] = 0, [15] = 0, [16] = 1, [17] = 0, [18] = 0, [19] = 0, [20] = 1, [21] = 0, [22] = 0, [23] = 0, [24] = 0, [25] = 1, [26] = 0, [27] = 1, [28] = 0, [29] = 1, [30] = 1, [31] = 0, [32] = 0, [33] = 1, [34] = 1, [35] = 1, [36] = 1, [37] = 1, [38] = 1, [39] = 0, [40] = 1, [41] = 0, [42] = 1, [43] = 0, [44] = 1, [45] = 0, [46] = 0, [47] = 0, [48] = 0, [49] = 0, [50] = 0, [51] = 1, [52] = 0, [53] = 0, [54] = 0, [55] = 0, [56] = 1, [57] = 0, [58] = 0, [59] = 1, [60] = 0, [61] = 0, [62] = 0, [63] = 1, [64] = 0, [65] = 0, [66] = 0, [67] = 1, [68] = 0, [69] = 1, [70] = 0 <repeats 1122 times>, [1192] = 1, [1193] = 0, [1194] = 0, [1195] = 1, [1196] = 0, [1197] = 0}
```

[`ggml_backend_sched_backend_id_from_cur`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L730)

```c++
// pass 1: assign backends to ops with pre-allocated inputs
*leaf_backend_id = ggml_backend_sched_backend_id_from_cur(sched, leaf);
*node_backend_id = ggml_backend_sched_backend_id_from_cur(sched, node);
----------
static int ggml_backend_sched_backend_id_from_cur(ggml_backend_sched_t sched, struct ggml_tensor * tensor) {
    // assign pre-allocated nodes to their backend
    --> ggml_backend_sched_backend_from_buffer(sched, tensor, tensor) --> int cur_backend_id;
        ggml_backend_buffer_t buffer = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
        if (buffer == NULL) return -1;
        // find highest prio backend that supports the buffer type and the op
        for (int i = 0; i < sched->n_backends; i++)
            if (ggml_backend_supports_buft(sched->backends[i], buffer->buft) && ggml_backend_supports_op(sched->backends[i], op))
                return i;
    return cur_backend_id; // SET_CAUSE(tensor, "1.dst");
    if (tensor->view_src != NULL) // view_src
        --> ggml_backend_sched_backend_from_buffer(sched, tensor->view_src, tensor) --> int cur_backend_id;
    return cur_backend_id; // SET_CAUSE(tensor, "1.vsrc");
    if (tensor->buffer || (tensor->view_src && tensor->view_src->buffer))
        GGML_ABORT;// since the tensor is pre-allocated, it cannot be moved to another backend
    if (tensor->flags & GGML_TENSOR_FLAG_INPUT) // graph input
        cur_backend_id = sched->n_backends - 1; // last backend (assumed CPU)
    for (int i = 0; i < GGML_MAX_SRC; i++) // operations with weights are preferably run on the same backend as the weights
        const struct ggml_tensor * src = tensor->src[i];
        // skip ROPE since the rope freqs tensor is too small to choose a backend based on it not an ideal solution
        if (tensor->op != GGML_OP_ROPE && src->buffer != NULL && src->buffer->usage == GGML_BACKEND_BUFFER_USAGE_WEIGHTS)
            --> ggml_backend_sched_backend_from_buffer(sched, src, tensor) --> int src_backend_id;
            // check if a backend with higher prio wants to offload the op
            if (sched->op_offload && src_backend_id == sched->n_backends - 1 && ggml_backend_buffer_is_host(src->buffer))
            for (int b = 0; b < src_backend_id; b++)
            if (ggml_backend_supports_op(sched->backends[b], tensor) && ggml_backend_offload_op(sched->backends[b], tensor))
                // 这里是区分PP和TG的关键！！！ *key*
                return b; // SET_CAUSE(tensor, "1.off");
            return src_backend_id; // SET_CAUSE(tensor, "1.wgt%d", i);
}
```

```c++
bool ggml_backend_supports_buft(ggml_backend_t backend, ggml_backend_buffer_type_t buft) {
    return ggml_backend_dev_supports_buft(backend->device, buft);
}
bool ggml_backend_dev_supports_buft(ggml_backend_dev_t device, ggml_backend_buffer_type_t buft) {
    return device->iface.supports_buft(device, buft);
}
static bool ggml_backend_cuda_device_supports_buft(ggml_backend_dev_t dev, ggml_backend_buffer_type_t buft) {
    ggml_backend_cuda_device_context * dev_ctx = (ggml_backend_cuda_device_context *) dev->context;
    const bool integrated = ggml_cuda_info().devices[dev_ctx->device].integrated;
    return (((ggml_backend_buft_is_cuda(buft) || ggml_backend_buft_is_cuda_split(buft)) && buft->device == dev) || (integrated && ggml_backend_buft_is_cuda_host(buft)));
}
static bool ggml_backend_cpu_device_supports_buft(ggml_backend_dev_t dev, ggml_backend_buffer_type_t buft) {
    return ggml_backend_buft_is_host(buft) || ggml_backend_cpu_is_extra_buffer_type(buft);
}

bool ggml_backend_supports_op(ggml_backend_t backend, const struct ggml_tensor * op) {
    return ggml_backend_dev_supports_op(backend->device, op);
}
bool ggml_backend_dev_supports_op(ggml_backend_dev_t device, const struct ggml_tensor * op) {
    return device->iface.supports_op(device, op);
    --> ggml_backend_cuda_device_supports_op;
    --> ggml_backend_cpu_device_supports_op;
}

bool ggml_backend_buffer_is_host(ggml_backend_buffer_t buffer) {
    return ggml_backend_buft_is_host(ggml_backend_buffer_get_type(buffer));
}
bool ggml_backend_buft_is_host(ggml_backend_buffer_type_t buft) {
    if (buft->iface.is_host) {
        return buft->iface.is_host(buft);
    }
    return false;
}
static bool ggml_backend_cpu_buffer_type_is_host(ggml_backend_buffer_type_t buft) {
    return true;
}

bool ggml_backend_offload_op(ggml_backend_t backend, const struct ggml_tensor * op) {
    return ggml_backend_dev_offload_op(backend->device, op);
}
bool ggml_backend_dev_offload_op(ggml_backend_dev_t device, const struct ggml_tensor * op) {
    if (device->iface.offload_op != NULL) {
        return device->iface.offload_op(device, op);
    }
    return false;
}
static bool ggml_backend_cuda_device_offload_op(ggml_backend_dev_t dev, const ggml_tensor * op) {
    const int min_batch_size = 32;
    return get_op_batch_size(op) >= min_batch_size;
        --> static int64_t get_op_batch_size(const ggml_tensor * op)
        switch (op->op) {
            case GGML_OP_GET_ROWS:
                return 0;
            case GGML_OP_MUL_MAT:
                return op->ne[1];
            case GGML_OP_MUL_MAT_ID:
            case GGML_OP_ROPE:
            case GGML_OP_ROPE_BACK:
                return op->ne[2];
            default:
                return ggml_nrows(op);
        }
}
```

##### [`ggml_backend_sched_synchronize`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1599)

```c++
ggml_backend_sched_synchronize(sched);
----------
void ggml_backend_sched_synchronize(ggml_backend_sched_t sched) {}
```

```c++
for (int i = 0; i < sched->n_backends; i++) {
    --> ggml_backend_synchronize(sched->backends[i]);
}
if (!sched->is_alloc) {
    // if the graph is not already allocated, always use copy 0 after a synchronization
    // this ensures that during generation the same copy is used every time,
    // which avoids changes in the graph that could cause CUDA or other graphs to be disabled
    sched->cur_copy = 0;
}
```

[`ggml_backend_cuda_synchronize`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu:2497)

```c++
ggml_backend_synchronize(sched->backends[i]);
----------
void ggml_backend_synchronize(ggml_backend_t backend) {
    if (backend->iface.synchronize == NULL) return; //CPU
    backend->iface.synchronize(backend); //GPU
}
static void ggml_backend_cuda_synchronize(ggml_backend_t backend) {
    ggml_backend_cuda_context * cuda_ctx = (ggml_backend_cuda_context *)backend->context;
    --> cudaStreamSynchronize(cuda_ctx->stream());
}
```

##### [`ggml_gallocr_reserve_n`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L673)

> 感觉就是验证PP/TG中最差情况的可运行性，好多没用的赋值。。。。

> 而这些复制的结构体中的成员变量是要和 `ggml_gallocr_alloc_graph` 相配合的

```c++
ggml_gallocr_reserve_n(sched->galloc, &sched->graph, sched->node_backend_ids, sched->leaf_backend_ids);
----------
bool ggml_gallocr_reserve_n(ggml_gallocr_t galloc, struct ggml_cgraph * graph, const int * node_buffer_ids, const int * leaf_buffer_ids) {}
```

<details>
<summary>struct hash_node</summary>

```c++
struct hash_node {
    int n_children;
    int n_views;
    int buffer_id;
    size_t offset; // offset within the buffer
    bool allocated;
};
```

</details>

> galloc->hash_set 的初始化

```c++
size_t min_hash_size = graph->n_nodes + graph->n_leafs;
// add 25% margin to avoid hash collisions
min_hash_size += min_hash_size / 4;
// initialize hash table
// 这里第二次有可能就不执行了
if (galloc->hash_set.size < min_hash_size) {
    ggml_hash_set_free(&galloc->hash_set);
    galloc->hash_set = ggml_hash_set_new(min_hash_size);
    free(galloc->hash_values);
    galloc->hash_values = malloc(sizeof(struct hash_node) * galloc->hash_set.size);
}
// reset allocators
for (int i = 0; i < galloc->n_buffers; i++) ggml_dyn_tallocr_reset(galloc->buf_tallocs[i]);
// allocate in hash table
--> ggml_gallocr_alloc_graph_impl --> sched->galloc->hash_set, sched->galloc->hash_values, sched->galloc->buf_tallocs;
```

<details>
<summary>struct tensor_alloc</summary>

```c++
struct tensor_alloc {
    int buffer_id;
    size_t offset;
    size_t size_max; // 0 = pre-allocated, unused, or view
};
```

</details>


<details>
<summary>struct node_alloc</summary>

```c++
struct node_alloc {
    struct tensor_alloc dst;
    struct tensor_alloc src[GGML_MAX_SRC];
};
```

</details>

> galloc->node_allocs的设置，但感觉并没有用到啊！！！

```c++
// set the node_allocs from the hash table
//这里第二次有可能就不执行了
if (galloc->n_nodes < graph->n_nodes) {
    free(galloc->node_allocs);
    galloc->node_allocs = calloc(graph->n_nodes, sizeof(struct node_alloc));
}
galloc->n_nodes = graph->n_nodes;
for (int i = 0; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];
    struct node_alloc * node_alloc = &galloc->node_allocs[i];
    if (node->view_src || node->data) {
        node_alloc->dst.buffer_id = -1;
        node_alloc->dst.offset = SIZE_MAX;
        node_alloc->dst.size_max = 0;
    } else {
        struct hash_node * hn = ggml_gallocr_hash_get(galloc, node);
        node_alloc->dst.buffer_id = hn->buffer_id;
        node_alloc->dst.offset    = hn->offset;
        node_alloc->dst.size_max  = ggml_backend_buft_get_alloc_size(galloc->bufts[hn->buffer_id], node);
    }
    for (int j = 0; j < GGML_MAX_SRC; j++) {
        struct ggml_tensor * src = node->src[j];
        if (!src || src->view_src || src->data) {
            node_alloc->src[j].buffer_id = -1;
            node_alloc->src[j].offset = SIZE_MAX;
            node_alloc->src[j].size_max = 0;
        } else {
            struct hash_node * hn = ggml_gallocr_hash_get(galloc, src);
            node_alloc->src[j].buffer_id = hn->buffer_id;
            node_alloc->src[j].offset   = hn->offset;
            node_alloc->src[j].size_max = ggml_backend_buft_get_alloc_size(galloc->bufts[hn->buffer_id], src);
        }
    }
}
```

<details>
<summary>struct leaf_alloc</summary>

```c++
struct leaf_alloc {
    struct tensor_alloc leaf;
};
```

</details>

> galloc->leaf_allocs的设置，但感觉并没有用到啊！！！

```c++
//这里第二次有可能就不执行了
if (galloc->n_leafs < graph->n_leafs) {
    free(galloc->leaf_allocs);
    galloc->leaf_allocs = calloc(graph->n_leafs, sizeof(galloc->leaf_allocs[0]));
}
galloc->n_leafs = graph->n_leafs;
for (int i = 0; i < graph->n_leafs; i++) {
    struct ggml_tensor * leaf = graph->leafs[i];
    struct hash_node * hn = ggml_gallocr_hash_get(galloc, leaf);
    if (leaf->view_src || leaf->data) {
        galloc->leaf_allocs[i].leaf.buffer_id = -1;
        galloc->leaf_allocs[i].leaf.offset = SIZE_MAX;
        galloc->leaf_allocs[i].leaf.size_max = 0;
    } else {
        galloc->leaf_allocs[i].leaf.buffer_id = hn->buffer_id;
        galloc->leaf_allocs[i].leaf.offset = hn->offset;
        galloc->leaf_allocs[i].leaf.size_max = ggml_backend_buft_get_alloc_size(galloc->bufts[hn->buffer_id], leaf);
    }
}
```

> 这里每次重新跑会不一样

```c++
// reallocate buffers if needed
for (int i = 0; i < galloc->n_buffers; i++) {
    // if the buffer type is used multiple times, we reuse the same buffer
    //...
    size_t cur_size = galloc->buffers[i] ? ggml_backend_buffer_get_size(galloc->buffers[i]) : 0;
    size_t new_size = ggml_dyn_tallocr_max_size(galloc->buf_tallocs[i]);
    //这里第二次有可能就不执行了
    // 第一次：cuda=189.31MB, cuda_host=0.532MB
    // 第二次：cuda=0.172MB,  cuda_host=0.529MB
    // even if there are no tensors allocated in this buffer, we still need to allocate it to initialize views
    if (new_size > cur_size || galloc->buffers[i] == NULL) {
        ggml_backend_buffer_free(galloc->buffers[i]);
        // galloc->buffers[i] = ggml_backend_buft_alloc_buffer(galloc->bufts[i], new_size);
        --> ggml_backend_buft_alloc_buffer --> galloc->buffers[i];
            --> ggml_backend_cuda_buffer_type_alloc_buffer;
            --> ggml_backend_cuda_host_buffer_type_alloc_buffer;
        ggml_backend_buffer_set_usage(galloc->buffers[i], GGML_BACKEND_BUFFER_USAGE_COMPUTE);
    }
}
return true;
```

###### [`ggml_gallocr_alloc_graph_impl`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L566)


```c++
// allocate in hash table
ggml_gallocr_alloc_graph_impl(galloc, graph, node_buffer_ids, leaf_buffer_ids);
----------
static void ggml_gallocr_alloc_graph_impl(ggml_gallocr_t galloc, struct ggml_cgraph * graph, const int * node_buffer_ids, const int * leaf_buffer_ids) {}
```

```log
>>> p *leaf_buffer_ids@graph->n_leafs
$1094 = {[0] = 1 <repeats 11 times>, [11] = 0, [12] = 1, [13] = 1, [14] = 1, [15] = 1, [16] = 1, [17] = 0 <repeats 336 times>, [353] = 1, [354] = 0, [355] = 0, [356] = 0, [357] = 1, [358] = 0, [359] = 0, [360] = 1}
>>> p *node_buffer_ids@graph->n_nodes
$1095 = {[0] = 1, [1] = 1, [2] = 0, [3] = 0, [4] = 1, [5] = 0, [6] = 0, [7] = 1, [8] = 0, [9] = 0, [10] = 0, [11] = 1, [12] = 0, [13] = 1, [14] = 0, [15] = 0, [16] = 1, [17] = 0, [18] = 0, [19] = 0, [20] = 1, [21] = 0, [22] = 0, [23] = 0, [24] = 0, [25] = 1, [26] = 0, [27] = 1, [28] = 0, [29] = 1, [30] = 1, [31] = 0, [32] = 0, [33] = 1, [34] = 1, [35] = 1, [36] = 1, [37] = 1, [38] = 1, [39] = 0, [40] = 1, [41] = 0, [42] = 1, [43] = 0, [44] = 1, [45] = 0, [46] = 0, [47] = 0, [48] = 0, [49] = 0, [50] = 0, [51] = 1, [52] = 0, [53] = 0, [54] = 0, [55] = 0, [56] = 1, [57] = 0, [58] = 0, [59] = 1, [60] = 0, [61] = 0, [62] = 0, [63] = 1, [64] = 0, [65] = 0, [66] = 0, [67] = 1, [68] = 0, [69] = 1, [70] = 0 <repeats 1122 times>, [1192] = 1, [1193] = 0, [1194] = 0, [1195] = 1, [1196] = 0, [1197] = 0}
```

```c++
// clear hash tables
ggml_hash_set_reset(&galloc->hash_set);
    --> memset(hash_set->used, 0, sizeof(ggml_bitset_t) * ggml_bitset_size(hash_set->size));
memset(galloc->hash_values, 0, sizeof(struct hash_node) * galloc->hash_set.size);
// allocate leafs
// these may be tensors that the application is not using in the graph, but may still want to allocate for other purposes
for (int i = 0; i < graph->n_leafs; i++) {
    struct ggml_tensor * leaf = graph->leafs[i];
    --> ggml_gallocr_allocate_node(galloc, leaf, leaf_buffer_ids[i]);
    // 没有 ggml_op_can_inplace 的情况
}
// count number of children and views
// allocate other graph inputs and leafs first to avoid overwriting them
for (int i = 0; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];
    // TODO: better way to add external dependencies
    // GGML_OP_NONE does not appear normally in the graph nodes, but is used by ggml-backend to add dependencies to
    // control when some tensors are allocated and freed. in this case, the dependencies are in `src`, but the node
    // itself is never used and should not be considered a dependency
    if (ggml_is_view(node) && node->op != GGML_OP_NONE)
        ggml_gallocr_hash_get(galloc, node->view_src)->n_views += 1;
    // if (node->flags & GGML_TENSOR_FLAG_INPUT) // 这里应该是不会走到的，因为既然是node了，怎么会是input呢
    for (int j = 0; j < GGML_MAX_SRC; j++) {
        // 这一步还挺关键的，对于buffer的回收 *key*
        ggml_gallocr_hash_get(galloc, node->src[j])->n_children += 1;
        // allocate explicit inputs
        if (node->src[j]->flags & GGML_TENSOR_FLAG_INPUT) {
            //第一次：i = [0-leaf_2,11-leaf_4,42-leaf_10,69-leaf_357]
            //第二次：i = [0-leaf_2,5-leaf_4,8-leaf_4,23-leaf_10,40-leaf_4,42-leaf_10,44-leaf_357]
            --> ggml_gallocr_allocate_node(galloc, node->src[j], node_buffer_ids[i]);
        }
    }
}
// allocate tensors
for (int i = 0; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];
    int buffer_id = get_node_buffer_id(node_buffer_ids, i);
    // allocate parents (only leafs need to be allocated at this point)
    // 感觉没什么作用？应该就只是确保src tensor没有不alloc的
    //// for (int j = 0; j < GGML_MAX_SRC; j++) ggml_gallocr_allocate_node(galloc, node->src[j], node_buffer_ids[i]); 
    // allocate node
    --> ggml_gallocr_allocate_node(galloc, node, node_buffer_ids[i]);
    // update parents
    for (int j = 0; j < GGML_MAX_SRC; j++) {
        struct ggml_tensor * parent = node->src[j];
        struct hash_node * p_hn = ggml_gallocr_hash_get(galloc, parent);
        p_hn->n_children -= 1;
        if (p_hn->n_children == 0 && p_hn->n_views == 0) {
            if (ggml_is_view(parent)) {
                struct ggml_tensor * view_src = parent->view_src;
                struct hash_node * view_src_hn = ggml_gallocr_hash_get(galloc, view_src);
                view_src_hn->n_views -= 1;
                if (view_src_hn->n_views == 0 && view_src_hn->n_children == 0 && view_src_hn->allocated) {
                    --> ggml_gallocr_free_node(galloc, view_src);
                }
            }
            else if (p_hn->allocated) {
                --> ggml_gallocr_free_node(galloc, parent);
            }
        }
    }
}
```

[`ggml_gallocr_allocate_node`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L478)

```c++
static void ggml_gallocr_allocate_node(ggml_gallocr_t galloc, struct ggml_tensor * node, int buffer_id) {
    GGML_ASSERT(buffer_id >= 0);
    struct hash_node * hn = ggml_gallocr_hash_get(galloc, node);
    if (!ggml_gallocr_is_allocated(galloc, node) && !ggml_is_view(node)) {
        --> return t->data != NULL || ggml_gallocr_hash_get(galloc, t)->allocated;
        hn->allocated = true;
        // try to reuse a parent's buffer (inplace)
        if (ggml_op_can_inplace(node->op))
        for (int i = 0; i < GGML_MAX_SRC; i++) {
            struct ggml_tensor * parent = node->src[i];
            // if the node's data is external, then we cannot re-use it
            if (!ggml_gallocr_is_own(galloc, parent)) continue;
            // outputs cannot be reused
            if (parent->flags & GGML_TENSOR_FLAG_OUTPUT || \
               (parent->view_src != NULL && parent->view_src->flags & GGML_TENSOR_FLAG_OUTPUT)) continue;
            if (!ggml_are_same_layout(node, parent)) continue;
            struct hash_node * p_hn = ggml_gallocr_hash_get(galloc, parent);
            if (p_hn->n_children == 1 && p_hn->n_views == 0) {
                if (ggml_is_view(parent)) {
                    struct ggml_tensor * view_src = parent->view_src;
                    struct hash_node * view_src_hn = ggml_gallocr_hash_get(galloc, view_src);
                    if (view_src_hn->n_views == 1 && view_src_hn->n_children == 0 && view_src->data == parent->data) {
                        assert(view_src_hn->offset == p_hn->offset);
                        hn->buffer_id = p_hn->buffer_id;
                        hn->offset = p_hn->offset;
                        p_hn->allocated = false; // avoid freeing the parent
                        view_src_hn->allocated = false;
                        return;
                    }
                } else {
                    hn->buffer_id = p_hn->buffer_id;
                    hn->offset = p_hn->offset;
                    p_hn->allocated = false; // avoid freeing the parent
                    return;
                }
            }
        }
        // allocate tensor from the buffer
        struct ggml_dyn_tallocr * alloc = galloc->buf_tallocs[buffer_id];
        ggml_backend_buffer_type_t buft = galloc->bufts[buffer_id];
        size_t size = ggml_backend_buft_get_alloc_size(buft, node);
        --> ggml_dyn_tallocr_alloc --> size_t offset;
        hn->buffer_id = buffer_id;
        hn->offset = offset;
    }
}
```

[`ggml_dyn_tallocr_alloc`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L153)

```c++
size_t offset = ggml_dyn_tallocr_alloc(alloc, size, node);
----------
static size_t ggml_dyn_tallocr_alloc(struct ggml_dyn_tallocr * alloc, size_t size, const struct ggml_tensor * tensor) {
    size = aligned_offset(NULL, size, alloc->alignment);
    // find the best fitting free block besides the last block
    int best_fit_block = -1;
    size_t best_fit_size = SIZE_MAX;
    for (int i = 0; i < alloc->n_free_blocks - 1; i++) {
        struct free_block * block = &alloc->free_blocks[i];
        if (block->size >= size && block->size <= best_fit_size) {
            best_fit_block = i;
            best_fit_size = block->size;
        }
    }
    if (best_fit_block == -1) {
        // the last block is our last resort
        // if (block->size >= size) {
        best_fit_block = alloc->n_free_blocks - 1;
    }
    struct free_block * block = &alloc->free_blocks[best_fit_block];
    size_t offset = block->offset;
    block->offset = offset + size;
    block->size -= size;
    alloc->max_size = MAX(alloc->max_size, offset + size);
    return offset;
}
```

[`ggml_gallocr_free_node`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L545)

```c++
static void ggml_gallocr_free_node(ggml_gallocr_t galloc, struct ggml_tensor * node) {
    // graph outputs are never freed
    if (node->flags & GGML_TENSOR_FLAG_OUTPUT) return;
    struct hash_node * hn = ggml_gallocr_hash_get(galloc, node);
    size_t offset = hn->offset;
    int buffer_id = hn->buffer_id;
    struct ggml_dyn_tallocr * alloc = galloc->buf_tallocs[buffer_id];
    ggml_backend_buffer_type_t buft = galloc->bufts[buffer_id];
    size_t size = ggml_backend_buft_get_alloc_size(buft, node);
    --> ggml_dyn_tallocr_free_tensor;
    hn->allocated = false;
}
```

[`ggml_dyn_tallocr_free_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L238)

```c++
ggml_dyn_tallocr_free_tensor(alloc, offset, size, node);
----------
// this is a very naive implementation, but for our case the number of free blocks should be very small
static void ggml_dyn_tallocr_free_tensor(struct ggml_dyn_tallocr * alloc, size_t offset, size_t size, const struct ggml_tensor * tensor) {
    size = aligned_offset(NULL, size, alloc->alignment);

    // see if we can merge with an existing block
    for (int i = 0; i < alloc->n_free_blocks; i++) {
        struct free_block * block = &alloc->free_blocks[i];
        // check if ptr is at the end of the block
        if (block->offset + block->size == offset) {
            block->size += size;
            // check if we can merge with the next block
            if (i < alloc->n_free_blocks - 1 && block->offset + block->size == alloc->free_blocks[i+1].offset) {
                block->size += alloc->free_blocks[i+1].size;
                alloc->n_free_blocks--;
                for (int j = i+1; j < alloc->n_free_blocks; j++) {
                    alloc->free_blocks[j] = alloc->free_blocks[j+1];
                }
            }
            return;
        }
        // check if ptr is at the beginning of the block
        if (offset + size == block->offset) {
            block->offset = offset;
            block->size += size;
            // check if we can merge with the previous block
            if (i > 0 && alloc->free_blocks[i-1].offset + alloc->free_blocks[i-1].size == block->offset) {
                alloc->free_blocks[i-1].size += block->size;
                alloc->n_free_blocks--;
                for (int j = i; j < alloc->n_free_blocks; j++) {
                    alloc->free_blocks[j] = alloc->free_blocks[j+1];
                }
            }
            return;
        }
    }
    // otherwise, add a new block
    // insert the new block in the correct position to keep the array sorted by address (to make merging blocks faster)
    int insert_pos = 0;
    while (insert_pos < alloc->n_free_blocks && alloc->free_blocks[insert_pos].offset < offset) {
        insert_pos++;
    }
    // shift all blocks from insert_pos onward to make room for the new block
    for (int i = alloc->n_free_blocks; i > insert_pos; i--) {
        alloc->free_blocks[i] = alloc->free_blocks[i-1];
    }
    // insert the new block
    alloc->free_blocks[insert_pos].offset = offset;
    alloc->free_blocks[insert_pos].size = size;
    alloc->n_free_blocks++;
}
```

## *[`llama_context::decode`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L880)

<details>
<summary>struct llama_batch</summary>

```c++
    // Input data for llama_decode
    // A llama_batch object can contain input about one or many sequences
    // The provided arrays (i.e. token, embd, pos, etc.) must have size of n_tokens
    //
    // - token  : the token ids of the input (used when embd is NULL)
    // - embd   : token embeddings (i.e. float vector of size n_embd) (used when token is NULL)
    // - pos    : the positions of the respective token in the sequence
    //            (if set to NULL, the token position will be tracked automatically by llama_decode)
    // - seq_id : the sequence to which the respective token belongs
    //            (if set to NULL, the sequence ID will be assumed to be 0)
    // - logits : if zero, the logits (and/or the embeddings) for the respective token will not be output
    //            (if set to NULL, only the logits for last token will be returned)
    //
    typedef struct llama_batch {
        int32_t n_tokens;

        llama_token  *  token;
        float        *  embd;
        llama_pos    *  pos;
        int32_t      *  n_seq_id; // TODO: remove, should belong to only 1 sequence
        llama_seq_id ** seq_id;   // TODO: become llama_seq_id * seq_id;
        int8_t       *  logits;   // TODO: rename this to "output"
    } llama_batch;


struct llama_batch llama_batch_get_one(
             llama_token * tokens,
                 int32_t   n_tokens) {
    return {
        /*n_tokens       =*/ n_tokens,
        /*tokens         =*/ tokens,
        /*embd           =*/ nullptr,
        /*pos            =*/ nullptr,
        /*n_seq_id       =*/ nullptr,
        /*seq_id         =*/ nullptr,
        /*logits         =*/ nullptr,
    };
}
```

</details>

```c++
llama_decode(lctx, llama_batch_get_one(tmp.data(), std::min(tmp.size(), (size_t) params.n_batch)));
// or the simpler one
llama_token token = llama_vocab_get_add_bos(vocab) ? llama_vocab_bos(vocab) : std::rand() % n_vocab;
int res = llama_decode(ctx, llama_batch_get_one(&token, 1));
----------
int32_t llama_decode(
        llama_context * ctx,
          llama_batch   batch) {
    --> const int ret = ctx->decode(batch);
    return ret;
}

int llama_context::decode(llama_batch & batch_inp) {}
```

```c++
--> llama_batch_allocr::init;
const llama_batch & batch = batch_allocr->get_batch();
const uint32_t n_tokens_all  = balloc->get_n_tokens();
const uint32_t n_outputs_all = balloc->get_n_outputs();
n_queued_tokens += n_tokens_all;
bool did_optimize = false;
// handle any pending defrags/shifts
--> llama_context::kv_self_update;
--> llama_kv_cache_unified::init_batch --> llama_memory_context_ptr mctx;
// reserve output buffer
// 第二次执行可能会被跳过内存alloc的部分
--> output_reserve(n_outputs_all) --> llama_context:: float * logits, ggml_backend_buffer_ptr buf_output;;
int64_t n_outputs_prev = 0;
do {
    const auto & ubatch = mctx->get_ubatch();
    // count the outputs in this u_batch
    int32_t n_outputs_new = 0;
    if (n_outputs_all == n_tokens_all) n_outputs_new = ubatch.n_tokens;
    else for (uint32_t i = 0; i < ubatch.n_tokens; i++) n_outputs_new += (int32_t) (ubatch.output[i] != 0);
    // needs to happen before the graph is built
    n_outputs = n_outputs_new;
    ggml_backend_sched_reset(sched.get());
    --> ggml_backend_sched_set_eval_callback(sched.get(), cparams.cb_eval, cparams.cb_eval_user_data);
        sched->callback_eval = callback;
        sched->callback_eval_user_data = user_data;
    --> llama_context::process_ubatch --> const auto res;
    // plot the computation graph in dot format (for debugging purposes)
    //if (n_past%100 == 0) ggml_graph_dump_dot(gf, NULL, "llama.dot");
    auto * t_logits = res->get_logits();
    // extract logits
    ggml_backend_t backend_res = ggml_backend_sched_get_tensor_backend(sched.get(), t_logits);
    float * logits_out = logits + n_outputs_prev*n_vocab;
    --> ggml_backend_tensor_get_async --> logits_out
    n_outputs_prev += n_outputs;
} while (mctx->next());
// set to total number of outputs in the batch, for use in llama_get_logits_ith
n_outputs = n_outputs_all;
// set output mappings
// if (n_outputs > 0)
auto & out_ids = balloc->get_out_ids();
//llama_context:: std::vector<int32_t> output_ids; // map batch token positions to ids of the logits and embd buffers
for (int64_t i = 0; i < n_outputs_all; ++i) output_ids[out_ids[i]] = i;
// wait for the computation to finish (automatically done when obtaining the model output)
//synchronize();
// Reset state for the next token before backend sync, to allow the CPU activities in the reset to overlap with device computation.
ggml_backend_sched_reset(sched.get());
```

### [`llama_batch_allocr::init`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L25)

<details>
<summary>class llama_batch_allocr</summary>

```c++
typedef int32_t llama_pos;
typedef int32_t llama_seq_id;

class llama_batch_allocr {
    llama_batch batch;

    uint32_t n_outputs;

    std::array<llama_seq_id, 1> seq_id_0 = { 0 }; // default sequence id

    std::vector<llama_pos>      pos;
    std::vector<int32_t>        n_seq_id;
    std::vector<llama_seq_id *> seq_id;
    std::vector<int8_t>         output;

    std::vector<std::set<llama_pos>> seq_pos; // seq_pos[s]: the set of positions in sequence s
    std::vector<std::vector<bool>>   seq_cpl; // seq_cpl[s0][s1]: if sequence s0 is coupled to sequence s1

    // batch indices of the output
    std::vector<int32_t> out_ids;

    // used[i] indicates if token i has already been used in a previous ubatch
    std::vector<bool> used;

    // llama_ubatch points to this data:
    struct ubatch {
        std::vector<llama_token>    token;
        std::vector<float>          embd;
        std::vector<llama_pos>      pos;
        std::vector<int32_t>        n_seq_id;
        std::vector<llama_seq_id *> seq_id;
        std::vector<llama_seq_id>   seq_id_unq;
        std::vector<int32_t>        seq_idx;
        std::vector<int8_t>         output;
    };

    // current splitting state:
    std::vector<ubatch> ubatches;
};
```

</details>

```c++
balloc->init(batch_inp, vocab, memory.get(), n_embd, output_all);
----------
bool llama_batch_allocr::init(
        const llama_batch & batch_inp,
        const llama_vocab & vocab,
        const llama_memory_i * memory,
        uint32_t n_embd,
        bool output_all) {}
```

```c++
--> clear();
    n_outputs = 0;
    batch = {};
    pos       .clear();
    n_seq_id  .clear();
    seq_id    .clear();
    seq_id_unq.clear();
    output    .clear();
    for (auto & cur : seq_pos) cur.clear();
    for (auto & cur : seq_cpl) std::fill(cur.begin(), cur.end(), false);
    seq_set.clear();
    seq_set_map.clear();
    std::fill(seq_idx.begin(), seq_idx.end(), -1);
batch = batch_inp;
// validate input batch
// auto-generate missing fields
if (!batch.n_seq_id) {
    n_seq_id.resize(batch.n_tokens);
    for (int32_t i = 0; i < batch.n_tokens; i++) n_seq_id[i] = seq_id_0.size();
    batch.n_seq_id = n_seq_id.data();
}
if (!batch.seq_id) {
    seq_id.resize(batch.n_tokens + 1);
    seq_id[batch.n_tokens] = NULL;
    for (int32_t i = 0; i < batch.n_tokens; i++) seq_id[i] = seq_id_0.data();
    batch.seq_id = seq_id.data();
}
if (!batch.pos) {
    pos.resize(batch.n_tokens);
    // initialize the starting position for each sequence based on the positions in the memory
    llama_pos p0[LLAMA_MAX_SEQ];
    for (int32_t s = 0; s < LLAMA_MAX_SEQ; ++s) p0[s] = memory? memory->seq_pos_max(s) + 1 : 0;
    for (int32_t i = 0; i < batch.n_tokens; i++) {
        const llama_seq_id seq_id = batch.seq_id[i][0];
        pos[i] = p0[seq_id];
        for (int32_t s = 0; s < batch.n_seq_id[i]; ++s) p0[batch.seq_id[i][s]] = pos[i] + 1;
    }
    batch.pos = pos.data();
}
if (!batch.logits) {
    // return the output only for the last token
    output.resize(batch.n_tokens, false);
    output[output.size() - 1] = true;
    batch.logits = output.data();
}
// compute stats
for (int32_t i = 0; i < batch.n_tokens; ++i) n_outputs += batch.logits[i] != 0;
// determine coupled sequences these are pairs of sequences that have at least one token in the input batch that is assigned to both of them
for (int32_t i = 0; i < batch.n_tokens; ++i)
    for (int32_t s = 0; s < batch.n_seq_id[i]; ++s)
        seq_pos[batch.seq_id[i][s]].insert(batch.pos[i]);
        // mark that sequence s1 is coupled to s0
        if (s > 0) seq_cpl[batch.seq_id[i][s]][batch.seq_id[i][0]] = true;
// precompute the sequence sets for each token and determine the unique sequence ids that participate in the batch
seq_set_t seq_set_unq;
for (int32_t i = 0; i < batch.n_tokens; ++i)
    seq_set_t cur;
    for (int32_t s = 0; s < batch.n_seq_id[i]; ++s)
        const llama_seq_id seq_id = batch.seq_id[i][s];
        cur        .set(seq_id);
        seq_set_unq.set(seq_id);
    seq_set.push_back(cur);
    seq_set_map[cur].push_back(i);
for (int32_t s = 0; s < LLAMA_MAX_SEQ; ++s)
    if (seq_set_unq.test(s))
        seq_idx[s] = seq_id_unq.size();
        seq_id_unq.push_back(s);
// consistency checks
// disallow partial sequence sub-sets:
--> split_reset();
    out_ids.clear();
    used.clear();
    used.resize(get_n_tokens(), false);
    ubatches.clear();
return true;
```

### [`llama_context::kv_self_update`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L437)


```c++
kv_self_update(false);
----------
// deprecated
bool llama_context::kv_self_update(bool optimize) {}
```

```c++
// TODO: remove in the future
optimize |= memory_force_optimize; //false
memory_force_optimize = false;

--> memory --> llama_kv_cache_unified::init_update --> const auto mctx;
switch (mctx->get_status())
case LLAMA_MEMORY_STATUS_NO_UPDATE: return false; // no updates need to be performed
```

[`lama_kv_cache_unified::init_update`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L340)

```c++
const auto mstate = memory->init_update(this, optimize);
----------
llama_memory_state_ptr llama_kv_cache_unified::init_update(llama_context * lctx, bool optimize) {
    bool do_shift = get_has_shift();
    defrag_info dinfo;
    --> return std::make_unique<llama_kv_cache_unified_state>(this, lctx, do_shift, std::move(dinfo));
}
```

[`llama_kv_cache_unified_state::llama_kv_cache_unified_state`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L1737)

```c++
llama_kv_cache_unified_state::llama_kv_cache_unified_state(
        llama_kv_cache_unified * kv,
        llama_context * lctx,
        bool do_shift,
        defrag_info dinfo) : status(LLAMA_MEMORY_STATUS_SUCCESS), kv(kv), lctx(lctx), do_shift(do_shift), dinfo(std::move(dinfo)) {
    if (!do_shift && this->dinfo.empty()) {
        status = LLAMA_MEMORY_STATUS_NO_UPDATE;
    }
}
```

### [`llama_kv_cache_unified::init_batch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L310)

```c++
mctx = memory->init_batch(*balloc, cparams.n_ubatch, output_all);
----------
llama_memory_context_ptr llama_kv_cache_unified::init_batch(
            llama_batch_allocr & balloc,
            uint32_t n_ubatch,
            bool embd_all) {}
```

```c++
balloc.split_reset();
std::vector<llama_ubatch> ubatches;
while (true) {
    -->llama_batch_allocr::split_simple --> llama_ubatch ubatch;
    if (ubatch.n_tokens == 0) break;
    std::vector<llama_ubatch> ubatches.push_back(std::move(ubatch)); // NOLINT
}
--> llama_kv_cache_unified::prepare --> llama_kv_cache_unified::ubatch_heads heads;
return std::make_unique<llama_kv_cache_unified_context>(
        this, std::move(heads), std::move(ubatches));
--> llama_kv_cache_unified_context::llama_kv_cache_unified_context(
    llama_kv_cache_unified * kv,
    llama_kv_cache_unified::ubatch_heads heads,
    std::vector<llama_ubatch> ubatches) : status(LLAMA_MEMORY_STATUS_SUCCESS), kv(kv), heads(std::move(heads)), ubatches(std::move(ubatches)) {}
```

#### [`llama_sbatch::split_simple`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L149)

```c++
ubatches.push_back(sbatch.split_simple(n_ubatch));
----------
llama_ubatch llama_sbatch::split_simple(size_t n_ubatch) {}
```

```c++
// find the first unused token
uint32_t cur_idx = 0;
while (cur_idx < used.size() && used[cur_idx]) ++cur_idx;
// we are done
if (cur_idx >= used.size()) return {};
std::vector<int32_t> idxs;
while (true)
    idxs.push_back(cur_idx);
    used[cur_idx] = true;
    ++cur_idx;
    if (cur_idx >= used.size()) break;
    if (idxs.size() >= n_ubatch) break;
--> return ubatch_add(idxs, idxs.size(), false);
```

[`llama_batch_allocr::ubatch_add`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L605)

```c++
llama_ubatch llama_batch_allocr::ubatch_add(const std::vector<int32_t> & idxs, uint32_t n_seqs, bool equal_seqs) {
    const uint32_t n_tokens = idxs.size();
    ubatches.emplace_back();
    auto & ubatch = ubatches.back();
    const int32_t n_pos_cur = 1;
    const int64_t n_pos_all  = (int64_t) n_tokens*n_pos_cur;
    ubatch.token     .resize(n_tokens);
    ubatch.embd      .resize(n_embd_all);
    ubatch.pos       .resize(n_pos_all);
    ubatch.n_seq_id  .resize(n_tokens);
    ubatch.seq_id    .resize(n_tokens);
    ubatch.seq_id_unq.resize(0);
    ubatch.seq_idx   .resize(LLAMA_MAX_SEQ, -1);
    ubatch.output    .resize(n_tokens);
    seq_set_t seq_set_unq;
    for (size_t i = 0; i < idxs.size(); ++i) {
        if (batch.token) ubatch.token[i] = batch.token[idxs[i]];
        for (int j = 0; j < n_pos_cur; ++j) ubatch.pos[j*n_tokens + i] = batch.pos[j*batch.n_tokens + idxs[i]];
        ubatch.n_seq_id[i] = batch.n_seq_id[idxs[i]];
        ubatch.seq_id[i]   = batch.seq_id[idxs[i]];
        ubatch.output[i]   = batch.logits[idxs[i]];
        for (int s = 0; s < ubatch.n_seq_id[i]; ++s) seq_set_unq.set(ubatch.seq_id[i][s]);
        if (ubatch.output[i]) out_ids.push_back(idxs[i]);
    }
    for (int32_t s = 0; s < LLAMA_MAX_SEQ; ++s) {
        if (seq_set_unq.test(s)) {
            ubatch.seq_idx[s] = ubatch.seq_id_unq.size();
            ubatch.seq_id_unq.push_back(s);
        }
    }
    llama_ubatch res {
        /*.equal_seqs   =*/ equal_seqs,
        /*.n_tokens     =*/ n_tokens,
        /*.n_seq_tokens =*/ n_tokens/n_seqs,
        /*.n_seqs       =*/ n_seqs,
        /*.n_seqs_unq   =*/ (uint32_t) ubatch.seq_id_unq.size(),
        /*.token        =*/ batch.token ? ubatch.token.data() : nullptr,
        /*.embd         =*/ batch.embd ? ubatch.embd.data() : nullptr,
        /*.pos          =*/ ubatch.pos.data(),
        /*.n_seq_id     =*/ ubatch.n_seq_id.data(),
        /*.seq_id       =*/ ubatch.seq_id.data(),
        /*.seq_id_unq   =*/ ubatch.seq_id_unq.data(),
        /*.seq_idx      =*/ ubatch.seq_idx.data(),
        /*.output       =*/ ubatch.output.data(),
    };
    return res;
}
```

#### [`llama_kv_cache_unified::prepare`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L373)

```c++
auto heads = prepare(ubatches);
----------
llama_kv_cache_unified::ubatch_heads llama_kv_cache_unified::prepare(const std::vector<llama_ubatch> & ubatches) {}
```

<details>
<summary>llama_kv_cache_unified::ubatch_heads</summary>

```c++
using llama_kv_cache_unified::ubatch_heads = std::vector<uint32_t>;
```

</details>


<details>
<summary>struct state</summary>

```c++
struct state {
    uint32_t head_old; // old position of the head, before placing the ubatch
    uint32_t head_new; // new position of the head, after placing the ubatch

    llama_kv_cells_unified cells; // copy of the old cells, before placing the ubatch
};
```

</details>


```c++
llama_kv_cache_unified::ubatch_heads res;

// remember the old state of the cells so we can restore it in the end
std::vector<state> states;
//for (const auto & ubatch : ubatches)
// only find a suitable slot for the ubatch. don't modify the cells yet
--> llama_kv_cache_unified::find_slot ---> const int32_t head_new;
// remeber the position that we found
res.push_back(head_new);
// store the old state of the cells in the recovery stack
--> states.push_back({head, (uint32_t) head_new, cells.cp(head_new, ubatch.n_tokens)});
// 意思就是打上标记，此地被占用了，for循环的下一次find_slot不会找上这一块
// now emplace the ubatch
--> llama_kv_cache_unified::apply_ubatch;
// iterate backwards and restore the cells to their original state
for (auto it = states.rbegin(); it != states.rend(); ++it)
    --> cells.set(it->head_new, it->cells);
    head = it->head_old;
return res;
```

##### [`llama_kv_cache_unified::find_slot`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L510)

```c++
const int32_t head_new = find_slot(ubatch);
----------
int32_t llama_kv_cache_unified::find_slot(const llama_ubatch & ubatch) const {}
```

```c++
const uint32_t n_tokens = ubatch.n_tokens;
uint32_t head_cur = this->head;
// if we have enough unused cells before the current head ->
// better to start searching from the beginning of the cache, hoping to fill it
if (head_cur > cells.get_used() + 2*n_tokens) head_cur = 0;
while (true) {
    if (head_cur + n_tokens > cells.size())
        head_cur = 0;
        continue;
    bool found = true;
    for (uint32_t i = 0; i < n_tokens; i++) {
        // can we use this cell? either:
        //- the cell is empty
        //- the cell is occupied only by one sequence: <-why?:/
        // - (disabled) mask causally, if the sequence is the same as the one we are inserting
        // - mask SWA, using current max pos for that sequence in the cache, always insert in the cell with minimum pos
        bool can_use = cells.is_empty(head_cur + i);
        if (!can_use && cells.seq_count(head_cur + i) == 1)
            const llama_pos pos_cell = cells.pos_get(head_cur + i);
            const llama_seq_id seq_id_cell = cells.seq_get(head_cur + i);
            // (disabled) causal mask
            // note: it's better to purge any "future" tokens beforehand
            //if (cells.seq_has(head_cur + i, seq_id)) {
            //    can_use = pos_cell >= pos;
            //}
            // SWA mask
            if (!can_use && is_masked_swa(pos_cell, cells.seq_pos_max(seq_id_cell) + 1)) can_use = true;
        if (!can_use)
            found = false;
            head_cur += i + 1;
            break;
    }
    if (found) break;
}
return head_cur;
```

##### [`llama_kv_cache_unified::apply_ubatch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L646)

```c++
apply_ubatch(head_new, ubatch);
----------
void llama_kv_cache_unified::apply_ubatch(uint32_t head_cur, const llama_ubatch & ubatch) {}
```

```c++
// keep track of the max sequence position that we would overwrite with this ubatch for non-SWA cache, this would be always empty
llama_seq_id seq_pos_max_rm[LLAMA_MAX_SEQ];
for (int s = 0; s < LLAMA_MAX_SEQ; ++s) seq_pos_max_rm[s] = -1;
for (uint32_t i = 0; i < ubatch.n_tokens; ++i)
    if (!cells.is_empty(head_cur + i)); // 一般不会执行到
    cells.pos_set(i=head_cur + i, p=ubatch.pos[i]);
    --> pos[i] = p;
        used.insert(i);
    for (int32_t s = 0; s < ubatch.n_seq_id[i]; s++)
        cells.seq_add(i=head_cur + i, seq_id=ubatch.seq_id[i][s]);
        --> seq[i].set(seq_id);
            seq_pos[seq_id].insert(pos[i]);
// note: we want to preserve the invariant that all positions between [pos_min, pos_max] for each sequence
//       will be present in the cache. so we have to purge any position which is less than those we would overwrite
//       ref: https://github.com/ggml-org/llama.cpp/pull/13746#issuecomment-2916057092
for (int s = 0; s < LLAMA_MAX_SEQ; ++s)
    if (seq_pos_max_rm[s] == -1) continue;
    if (cells.seq_pos_min(s) <= seq_pos_max_rm[s]); // 一般不会执行到

// move the head at the end of the slot
head = head_cur + ubatch.n_tokens;
```

### **[`llama_context::process_ubatch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L681)

```c++
const auto res = process_ubatch(ubatch, LLM_GRAPH_TYPE_DECODER, mctx.get(), status);
----------
llm_graph_result_ptr llama_context::process_ubatch(const llama_ubatch & ubatch, llm_graph_type gtype, llama_memory_context_i * mctx, ggml_status & ret) {}
```

```c++
--> llama_kv_cache_unified_context::apply;
--> graph_init --> ggml_cgraph * gf;
// auto res = graph_build(ctx_compute.get(), gf, ubatch, gtype, mctx);
--> llama_context::graph_build --> llama_model::build_graph --> llm_build_llama::llm_build_llama --> std::unique_ptr<llm_graph_result> res;
// LLAMA_LOG_INFO("graph build time: %.3f ms (%d nodes, %d leafs)\n", (ggml_time_us() - t_start_us)/1000.0, gf->n_nodes, gf->n_leafs);
--> ggml_backend_sched_alloc_graph;
--> ggml_backend_cpu_buffer_set_tensor;/llama_kv_cache_unified::set_input_kq_mask;
--> llama_context::graph_compute --> ggml_backend_sched_graph_compute_async --> ggml_status ggml_backend_sched_compute_splits --> ggml_backend_graph_compute_async --> ggml_backend_cpu_graph_compute;
ret = GGML_STATUS_SUCCESS;
return res;
```

#### [`llama_kv_cache_unified_context::apply`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L1753)

> 不同于llama_kv_cache_unified::prepare, 这回是真的起作用了

```c++
if (mctx && !mctx->apply()) {}
----------
llm_graph_result_ptr llama_context::process_ubatch(const llama_ubatch & ubatch, llm_graph_type gtype, llama_memory_context_i * mctx, ggml_status & ret) {}
```

```c++
// no ubatches -> this is a KV cache update
if (ubatches.empty()) {
    kv->update(lctx, do_shift, dinfo);
    return true;
}
--> kv->apply_ubatch(heads[i_next], ubatches[i_next]);
n_kv = kv->get_n_kv(); // 更新n_kv的值，非常关键！ *key*
head = heads[i_next];
return true;
```

#### [`ggml_gallocr_init_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L787)

[`ggml_backend_sched_alloc_graph`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1565)

```c++
ggml_backend_sched_alloc_graph(sched.get(), gf);
----------
bool ggml_backend_sched_alloc_graph(ggml_backend_sched_t sched, struct ggml_cgraph * graph) {
    --> ggml_backend_sched_split_graph(sched, graph);
    --> ggml_backend_sched_alloc_splits(sched);
    sched->is_alloc = true;
    return true;
}
```

[`ggml_backend_sched_alloc_splits`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1321)

```c++
static bool ggml_backend_sched_alloc_splits(ggml_backend_sched_t sched) {
    bool backend_ids_changed = false;
    for (int i = 0; i < sched->graph.n_nodes; i++)
        if (sched->node_backend_ids[i] != sched->prev_node_backend_ids[i] &&
            sched->bufts[sched->node_backend_ids[i]] != sched->bufts[sched->prev_node_backend_ids[i]])
            backend_ids_changed = true;
            break;
    if (!backend_ids_changed)
    for (int i = 0; i < sched->graph.n_leafs; i++)
        if (sched->leaf_backend_ids[i] != sched->prev_leaf_backend_ids[i] &&
            sched->bufts[sched->leaf_backend_ids[i]] != sched->bufts[sched->prev_leaf_backend_ids[i]])
            backend_ids_changed = true;
            break;
    // allocate graph
    // 一般情况下是不会进入if里面的语句的
    --> if (backend_ids_changed || !ggml_gallocr_alloc_graph(sched->galloc, &sched->graph))
        // the re-allocation may cause the split inputs to be moved to a different address
        // synchronize without ggml_backend_sched_synchronize to avoid changing cur_copy
        --> for (int i = 0; i < sched->n_backends; i++) ggml_backend_synchronize(sched->backends[i]);
        --> ggml_gallocr_reserve_n(sched->galloc, &sched->graph, sched->node_backend_ids, sched->leaf_backend_ids);
        --> ggml_gallocr_alloc_graph(sched->galloc, &sched->graph)
    return true;
}
```

[`ggml_gallocr_alloc_graph`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L871)

> 真正开始给每个tensor的 buffer 和 data 域

```c++
bool ggml_gallocr_alloc_graph(ggml_gallocr_t galloc, struct ggml_cgraph * graph) {
    // 一般不会进入
    if (ggml_gallocr_needs_realloc(galloc, graph));
    // reset buffers // 一般都是满足的, 但是CPU和GPU的reset函数指针都为空
    for (int i = 0; i < galloc->n_buffers; i++);
    // allocate the graph tensors from the previous assignments
    // leafs
    for (int i = 0; i < graph->n_leafs; i++) {
        struct ggml_tensor * leaf = graph->leafs[i];
        struct leaf_alloc * leaf_alloc = &galloc->leaf_allocs[i];
        --> ggml_gallocr_init_tensor(galloc, leaf, &leaf_alloc->leaf);
    }
    // nodes
    for (int i = 0; i < graph->n_nodes; i++) {
        struct ggml_tensor * node = graph->nodes[i];
        struct node_alloc * node_alloc = &galloc->node_allocs[i];
        for (int j = 0; j < GGML_MAX_SRC; j++) {
            struct ggml_tensor * src = node->src[j];
            if (src == NULL) continue;
            --> ggml_gallocr_init_tensor(galloc, src, &node_alloc->src[j]);
        }
        --> ggml_gallocr_init_tensor(galloc, node, &node_alloc->dst);
    }
    return true;
}
```

```c++
static void ggml_gallocr_init_tensor(ggml_gallocr_t galloc, struct ggml_tensor * tensor, struct tensor_alloc * tensor_alloc) {
    int buffer_id = tensor_alloc->buffer_id;
    if (tensor->view_src != NULL && tensor->buffer == NULL && tensor->view_src->buffer != NULL)
        --> ggml_backend_view_init(tensor);
    if (tensor->view_src == NULL && tensor->data == NULL)
        void * base = ggml_backend_buffer_get_base(galloc->buffers[buffer_id]);
        void * addr = (char *)base + tensor_alloc->offset;
        // 分配地址空间 *key*
        --> ggml_backend_tensor_alloc(galloc->buffers[buffer_id], tensor, addr);
            --> tensor->buffer = buffer; tensor->data = addr;
                // 最后的 ggml_backend_buffer_init_tensor 在 CPU, CUDA_HOST上没有对应的函数
                // CUDA上上有，但是因为这个buffer的usaga是GGML_BACKEND_BUFFER_USAGE_COMPUTE，所以不起作用。
}
```

#### [`ggml_backend_cpu_buffer_set_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1890)

```c++
res->set_inputs(&ubatch);
llm_graph_result::set_inputs -->
    for (auto & input : inputs) input->set_input(ubatch);
    llm_graph_input_embd::set_input;
    llm_graph_input_pos::set_input;

ggml_backend_tensor_set(llm_graph_input_embd:: ggml_tensor * tokens, ubatch->token, 0, n_tokens*ggml_element_size(tokens));
ggml_backend_tensor_set(llm_graph_input_pos:: ggml_tensor * pos, ubatch->pos, 0, n_tokens*n_pos_per_embd*ggml_element_size(pos));
----------
void ggml_backend_tensor_set(struct ggml_tensor * tensor, const void * data, size_t offset, size_t size) {
    ggml_backend_buffer_t buf = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
    if (size == 0) {
        return;
    }
    --> buf->iface.set_tensor(buf, tensor, data, offset, size);
}

static void ggml_backend_cpu_buffer_set_tensor(ggml_backend_buffer_t buffer, struct ggml_tensor * tensor, const void * data, size_t offset, size_t size)
    memcpy((char *)tensor->data + offset, data, size);
```

```log
>>> p *ubatch->pos
$1817 = 0
>>> p *ubatch->token 
$1818 = 128000
```

#### [`llama_kv_cache_unified::set_input_kq_mask`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L794)

```c++
res->set_inputs(&ubatch);
llm_graph_result::set_inputs -->
    for (auto & input : inputs) input->set_input(ubatch);
----------
void llm_graph_input_attn_kv_unified::set_input(const llama_ubatch * ubatch) {
    --> kv_state->set_input_kq_mask(llama_kv_cache_unified:: ggml_tensor * self_kq_mask, ubatch, cparams.causal_attn);
}
void llama_kv_cache_unified_state::set_input_kq_mask(ggml_tensor * dst, const llama_ubatch * ubatch, bool causal_attn) const {
    --> kv->set_input_kq_mask(dst, ubatch, causal_attn);
}
void llama_kv_cache_unified::set_input_kq_mask(ggml_tensor * dst, const llama_ubatch * ubatch, bool causal_attn) const {}
```

```c++
const uint32_t n_tokens     = ubatch->n_tokens;
float * data = (float *) dst->data;
GGML_ASSERT(ggml_backend_buffer_is_host(dst->buffer));
const int64_t n_kv = dst->ne[0];
// Use only the previous KV cells of the correct sequence for each token of the ubatch.
// It's assumed that if a token in the batch has multiple sequences, they are equivalent.
// Example with a cache of 10 tokens, 2 tokens populated in cache and 3 tokens in batch:
//   Causal mask:
//      xxx-------
//      xxxx------
//      xxxxx-----
//   Non-causal mask:
//      xxxxx-----
//      xxxxx-----
//      xxxxx-----
// To visualize the mask, see https://github.com/ggml-org/llama.cpp/pull/12615
for (uint32_t s = 0; s < n_seqs; ++s) {
    const llama_seq_id seq_id = ubatch->seq_id[s][0];
    // for (uint32_t j = 0; j < n_seq_tokens; ++j)
        // const uint32_t idx = s*n_seq_tokens + j;
    const llama_pos p1 = ubatch->pos[s];
    for (uint32_t i = 0; i < n_kv; ++i) {
        float f = 0.0f;
        bool masked = false;
        if (cells.is_empty(i)) {
            masked = true;
        } else {
            const llama_pos p0 = cells.pos_get(i);
            // mask the token if not the same sequence
            masked = masked || (!cells.seq_has(i, seq_id));
            // mask future tokens
            masked = masked || (causal_attn && p0 > p1);
            // apply SWA if any
            masked = masked || (is_masked_swa(p0, p1));
            if (!masked && hparams.use_alibi) f = -std::abs(p0 - p1);
        }
        if (masked) f = -INFINITY;
        data[n_kv*n_tokens + s*n_kv + i] = f;
    }
}

// mask padded tokens
if (data) {
    for (uint32_t j = n_tokens; j < GGML_PAD(n_tokens, GGML_KQ_MASK_PAD); ++j) {
        for (uint32_t i = 0; i < n_kv; ++i) {
            data[n_kv*n_tokens + j*n_kv + i] = -INFINITY;
        }
    }
}
```

#### [`llm_graph_input_out_ids::set_input`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L94)


```c++
void llm_graph_input_out_ids::set_input(const llama_ubatch * ubatch) {
    const int64_t n_tokens = ubatch->n_tokens;
    ggml_backend_buffer_is_host(out_ids->buffer);
    int32_t * data = (int32_t *) out_ids->data;
    if (n_outputs == n_tokens)
        for (int i = 0; i < n_tokens; ++i) data[i] = i;
        return;
    int n_outputs = 0;
    for (int i = 0; i < n_tokens; ++i)
        if (ubatch->output[i]) data[n_outputs++] = i;
}
```

#### ***[`ggml_backend_sched_compute_splits`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1360)

> 除了少数特殊情况，一般都是只靠的sched split完之后生成的动态的计算图

- [`llama_context::graph_compute`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1369)

```c++
const auto status = graph_compute(gf, ubatch.n_tokens > 1);
----------
ggml_status llama_context::graph_compute(
            ggml_cgraph * gf,
                   bool   batched) {
    int n_threads        = batched ? cparams.n_threads_batch : cparams.n_threads;
    
    ggml_threadpool_t tp = batched ? threadpool_batch        : threadpool;
    auto * reg = ggml_backend_dev_backend_reg(ggml_backend_get_device(backend_cpu));
    auto * set_threadpool_fn = (decltype(ggml_backend_cpu_set_threadpool) *) ggml_backend_reg_get_proc_address(reg, "ggml_backend_cpu_set_threadpool");
    set_threadpool_fn(backend_cpu, tp);
        void ggml_backend_cpu_set_threadpool(ggml_backend_t backend_cpu, ggml_threadpool_t threadpool) {}
        --> ctx->threadpool = threadpool;
    // set the number of threads for all the backends
    for (const auto & set_n_threads_fn : set_n_threads_fns)
        set_n_threads_fn.second(set_n_threads_fn.first, n_threads);
            void ggml_backend_cpu_set_n_threads(ggml_backend_t backend_cpu, int n_threads) {}
    
    --> auto status = ggml_backend_sched_graph_compute_async(sched.get(), gf);
    return status;
}

enum ggml_status ggml_backend_sched_graph_compute_async(ggml_backend_sched_t sched, struct ggml_cgraph * graph) {
    // 这两个条件在decode里都不会被满足，也就是说看是靠的sched split完之后生成的动态的计算图
    // if (!sched->is_reset && !sched->is_alloc) ggml_backend_sched_reset(sched);
    // if (!sched->is_alloc) if (!ggml_backend_sched_alloc_graph(sched, graph)) return GGML_STATUS_ALLOC_FAILED;
    --> return ggml_backend_sched_compute_splits(sched);
}

static enum ggml_status ggml_backend_sched_compute_splits(ggml_backend_sched_t sched) {
    struct ggml_backend_sched_split * splits = sched->splits;
    for (int i = 0; i < sched->n_splits; i++)
        struct ggml_backend_sched_split * split = &splits[i];
        int split_backend_id = split->backend_id;
        ggml_backend_t split_backend = sched->backends[split_backend_id];
        // copy the input tensors to the split backend
        for (int j = 0; j < split->n_inputs; j++) {
            ggml_backend_t input_backend = ggml_backend_sched_get_tensor_backend(sched, split->inputs[j]);
            struct ggml_tensor * input = split->inputs[j];
            struct ggml_tensor * input_cpy = tensor_copy(input, split_backend_id, sched->cur_copy);

            if (input->flags & GGML_TENSOR_FLAG_INPUT)
                // inputs from the user must be copied immediately to prevent the user overwriting the data before the copy is done
                ggml_backend_synchronize(split_backend);
                --> ggml_backend_tensor_copy(input, input_cpy);
            else
                // wait for the split backend to finish using the input before overwriting it
                ggml_backend_synchronize(split_backend);
                // try async copy, but if not possible, we can still use a sync copy without synchronizing the dst backend, since we handle the synchronization here with multiple copies and events
                // TODO: add public function to facilitate this, since applications do not have direct access to the backend interface
                if (!split_backend->iface.cpy_tensor_async ||
                // 貌似不work！
                --> !split_backend->iface.cpy_tensor_async(input_backend, split_backend, input, input_cpy))
                    ggml_backend_synchronize(input_backend);
                    ggml_backend_synchronize(split_backend);
                    --> ggml_backend_tensor_copy(input, input_cpy);
        }

        if (!sched->callback_eval)
            --> ggml_backend_graph_compute_async(split_backend, &split->graph);
        // similar to ggml_backend_compare_graph_backend
        else for (int j0 = 0; j0 < split->graph.n_nodes; j0++) {
            struct ggml_tensor * t = split->graph.nodes[j0];
            // check if the user needs data from this node
            bool need = sched->callback_eval(t, true, sched->callback_eval_user_data);
            int j1 = j0;
            // determine the range [j0, j1] of nodes that can be computed together
            while (!need && j1 < split->graph.n_nodes - 1) need = sched->callback_eval(split->graph.nodes[++j1], true, sched->callback_eval_user_data);
            struct ggml_cgraph gv = ggml_graph_view(&split->graph, j0, j1 + 1);
            --> ggml_backend_graph_compute_async(split_backend, &gv);
            // TODO: pass backend to the callback, then the user can decide if they want to synchronize
            ggml_backend_synchronize(split_backend);
            if (need) sched->callback_eval(t, false, sched->callback_eval_user_data);
            j0 = j1;
        }
        // record the event of this copy
        // 我们不会用到 events 变量
        // if (split->n_inputs > 0 && sched->events[split_backend_id][sched->cur_copy] != NULL) {}
    // 我们并不关心，不起作用
    // sched->cur_copy = (sched->cur_copy + 1) % sched->n_copies;
    return GGML_STATUS_SUCCESS;
}

enum ggml_status ggml_backend_graph_compute_async(ggml_backend_t backend, struct ggml_cgraph * cgraph) {
    --> return backend->iface.graph_compute(backend, cgraph);
        --> ggml_backend_cpu_graph_compute;
        --> ggml_backend_cuda_graph_compute;
}
```

```log
>>> p sched->backends[0]->iface
$1883 = {
  get_name = 0x7fffe6d38049 <ggml_backend_cuda_get_name(ggml_backend_t)>,
  free = 0x7fffe6d38077 <ggml_backend_cuda_free(ggml_backend_t)>,
set_tensor_async = <ggml_backend_cuda_set_tensor_async(ggml_backend_t, ggml_tensor*, void const*, size_t, size_t)>,
get_tensor_async = <ggml_backend_cuda_get_tensor_async(ggml_backend_t, ggml_tensor const*, void*, size_t, size_t)>,
cpy_tensor_async = <ggml_backend_cuda_cpy_tensor_async(ggml_backend_t, ggml_backend_t, ggml_tensor const*, ggml_tensor*)>,
  synchronize = 0x7fffe6d3879f <ggml_backend_cuda_synchronize(ggml_backend_t)>,
  graph_plan_create = 0x0,
  graph_plan_free = 0x0,
  graph_plan_update = 0x0,
  graph_plan_compute = 0x0,
  graph_compute = <ggml_backend_cuda_graph_compute(ggml_backend_t, ggml_cgraph*)>,
  event_record = <ggml_backend_cuda_event_record(ggml_backend_t, ggml_backend_event_t)>,
  event_wait = <ggml_backend_cuda_event_wait(ggml_backend_t, ggml_backend_event_t)>
}

>>> p sched->splits[2]->inputs[0]->buffer->iface
$1886 = {
  free_buffer = 0x7fffe6d2f0f2 <ggml_backend_cuda_buffer_free_buffer(ggml_backend_buffer_t)>,
  get_base = 0x7fffe6d2f156 <ggml_backend_cuda_buffer_get_base(ggml_backend_buffer_t)>,
  init_tensor = 0x7fffe6d2f178 <ggml_backend_cuda_buffer_init_tensor(ggml_backend_buffer_t, ggml_tensor*)>,
  memset_tensor = 0x7fffe6d2f2f1 <ggml_backend_cuda_buffer_memset_tensor(ggml_backend_buffer_t, ggml_tensor*, uint8_t, size_t, size_t)>,
set_tensor = <ggml_backend_cuda_buffer_set_tensor(ggml_backend_buffer_t, ggml_tensor*, void const*, size_t, size_t)>,
get_tensor = <ggml_backend_cuda_buffer_get_tensor(ggml_backend_buffer_t, ggml_tensor const*, void*, size_t, size_t)>,
cpy_tensor = <ggml_backend_cuda_buffer_cpy_tensor(ggml_backend_buffer_t, ggml_tensor const*, ggml_tensor*)>,
  clear = 0x7fffe6d2f76b <ggml_backend_cuda_buffer_clear(ggml_backend_buffer_t, uint8_t)>,
  reset = 0x0
}

>>> p sched->backends[1]->iface
$1882 = {
  get_name = 0x7ffff7743276 <ggml_backend_cpu_get_name(ggml_backend_t)>,
  free = 0x7ffff774328b <ggml_backend_cpu_free(ggml_backend_t)>,
  set_tensor_async = 0x0,
  get_tensor_async = 0x0,
  cpy_tensor_async = 0x0,
  synchronize = 0x0,
  graph_plan_create = 0x7ffff77432f3 <ggml_backend_cpu_graph_plan_create(ggml_backend_t, ggml_cgraph const*)>,
  graph_plan_free = 0x7ffff7743486 <ggml_backend_cpu_graph_plan_free(ggml_backend_t, ggml_backend_graph_plan_t)>,
  graph_plan_update = 0x0,
  graph_plan_compute = 0x7ffff77434d8 <ggml_backend_cpu_graph_plan_compute(ggml_backend_t, ggml_backend_graph_plan_t)>,
  graph_compute = 0x7ffff774350d <ggml_backend_cpu_graph_compute(ggml_backend_t, ggml_cgraph*)>,
  event_record = 0x0,
  event_wait = 0x0
}

>>> p sched->splits[1]->inputs[0]->buffer->iface
$1885 = {
  free_buffer = 0x7fffe6d312c0 <ggml_backend_cuda_host_buffer_free_buffer(ggml_backend_buffer_t)>,
  get_base = 0x7fffe6b714da <ggml_backend_cpu_buffer_get_base(ggml_backend_buffer_t)>,
  init_tensor = 0x0,
  memset_tensor = 0x7fffe6b71542 <ggml_backend_cpu_buffer_memset_tensor(ggml_backend_buffer_t, ggml_tensor*, uint8_t, size_t, size_t)>,
set_tensor = <ggml_backend_cpu_buffer_set_tensor(ggml_backend_buffer_t, ggml_tensor*, void const*, size_t, size_t)>,
get_tensor = <ggml_backend_cpu_buffer_get_tensor(ggml_backend_buffer_t, ggml_tensor const*, void*, size_t, size_t)>,
cpy_tensor = <ggml_backend_cpu_buffer_cpy_tensor(ggml_backend_buffer_t, ggml_tensor const*, ggml_tensor*)>,
  clear = 0x7fffe6b71686 <ggml_backend_cpu_buffer_clear(ggml_backend_buffer_t, uint8_t)>,
  reset = 0x0
}
```

##### [`ggml_backend_cuda_cpy_tensor_async`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L2442)

> 注：CPU没有

```c++
static bool ggml_backend_cuda_cpy_tensor_async(ggml_backend_t backend_src, ggml_backend_t backend_dst, const ggml_tensor * src, ggml_tensor * dst) {
    ggml_backend_buffer_t buf_src = src->view_src ? src->view_src->buffer : src->buffer;
    ggml_backend_buffer_t buf_dst = dst->view_src ? dst->view_src->buffer : dst->buffer;
    --> if (!ggml_backend_is_cuda(backend_src) || !ggml_backend_is_cuda(backend_dst)) return false;
    // 后面的貌似都用不到
}
```

##### [`ggml_backend_tensor_copy`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L370)/`ggml_backend_tensor_set`

```c++
void ggml_backend_tensor_copy(struct ggml_tensor * src, struct ggml_tensor * dst) {
    GGML_ASSERT(ggml_are_same_layout(src, dst) && "cannot copy tensors with different layouts");
    // if (src == dst) return;
    if (ggml_backend_buffer_is_host(src->buffer))
        --> ggml_backend_tensor_set(dst, src->data, 0, ggml_nbytes(src));
    else if (ggml_backend_buffer_is_host(dst->buffer))
        --> ggml_backend_tensor_get(src, dst->data, 0, ggml_nbytes(src));
    //最后一个分支我们不会走到
    // } else if (!ggml_backend_buffer_copy_tensor(src, dst));
}
```

[`ggml_backend_cuda_buffer_set_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L607)

```c++
void ggml_backend_tensor_set(struct ggml_tensor * tensor, const void * data, size_t offset, size_t size) {
    ggml_backend_buffer_t buf = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
    --> buf->iface.set_tensor(buf, tensor, data, offset, size);
}
static void ggml_backend_cuda_buffer_set_tensor(ggml_backend_buffer_t buffer, ggml_tensor * tensor, const void * data, size_t offset, size_t size) {
    ggml_backend_cuda_buffer_context * ctx = (ggml_backend_cuda_buffer_context *)buffer->context;
    ggml_cuda_set_device(ctx->device);
    cudaMemcpyAsync((char *)tensor->data + offset, data, size, cudaMemcpyHostToDevice, cudaStreamPerThread);
    cudaStreamSynchronize(cudaStreamPerThread);
}
```

[`ggml_backend_cuda_buffer_get_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L615)

```c++
void ggml_backend_tensor_get(const struct ggml_tensor * tensor, void * data, size_t offset, size_t size) {
    GGML_ASSERT(tensor);
    ggml_backend_buffer_t buf = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
    --> buf->iface.get_tensor(buf, tensor, data, offset, size);
}
static void ggml_backend_cuda_buffer_get_tensor(ggml_backend_buffer_t buffer, const ggml_tensor * tensor, void * data, size_t offset, size_t size) {
    ggml_backend_cuda_buffer_context * ctx = (ggml_backend_cuda_buffer_context *)buffer->context;
    ggml_cuda_set_device(ctx->device);
    cudaMemcpyAsync(data, (const char *)tensor->data + offset, size, cudaMemcpyDeviceToHost, cudaStreamPerThread);
    cudaStreamSynchronize(cudaStreamPerThread);
}
```

##### [`ggml_backend_cpu_graph_compute`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.cpp#L153)


```c++
static enum ggml_status ggml_backend_cpu_graph_compute(ggml_backend_t backend, struct ggml_cgraph * cgraph) {
    struct ggml_backend_cpu_context * cpu_ctx = (struct ggml_backend_cpu_context *)backend->context;
    --> ggml_graph_plan --> struct ggml_cplan cplan;
    if (cpu_ctx->work_size < cplan.work_size) {
        delete[] cpu_ctx->work_data;
        cpu_ctx->work_data = new uint8_t[cplan.work_size];
        cpu_ctx->work_size = cplan.work_size;
    }
    cplan.work_data = (uint8_t *)cpu_ctx->work_data;
    --> return ggml_graph_compute;
}
```

###### [`ggml_graph_plan`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L2597)

<details>
<summary>struct ggml_cplan</summary>

```c++
// the compute plan that needs to be prepared for ggml_graph_compute()
// since https://github.com/ggml-org/ggml/issues/287
struct ggml_cplan {
    size_t    work_size; // size of work buffer, calculated by `ggml_graph_plan()`
    uint8_t * work_data; // work buffer, to be allocated by caller before calling to `ggml_graph_compute()`

    int n_threads;
    struct ggml_threadpool * threadpool;

    // abort ggml_graph_compute when true
    ggml_abort_callback abort_callback;
    void *              abort_callback_data;
};
```

</details>

```c++
struct ggml_cplan cplan = ggml_graph_plan(cgraph, cpu_ctx->n_threads, cpu_ctx->threadpool);
----------
struct ggml_cplan ggml_graph_plan(
          const struct ggml_cgraph * cgraph,
                               int   n_threads,
            struct ggml_threadpool * threadpool) {}
```

<details>
<summary>struct ggml_type_traits_cpu</summary>

```c++
struct ggml_type_traits_cpu {
    ggml_from_float_t        from_float;
    ggml_vec_dot_t           vec_dot;
    enum ggml_type           vec_dot_type;
    int64_t                  nrows; // number of rows to process simultaneously
};

[GGML_TYPE_Q4_0] = {
    .from_float               = quantize_row_q4_0,
    .vec_dot                  = ggml_vec_dot_q4_0_q8_0,
    .vec_dot_type             = GGML_TYPE_Q8_0,
    .nrows                    = 1,
},
```

</details>

```c++
size_t work_size = 0;
struct ggml_cplan cplan;
memset(&cplan, 0, sizeof(struct ggml_cplan));
int max_tasks = 1;
// thread scheduling for the different operations + work buffer size estimation
for (int i = 0; i < cgraph->n_nodes; i++)
    struct ggml_tensor * node = cgraph->nodes[i];
    const int n_tasks = ggml_get_n_tasks(node, n_threads);
    max_tasks = MAX(max_tasks, n_tasks);
    size_t cur = 0;
    // 暂时将extra功能给禁了
    // if (!ggml_cpu_extra_work_size(n_threads, node, &cur));
    switch (node->op)
        case GGML_OP_MUL_MAT:
            {
                const enum ggml_type vec_dot_type = type_traits_cpu[node->src[0]->type].vec_dot_type;

                if (node->src[1]->type != vec_dot_type) {
                    cur = ggml_row_size(vec_dot_type, ggml_nelements(node->src[1]));
                }
            } break;
        case GGML_OP_SOFT_MAX:
        case GGML_OP_ROPE:
        case GGML_OP_ROPE_BACK:
            {
                cur = ggml_type_size(GGML_TYPE_F32) * node->ne[0] * n_tasks;
            } break;
        case GGML_OP_CPY:
        case GGML_OP_DUP:
            {
                if (ggml_is_quantized(node->type) ||
                    // F16 -> BF16 and BF16 -> F16 copies go through intermediate F32
                    (node->src[0]->type == GGML_TYPE_F16  && node->src[1] && node->src[1]->type == GGML_TYPE_BF16) ||
                    (node->src[0]->type == GGML_TYPE_BF16 && node->src[1] && node->src[1]->type == GGML_TYPE_F16)) {
                    cur = ggml_type_size(GGML_TYPE_F32) * node->ne[0] * n_tasks;
                }
            } break;
        case GGML_OP_ADD:
        case GGML_OP_ADD1:
            {
                if (ggml_is_quantized(node->src[0]->type)) {
                    cur = ggml_type_size(GGML_TYPE_F32) * node->src[0]->ne[0] * n_tasks;
                }
            } break;
    work_size = MAX(work_size, cur);

work_size += CACHE_LINE_SIZE*(n_threads);
cplan.threadpool = threadpool;
cplan.n_threads  = MIN(max_tasks, n_threads);
cplan.work_size  = work_size;
cplan.work_data  = NULL;

return cplan;
```

###### [`ggml_graph_compute`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L3041)

<details>
<summary>struct ggml_threadpool</summary>

```c++
// Threadpool def
struct ggml_threadpool {
    ggml_mutex_t mutex;       // mutex for cond.var
    ggml_cond_t  cond;        // cond.var for waiting for new work

    struct ggml_cgraph * cgraph;
    struct ggml_cplan  * cplan;

    // synchronization primitives
    atomic_int n_graph;       // incremented when there is work to be done (i.e each graph)
    atomic_int GGML_CACHE_ALIGN n_barrier;
    atomic_int GGML_CACHE_ALIGN n_barrier_passed;
    atomic_int GGML_CACHE_ALIGN current_chunk; // currently processing chunk during Mat_Mul, shared between all the threads.

    // these are atomic as an annotation for thread-sanitizer
    atomic_bool stop;         // Used for stopping the threadpool altogether
    atomic_bool pause;        // Used for pausing the threadpool or individual threads
    atomic_int abort;         // Used for aborting processing of a graph

    struct ggml_compute_state * workers;   // per thread state
    int          n_threads_max; // number of threads in the pool
    atomic_int   n_threads_cur; // number of threads used in the current graph

    int32_t      prio;        // Scheduling priority
    uint32_t     poll;        // Polling level (0 - no polling)

    enum ggml_status ec;
};

static struct ggml_threadpool * ggml_threadpool_new_impl(
    struct ggml_threadpool_params * tpp,
               struct ggml_cgraph * cgraph,
                struct ggml_cplan * cplan) {

    struct ggml_threadpool * threadpool =
        ggml_aligned_malloc(sizeof(struct ggml_threadpool));
    {
        threadpool->cgraph           = cgraph;
        threadpool->cplan            = cplan;
        threadpool->n_graph          = 0;
        threadpool->n_barrier        = 0;
        threadpool->n_barrier_passed = 0;
        threadpool->current_chunk    = 0;
        threadpool->stop             = false;
        threadpool->pause            = tpp->paused;
        threadpool->abort            = -1;
        threadpool->workers          = NULL;
        threadpool->n_threads_max    = tpp->n_threads;
        threadpool->n_threads_cur    = tpp->n_threads;
        threadpool->poll             = tpp->poll;
        threadpool->prio             = tpp->prio;
        threadpool->ec               = GGML_STATUS_SUCCESS;
    }

    // Allocate and init workers state
    const size_t workers_size = sizeof(struct ggml_compute_state) * tpp->n_threads;
    struct ggml_compute_state * workers = ggml_aligned_malloc(workers_size);

    memset(workers, 0, workers_size);
    for (int j = 0; j < tpp->n_threads; j++) {
        workers[j].threadpool = threadpool;
        workers[j].ith        = j;
    }

    threadpool->workers = workers;

    return threadpool;
}

void ggml_threadpool_free(struct ggml_threadpool* threadpool) {
    if (!threadpool) return;
    const int n_threads = threadpool->n_threads_max;
    const size_t workers_size = sizeof(struct ggml_compute_state) * n_threads;
    ggml_aligned_free(threadpool->workers, workers_size);
    ggml_aligned_free(threadpool, sizeof(struct ggml_threadpool));
}

```

</details>

<details>
<summary>struct ggml_threadpool_params</summary>

```c++
// threadpool params
// Use ggml_threadpool_params_default() or ggml_threadpool_params_init() to populate the defaults
struct ggml_threadpool_params {
    bool                cpumask[GGML_MAX_N_THREADS]; // mask of cpu cores (all-zeros means use default affinity settings)
    int                 n_threads;                   // number of threads
    enum ggml_sched_priority prio;                   // thread priority
    uint32_t            poll;                        // polling level (0 - no polling, 100 - aggressive polling)
    bool                strict_cpu;                  // strict cpu placement
    bool                paused;                      // start in paused state
};

void ggml_threadpool_params_init(struct ggml_threadpool_params * p, int n_threads) {
    p->n_threads  = n_threads;
    p->prio       = 0;     // default priority (usually means normal or inherited)
    p->poll       = 50;    // hybrid-polling enabled
    p->strict_cpu = false; // no strict placement (all threads share same cpumask)
    p->paused     = false; // threads are ready to go
    memset(p->cpumask, 0, GGML_MAX_N_THREADS); // all-zero means use the default affinity (usually inherited)
}

struct ggml_threadpool_params ggml_threadpool_params_default(int n_threads) {
    struct ggml_threadpool_params p;
    ggml_threadpool_params_init(&p, n_threads);
    return p;
}
```

</details>

```c++
ggml_graph_compute(cgraph, &cplan);
----------
enum ggml_status ggml_graph_compute(struct ggml_cgraph * cgraph, struct ggml_cplan * cplan) {}
```

```c++
ggml_cpu_init();
int n_threads = cplan->n_threads;
struct ggml_threadpool * threadpool = cplan->threadpool;
bool disposable_threadpool = false;
if (threadpool == NULL) {
    //GGML_PRINT_DEBUG("Threadpool is not specified. Will create a disposable threadpool : n_threads %d\n", n_threads);
    disposable_threadpool = true;
    struct ggml_threadpool_params ttp = ggml_threadpool_params_default(n_threads);
    threadpool = ggml_threadpool_new_impl(&ttp, cgraph, cplan);
}
if (n_threads > 1) {
    #pragma omp parallel num_threads(n_threads)
    {
        #pragma omp single
        {
            // update the number of threads from the actual number of threads that we got from OpenMP
            n_threads = omp_get_num_threads();
            atomic_store_explicit(&threadpool->n_threads_cur, n_threads, memory_order_relaxed);
        }
        --> ggml_graph_compute_thread(&threadpool->workers[omp_get_thread_num()]);
    }
} else {
    atomic_store_explicit(&threadpool->n_threads_cur, 1, memory_order_relaxed);
    --> ggml_graph_compute_thread(&threadpool->workers[0]);
}
enum ggml_status ret = threadpool->ec;
if (disposable_threadpool) ggml_threadpool_free(threadpool);
return ret;
```

[`ggml_compute_forward`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L1648)

<details>
<summary>struct ggml_compute_state</summary>

```c++
// Per-thread state
struct ggml_compute_state {
    struct ggml_threadpool * threadpool;
    int ith;
};
```

</details>

<details>
<summary>struct ggml_compute_params</summary>

```c++
struct ggml_compute_params {
    // ith = thread index, nth = number of threads
    int ith, nth;

    // work buffer for all threads
    size_t wsize;
    void * wdata;

    struct ggml_threadpool * threadpool;
};
```

</details>

> [`ggml_graph_compute_thread`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L2785) 开始遍历当前split子图的每个node

```c++
static thread_ret_t ggml_graph_compute_thread(void * data) {
    struct ggml_compute_state * state = (struct ggml_compute_state *) data;
    struct ggml_threadpool    * tp    = state->threadpool;

    const struct ggml_cgraph * cgraph = tp->cgraph;
    const struct ggml_cplan  * cplan  = tp->cplan;

    struct ggml_compute_params params = {
        /*.ith       =*/ state->ith,
        /*.nth       =*/ atomic_load_explicit(&tp->n_threads_cur, memory_order_relaxed),
        /*.wsize     =*/ cplan->work_size,
        /*.wdata     =*/ cplan->work_data,
        /*.threadpool=*/ tp,
    };

    for (int node_n = 0; node_n < cgraph->n_nodes && atomic_load_explicit(&tp->abort, memory_order_relaxed) != node_n; node_n++) {
        struct ggml_tensor * node = cgraph->nodes[node_n];
        --> ggml_compute_forward(&params, node);
        if (node_n + 1 < cgraph->n_nodes) {
            --> ggml_barrier(state->threadpool);
        }
    }

    --> ggml_barrier(state->threadpool);

    return 0;
}

static void ggml_compute_forward(struct ggml_compute_params * params, struct ggml_tensor * tensor) {
    if (tensor->op == GGML_OP_NONE || ggml_is_empty(tensor)) return;

    // extra_buffer op? 暂时将extra功能给禁了
    if (ggml_cpu_extra_compute_forward(params, tensor)) return;
    --> switch (tensor->op) {}
}

void ggml_barrier(struct ggml_threadpool * tp) {
    int n_threads = atomic_load_explicit(&tp->n_threads_cur, memory_order_relaxed);
    if (n_threads == 1) return;

    #pragma omp barrier
}
```

[`ggml_compute_forward_get_rows_q`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ops.cpp#L4236)

```c++
ggml_compute_forward_get_rows_q(params, dst);
----------
static void ggml_compute_forward_get_rows_q(
        const ggml_compute_params * params,
              ggml_tensor * dst) {
    const ggml_tensor * src0 = dst->src[0];
    const ggml_tensor * src1 = dst->src[1];

    GGML_TENSOR_BINARY_OP_LOCALS

    const int64_t nc = ne00;
    const int64_t nr = ggml_nelements(src1);

    const ggml_type type = src0->type;
    ggml_to_float_t const dequantize_row_q = ggml_get_type_traits(type)->to_float;

    const int ith = params->ith;
    const int nth = params->nth;

    // rows per thread
    const int dr = (nr + nth - 1)/nth;

    // row range for this thread
    const int ir0 = dr*ith;
    const int ir1 = MIN(ir0 + dr, nr);

    for (int64_t i = ir0; i < ir1; ++i) {
        const int64_t i12 = i/(ne11*ne10);
        const int64_t i11 = (i - i12*ne11*ne10)/ne10;
        const int64_t i10 = (i - i12*ne11*ne10 - i11*ne10);
        const int64_t i01 = *(int32_t *) ((char *) src1->data + i10*nb10 + i11*nb11 + i12*nb12);
        dequantize_row_q(
                (const void *) ((char *) src0->data + i01*nb01 + i11*nb02 + i12*nb03),
                     (float *) ((char *)  dst->data + i10*nb1  + i11*nb2  + i12*nb3), nc);
    }
}

void dequantize_row_q8_0(const block_q8_0 * GGML_RESTRICT x, float * GGML_RESTRICT y, int64_t k) {
    static const int qk = QK8_0;
    const int nb = k / qk;
    for (int i = 0; i < nb; i++) {
        const float d = GGML_FP16_TO_FP32(x[i].d);
        for (int j = 0; j < qk; ++j) {
            y[i*qk + j] = x[i].qs[j]*d;
        }
    }
}
```

[`ggml_compute_forward_rms_norm_f32`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ops.cpp#L3270)

```c++
ggml_compute_forward_rms_norm(params, tensor);
----------
static void ggml_compute_forward_rms_norm_f32(
        const ggml_compute_params * params,
        ggml_tensor * dst) {
    const ggml_tensor * src0 = dst->src[0];
    const int ith = params->ith;
    const int nth = params->nth;
    GGML_TENSOR_UNARY_OP_LOCALS
    float eps;
    memcpy(&eps, dst->op_params, sizeof(float));
    // TODO: optimize
    for (int64_t i03 = 0; i03 < ne03; i03++) {
        for (int64_t i02 = 0; i02 < ne02; i02++) {
            for (int64_t i01 = ith; i01 < ne01; i01 += nth) {
                const float * x = (float *) ((char *) src0->data + i01*nb01 + i02*nb02 + i03*nb03);
                ggml_float sum = 0.0;
                for (int64_t i00 = 0; i00 < ne00; i00++) sum += (ggml_float)(x[i00] * x[i00]);
                const float mean = sum/ne00;
                float * y = (float *) ((char *) dst->data + i01*nb1 + i02*nb2 + i03*nb3);
                memcpy(y, x, ne00 * sizeof(float));
                // for (int i00 = 0; i00 < ne00; i00++) {
                //     y[i00] = x[i00];
                // }
                const float scale = 1.0f/sqrtf(mean + eps);
                ggml_vec_scale_f32(ne00, y, scale);
            }
        }
    }
}
```

[`apply_binary_op`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/binary-ops.cpp#L50)

```c++
ggml_compute_forward_mul(params, tensor);
----------
void ggml_compute_forward_mul(const ggml_compute_params * params, ggml_tensor * dst) {
    binary_op<op_mul>(params, dst);
}
static void binary_op(const ggml_compute_params * params, ggml_tensor * dst)
    const ggml_tensor * src0 = dst->src[0];
    const ggml_tensor * src1 = dst->src[1];
    /*  */ if (src0->type == GGML_TYPE_F32  && src1->type == GGML_TYPE_F32  && dst->type == GGML_TYPE_F32) // all f32
        apply_binary_op<op, float, float, float>(params, dst);

template <float (*op)(float, float), typename src0_t, typename src1_t, typename dst_t>
static void apply_binary_op(const ggml_compute_params * params, ggml_tensor * dst) {
    const ggml_tensor * src0 = dst->src[0];
    const ggml_tensor * src1 = dst->src[1];

    GGML_ASSERT(ggml_can_repeat(src1, src0) && ggml_are_same_shape(src0, dst));

    GGML_TENSOR_BINARY_OP_LOCALS

    GGML_ASSERT( nb0 == sizeof(dst_t));
    GGML_ASSERT(nb00 == sizeof(src0_t));

    const auto [ir0, ir1] = get_thread_range(params, src0);
    const bool is_src1_contiguous = (nb10 == sizeof(src1_t)); // why:-?

    if (!is_src1_contiguous) { // broadcast not implemented yet for non-contiguous
        GGML_ASSERT(ggml_are_same_shape(src0, src1));
    }

    for (int64_t ir = ir0; ir < ir1; ++ir) {
        const int64_t i03 = ir/(ne02*ne01);
        const int64_t i02 = (ir - i03*ne02*ne01)/ne01;
        const int64_t i01 = (ir - i03*ne02*ne01 - i02*ne01);

        const int64_t i13 = i03 % ne13;
        const int64_t i12 = i02 % ne12;
        const int64_t i11 = i01 % ne11;

        dst_t        * dst_ptr  = (dst_t  *)       ((char *)       dst->data  + i03*nb3  + i02*nb2  + i01*nb1 );
        const src0_t * src0_ptr = (const src0_t *) ((const char *) src0->data + i03*nb03 + i02*nb02 + i01*nb01);
        const src1_t * src1_ptr = (const src1_t *) ((const char *) src1->data + i13*nb13 + i12*nb12 + i11*nb11);

        if (is_src1_contiguous) {
            // src1 is broadcastable across src0 and dst in i1, i2, i3
            const int64_t nr0 = ne00 / ne10;

            for (int64_t r = 0; r < nr0; ++r) {
                --> vec_binary_op_contiguous<op>(ne10, dst_ptr + r*ne10, src0_ptr + r*ne10, src1_ptr);
            }
        } else {
            vec_binary_op_non_contiguous<op>(ne0, ne10, nb10, dst_ptr, src0_ptr, src1_ptr);
        }
    }
}

template <float (*op)(float, float), typename src0_t, typename src1_t, typename dst_t>
static inline void vec_binary_op_contiguous(const int64_t n, dst_t * z, const src0_t * x, const src1_t * y) {
    constexpr auto src0_to_f32 = type_conversion_table<src0_t>::to_f32;
    constexpr auto src1_to_f32 = type_conversion_table<src1_t>::to_f32;
    constexpr auto f32_to_dst  = type_conversion_table<dst_t >::from_f32;

    for (int i = 0; i < n; i++) {
        z[i] = f32_to_dst(op(src0_to_f32(x[i]), src1_to_f32(y[i])));
    }
}

static inline float f32_to_f32(float x) {
    return x;
}

template <float (*op)(float, float), typename src0_t, typename src1_t, typename dst_t>
static inline void vec_binary_op_non_contiguous(const int64_t n, const int64_t ne10, const int64_t nb10, dst_t * z, const src0_t * x, const src1_t * y) {
    constexpr auto src0_to_f32 = type_conversion_table<src0_t>::to_f32;
    constexpr auto src1_to_f32 = type_conversion_table<src1_t>::to_f32;
    constexpr auto f32_to_dst  = type_conversion_table<dst_t >::from_f32;

    for (int i = 0; i < n; i++) {
        int i10 = i % ne10;
        const src1_t * y_ptr = (const src1_t *)((const char *)y + i10*nb10);
        z[i] = f32_to_dst(op(src0_to_f32(x[i]), src1_to_f32(*y_ptr)));
    }
}
```

[`ggml_compute_forward_mul_mat`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L1192
)


```c++
static void ggml_compute_forward_mul_mat(
        const struct ggml_compute_params * params,
              struct ggml_tensor * dst) {

    const struct ggml_tensor * src0 = dst->src[0];
    const struct ggml_tensor * src1 = dst->src[1];

    GGML_TENSOR_BINARY_OP_LOCALS

    const int ith = params->ith;
    const int nth = params->nth;

    enum ggml_type           const vec_dot_type         = type_traits_cpu[src0->type].vec_dot_type;
    ggml_from_float_t        const from_float           = type_traits_cpu[vec_dot_type].from_float;
    int64_t                  const vec_dot_num_rows     = type_traits_cpu[src0->type].nrows;

    // nb01 >= nb00 - src0 is not transposed
    //   compute by src0 rows

    // TODO: extract to "extra_op"

    // if (src1->type != vec_dot_type) {
    char * wdata = params->wdata;
    const size_t nbw0 = ggml_type_size(vec_dot_type);
    const size_t nbw1 = ggml_row_size(vec_dot_type, ne10);
    const size_t nbw2 = nbw1*ne11;
    const size_t nbw3 = nbw2*ne12;

    for (int64_t i11 = 0; i11 < ne11; ++i11) {
        size_t bs = ggml_blck_size(vec_dot_type);
        int64_t ne10_block_start = (ith * ne10/bs) / nth;
        int64_t ne10_block_end   = ((ith + 1) * ne10/bs) / nth;
        --> from_float(
          (float *)((char *) src1->data + i11*nb11 + ne10_block_start*bs*nb10),
          (void *) (wdata + i11*nbw1 + ne10_block_start*nbw0),
          (ne10_block_end - ne10_block_start) * bs);
    }

    if (ith == 0) {
        // Every thread starts at ith, so the first unprocessed chunk is nth.  This save a bit of coordination right at the start.
        // 没看懂这个是什么意思啊
        // 结合着后面的atomic_fetch_add_explicit原子操作，这下看懂了
        atomic_store_explicit(&params->threadpool->current_chunk, nth, memory_order_relaxed);
    }

    ggml_barrier(params->threadpool);

    // This is the size of the first dimension of the result, so we can iterate that way. (see the ASSERT above, these are the same numbers)
    const int64_t nr0 = ne0;

    // This is the size of the rest of the dimensions of the result
    const int64_t nr1 = ne1 * ne2 * ne3;

    // Now select a reasonable chunk size.
    int chunk_size = 16;

    // We need to step up the size if it's small
    if (nr0 == 1 || nr1 == 1) {
        chunk_size = 64;
    }

    // distribute the work across the inner or outer loop based on which one is larger
    // The number of chunks in the 0/1 dim.
    // CEIL(nr0/chunk_size)
    int64_t nchunk0 = (nr0 + chunk_size - 1) / chunk_size;
    int64_t nchunk1 = (nr1 + chunk_size - 1) / chunk_size;

    // If the chunking is poor for the number of threads on this setup, scrap the whole plan.  Re-chunk it by thread.
    //   Also, chunking by thread was measured to have perform better on NUMA systems.  See https://github.com/ggml-org/llama.cpp/pull/6915
    //   In theory, chunking should be just as useful on NUMA and non NUMA systems, but testing disagreed with that.
    if (nchunk0 * nchunk1 < nth * 4 || ggml_is_numa()) {
        // distribute the thread work across the inner or outer loop based on which one is larger
        nchunk0 = nr0 > nr1 ? nth : 1; // parallelize by src0 rows
        nchunk1 = nr0 > nr1 ? 1 : nth; // parallelize by src1 rows
    }

    // The number of elements in each chunk
    const int64_t dr0 = (nr0 + nchunk0 - 1) / nchunk0;
    const int64_t dr1 = (nr1 + nchunk1 - 1) / nchunk1;

    // The first chunk comes from our thread_id, the rest will get auto-assigned.
    int current_chunk = ith;

    while (current_chunk < nchunk0 * nchunk1) {
        const int64_t ith0 = current_chunk % nchunk0;
        const int64_t ith1 = current_chunk / nchunk0;

        const int64_t ir0_start = dr0 * ith0;
        const int64_t ir0_end = MIN(ir0_start + dr0, nr0);

        const int64_t ir1_start = dr1 * ith1;
        const int64_t ir1_end = MIN(ir1_start + dr1, nr1);

        // dot kernels can handle 1 row and col at a time, but mmla kernels can process 2 rows and cols
        int64_t num_rows_per_vec_dot = vec_dot_num_rows;

        // these checks are needed to avoid crossing dim1 boundaries
        // can be optimized, but the logic would become more complicated, so keeping it like this for simplicity
        if ((nr0 % 2 != 0) || (ne11 % 2 != 0) || ((ir0_end - ir0_start) % 2 != 0) || ((ir1_end - ir1_start) % 2 != 0)) {
            num_rows_per_vec_dot = 1;
        }
        --> ggml_compute_forward_mul_mat_one_chunk(params, dst, src0->type, num_rows_per_vec_dot, ir0_start, ir0_end, ir1_start, ir1_end);

        if (nth >= nchunk0 * nchunk1) {
            break;
        }

        current_chunk = atomic_fetch_add_explicit(&params->threadpool->current_chunk, 1, memory_order_relaxed);
    }
}
```


```c++
static void ggml_compute_forward_mul_mat_one_chunk(
    const struct ggml_compute_params * params,
    struct ggml_tensor * dst,
    const enum ggml_type type,
    const int64_t num_rows_per_vec_dot,
    const int64_t ir0_start,
    const int64_t ir0_end,
    const int64_t ir1_start,
    const int64_t ir1_end) {

    const struct ggml_tensor * src0 = dst->src[0];
    const struct ggml_tensor * src1 = dst->src[1];

    GGML_TENSOR_BINARY_OP_LOCALS

    const bool src1_cont = ggml_is_contiguous(src1);

    ggml_vec_dot_t const vec_dot      = type_traits_cpu[type].vec_dot;
    enum ggml_type const vec_dot_type = type_traits_cpu[type].vec_dot_type;

    // broadcast factors 我们的case貌似不需要考虑？
    // threads with no work simply yield (not sure if it helps)
    if (ir0_start >= ir0_end || ir1_start >= ir1_end) {
        return;
    }

    const void * wdata = params->wdata;
    const size_t row_size = ggml_row_size(vec_dot_type, ne10);

    assert(ne12 % ne02 == 0);
    assert(ne13 % ne03 == 0);

    // block-tiling attempt
    const int64_t blck_0 = 16;
    const int64_t blck_1 = 16;

    // const size_t src1_col_stride = src1_cont || src1->type != vec_dot_type ? row_size : nb11; 貌似我们不用考虑src1不cont的情况
    const size_t src1_col_stride = row_size;

    // attempt to reduce false-sharing (does not seem to make a difference)
    // 16 * 2, accounting for mmla kernels
    // float tmp[32];
    for (int64_t iir1 = ir1_start; iir1 < ir1_end; iir1 += blck_1)
    for (int64_t iir0 = ir0_start; iir0 < ir0_end; iir0 += blck_0)
    // 理想情况下，上面两个循环是没有用的
    for (int64_t ir1 = iir1; ir1 < iir1 + blck_1 && ir1 < ir1_end; ++ir1)
    for (int64_t ir0 = iir0; ir0 < iir0 + blck_0 && ir0 < ir0_end; ++ir0)
        --> vec_dot(
              n=ne00,
              s=(float*)((char*)dst->data + ir0 + ir1 * nb1), //dst_col
              bs=0, // UNUSED
              vx=(const char*)src0->data + ir0 * nb01, //src0_row
              bx=0, // UNUSED
              vy=(const char*)wdata + ir1 * row_size, // src1_col
              by=0, // UNUSED
              nrc=1 // UNUSED
          );
    }
```

[`ggml_vec_dot_q4_0_q8_0`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/arch/x86/quants.c#L531)

| 特性                | `_mm256_maddubs_epi16`               | `_mm256_madd_epi16`               |
|---------------------|--------------------------------------|-----------------------------------|
| **输入数据类型**    | 无符号 8 位 × 有符号 8 位            | 有符号 16 位 × 有符号 16 位       |
| **输出数据类型**    | 16 位有符号整数（16 个结果）         | 32 位有符号整数（8 个结果）       |
| **运算粒度**        | 8 位 → 16 位                         | 16 位 → 32 位                     |
| **相邻加法**        | 每对 8 位乘积相加                    | 每对 16 位乘积相加                |
| **适用场景**        | 低精度计算（如图像处理）             | 高精度计算（如数值运算）          |

<details>
<summary>original implementation</summary>

```c++
float sumf = 0;
for (ib = 0; ib < nb; ++ib) {
    int sumi0 = 0;
    int sumi1 = 0;

    for (int j = 0; j < qk/2; ++j) {
        const int v0 = (x[ib].qs[j] & 0x0F) - 8;
        const int v1 = (x[ib].qs[j] >>   4) - 8;

        sumi0 += (v0 * y[ib].qs[j]);
        sumi1 += (v1 * y[ib].qs[j + qk/2]);
    }

    int sumi = sumi0 + sumi1;
    sumf += sumi*GGML_FP16_TO_FP32(x[ib].d)*GGML_FP16_TO_FP32(y[ib].d);
}
```

</details>

```c++
void ggml_vec_dot_q4_0_q8_0(int n, float * GGML_RESTRICT s, size_t, const void * GGML_RESTRICT vx, size_t, const void * GGML_RESTRICT vy, size_t, int) {
    const int qk = QK8_0;
    const int nb = n / qk;
    const block_q4_0 * GGML_RESTRICT x = vx;
    const block_q8_0 * GGML_RESTRICT y = vy;
    int ib = 0;
    float sumf = 0;
    // Initialize accumulator with zeros
    __m256 acc = _mm256_setzero_ps();

    // Main loop
    for (; ib < nb; ++ib) {
        /* Compute combined scale for the block */
        const __m256 d = _mm256_set1_ps( GGML_FP16_TO_FP32(x[ib].d) * GGML_FP16_TO_FP32(y[ib].d) );

        --> __m256i qx = bytes_from_nibbles_32(x[ib].qs);

        // Now we have a vector with bytes in [ 0 .. 15 ] interval. Offset them into [ -8 .. +7 ] interval.
        const __m256i off = _mm256_set1_epi8( 8 );
        qx = _mm256_sub_epi8( qx, off );

        __m256i qy = _mm256_loadu_si256((const __m256i *)y[ib].qs);

        --> const __m256 q = mul_sum_i8_pairs_float(qx, qy);

        /* Multiply q with scale and accumulate */
        acc = _mm256_fmadd_ps( d, q, acc );
    }

    --> sumf = hsum_float_8(acc);
    *s = sumf;
}

// Unpack 32 4-bit fields into 32 bytes
// The output vector contains 32 bytes, each one in [ 0 .. 15 ] interval
static inline __m256i bytes_from_nibbles_32(const uint8_t * rsi)
{
    const __m128i tmp = _mm_loadu_si128((const __m128i *)rsi);
    const __m256i bytes = MM256_SET_M128I(_mm_srli_epi16(tmp, 4), tmp);
    const __m256i lowMask = _mm256_set1_epi8( 0xF );
    return _mm256_and_si256(lowMask, bytes);
}

// 4->8->16->32

// multiply int8_t, add results pairwise twice and return as float vector
static inline __m256 mul_sum_i8_pairs_float(const __m256i x, const __m256i y) {
#if __AVXVNNIINT8__
    const __m256i zero = _mm256_setzero_si256();
    const __m256i summed_pairs = _mm256_dpbssd_epi32(zero, x, y);
    return _mm256_cvtepi32_ps(summed_pairs);
#else
    // Get absolute values of x vectors
    const __m256i ax = _mm256_sign_epi8(x, x);
    // Sign the values of the y vectors
    const __m256i sy = _mm256_sign_epi8(y, x);
    return mul_sum_us8_pairs_float(ax, sy);
#endif
}

static inline __m256 mul_sum_us8_pairs_float(const __m256i ax, const __m256i sy) {
#if defined(__AVXVNNI__)
    const __m256i zero = _mm256_setzero_si256();
    // const __m256i summed_pairs = _mm256_dpbusd_epi32(zero, ax, sy);
    const __m256i summed_pairs = _mm256_dpbusd_avx_epi32(zero, ax, sy);
    return _mm256_cvtepi32_ps(summed_pairs);
#else
    // Perform multiplication and create 16-bit values
    const __m256i dot = _mm256_maddubs_epi16(ax, sy);
    const __m256i ones = _mm256_set1_epi16(1);
    const __m256i summed_pairs = _mm256_madd_epi16(ones, dot);
    return _mm256_cvtepi32_ps(summed_pairs);
#endif
}

// horizontally add 8 floats
static inline float hsum_float_8(const __m256 x) {
    __m128 res = _mm256_extractf128_ps(x, 1);
    res = _mm_add_ps(res, _mm256_castps256_ps128(x));
    res = _mm_add_ps(res, _mm_movehl_ps(res, res));
    res = _mm_add_ss(res, _mm_movehdup_ps(res));
    return _mm_cvtss_f32(res);
}
```

[`llamafile_sgemm`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/llamafile/sgemm.cpp#L3310)

加速比大概是2倍，但是为了简单起见，我们先禁用了。

```c++
llamafile_sgemm(params,
                m=ne01, n=ne11, k=ne00/ggml_blck_size(src0->type),
                A=(const char *)src0->data,
                lda=nb01/ggml_type_size(src0->type),
                B=(const char *)src1->data,
                ldb=nb11/ggml_type_size(src1->type),
                C=(char *)dst->data,
                ldc=nb1/ggml_type_size(dst->type),
                Atype=src0->type,
                Btype=src1->type,
                Ctype=dst->type);

const void* wdata = params->wdata;
const size_t row_size = ggml_row_size(vec_dot_type, ne10);
llamafile_sgemm(params,
                ne01, ne11, ne00/ggml_blck_size(src0->type),
                (const char *)src0->data,
                nb01/ggml_type_size(src0->type),
                (const char *)wdata,
                row_size/ggml_type_size(vec_dot_type),
                (char *)dst->data,
                nb1/ggml_type_size(dst->type),
                src0->type,
                vec_dot_type,
                dst->type);
----------
/**
 * Performs optimized matrix multiplication on CPU.
 *
 * This subroutine may compute C = Aᵀ * B with column major ordering.
 * Despite its name, this isn't a generalized implementation. Work is
 * only performed when a handwritten kernel is written and available.
 * Otherwise the caller should fall back to a general matmul routine.
 *
 * For example, for single-threaded single-precision GEMM you can say
 *
 *     llamafile_sgemm(m, n, k, A, lda, B, ldb, C, ldc,
 *                     0, 1,
 *                     GGML_TYPE_F32, GGML_TYPE_F32, GGML_TYPE_F32);
 *
 * @param m is rows in `A` and `C`
 * @param n is cols in `B` and `C`
 * @param k is cols in `A` and rows in `B`
 * @param A is first input matrix (always transposed)
 * @param lda is row stride of `A`
 * @param B is second input matrix (never transposed)
 * @param ldb is row stride of `B`
 * @param C is input/output array of output matrices
 * @param ldc is row stride of `C`
 * @param ith is thread id (must be less than `nth`)
 * @param nth is number of threads (must be greater than zero)
 * @param Atype is GGML data type of `A`
 * @param Btype is GGML data type of `B`
 * @param Ctype is GGML data type of `C`
 * @return true if this function was able to service the matmul request
 */
bool llamafile_sgemm(const struct ggml_compute_params * params, int64_t m, int64_t n, int64_t k,
                     const void *A, int64_t lda, const void *B, int64_t ldb, void *C,
                     int64_t ldc, int Atype, int Btype, int Ctype) {}
```


```c++
void quantize_row_q8_0(const float * GGML_RESTRICT x, void * GGML_RESTRICT vy, int64_t k) {
    assert(QK8_0 == 32);
    assert(k % QK8_0 == 0);
    const int nb = k / QK8_0;
    block_q8_0 * GGML_RESTRICT y = vy;

    for (int i = 0; i < nb; i++) {
        // Load elements into 4 AVX vectors
        __m256 v0 = _mm256_loadu_ps( x );
        __m256 v1 = _mm256_loadu_ps( x + 8 );
        __m256 v2 = _mm256_loadu_ps( x + 16 );
        __m256 v3 = _mm256_loadu_ps( x + 24 );
        x += 32;
        // Compute max(abs(e)) for the block
        const __m256 signBit = _mm256_set1_ps( -0.0f );
        __m256 maxAbs = _mm256_andnot_ps( signBit, v0 );
        maxAbs = _mm256_max_ps( maxAbs, _mm256_andnot_ps( signBit, v1 ) );
        maxAbs = _mm256_max_ps( maxAbs, _mm256_andnot_ps( signBit, v2 ) );
        maxAbs = _mm256_max_ps( maxAbs, _mm256_andnot_ps( signBit, v3 ) );
        __m128 max4 = _mm_max_ps( _mm256_extractf128_ps( maxAbs, 1 ), _mm256_castps256_ps128( maxAbs ) );
        max4 = _mm_max_ps( max4, _mm_movehl_ps( max4, max4 ) );
        max4 = _mm_max_ss( max4, _mm_movehdup_ps( max4 ) );
        const float maxScalar = _mm_cvtss_f32( max4 );
        // Quantize these floats
        const float d = maxScalar / 127.f;
        y[i].d = GGML_FP32_TO_FP16(d);
        const float id = ( maxScalar != 0.0f ) ? 127.f / maxScalar : 0.0f;
        const __m256 mul = _mm256_set1_ps( id );
        // Apply the multiplier
        v0 = _mm256_mul_ps( v0, mul );
        v1 = _mm256_mul_ps( v1, mul );
        v2 = _mm256_mul_ps( v2, mul );
        v3 = _mm256_mul_ps( v3, mul );
        // Round to nearest integer
        v0 = _mm256_round_ps( v0, _MM_ROUND_NEAREST );
        v1 = _mm256_round_ps( v1, _MM_ROUND_NEAREST );
        v2 = _mm256_round_ps( v2, _MM_ROUND_NEAREST );
        v3 = _mm256_round_ps( v3, _MM_ROUND_NEAREST );
        // Convert floats to integers
        __m256i i0 = _mm256_cvtps_epi32( v0 );
        __m256i i1 = _mm256_cvtps_epi32( v1 );
        __m256i i2 = _mm256_cvtps_epi32( v2 );
        __m256i i3 = _mm256_cvtps_epi32( v3 );
        // Convert int32 to int16
        i0 = _mm256_packs_epi32( i0, i1 );	// 0, 1, 2, 3,  8, 9, 10, 11,  4, 5, 6, 7, 12, 13, 14, 15
        i2 = _mm256_packs_epi32( i2, i3 );	// 16, 17, 18, 19,  24, 25, 26, 27,  20, 21, 22, 23, 28, 29, 30, 31
                                            // Convert int16 to int8
        i0 = _mm256_packs_epi16( i0, i2 );	// 0, 1, 2, 3,  8, 9, 10, 11,  16, 17, 18, 19,  24, 25, 26, 27,  4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31
        // We got our precious signed bytes, but the order is now wrong
        // These AVX2 pack instructions process 16-byte pieces independently
        // The following instruction is fixing the order
        const __m256i perm = _mm256_setr_epi32( 0, 4, 1, 5, 2, 6, 3, 7 );
        i0 = _mm256_permutevar8x32_epi32( i0, perm );
        _mm256_storeu_si256((__m256i *)y[i].qs, i0);
    }
}

```

##### [`ggml_backend_cuda_graph_compute`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L2752)

<details>
<summary>struct ggml_cuda_graph</summary>

```c++
struct ggml_cuda_graph {
// #ifdef USE_CUDA_GRAPH
    ~ggml_cuda_graph() {
        if (instance != nullptr) CUDA_CHECK(cudaGraphExecDestroy(instance));
        if (graph != nullptr) CUDA_CHECK(cudaGraphDestroy(graph));
    }
    cudaGraph_t graph = nullptr;
    cudaGraphExec_t instance = nullptr;
    size_t num_nodes = 0;
    std::vector<cudaGraphNode_t> nodes;
    std::vector<cudaKernelNodeParams> params;
    bool disable_due_to_gpu_arch = false;
    bool disable_due_to_too_many_updates = false;
    bool disable_due_to_failed_graph_capture = false;
    int number_consecutive_updates = 0;
    std::vector<ggml_graph_node_properties> ggml_graph_properties;
    bool use_cpy_indirection = false;
    std::vector<char *> cpy_dest_ptrs;
    char ** dest_ptrs_d;
    int dest_ptrs_size = 0;
    // Index to allow each cpy kernel to be aware of it's position within the graph
    // relative to other cpy nodes.
    int graph_cpynode_index = -1;
// #endif
};

>>> p *cuda_ctx->cuda_graph
$1707 = {
  graph = 0x0,
  instance = 0x0,
  num_nodes = 0,
  nodes = std::vector of length 0, capacity 0,
  params = std::vector of length 0, capacity 0,
  disable_due_to_gpu_arch = false,
  disable_due_to_too_many_updates = false,
  disable_due_to_failed_graph_capture = false,
  number_consecutive_updates = 0,
  ggml_graph_properties = std::vector of length 0, capacity 0,
  use_cpy_indirection = false,
  cpy_dest_ptrs = std::vector of length 0, capacity 0,
  dest_ptrs_d = 0x0,
  dest_ptrs_size = 0,
  graph_cpynode_index = -1
}
```

</details>

> 这个函数说明了，难怪要warmup

```c++
static enum ggml_status ggml_backend_cuda_graph_compute(ggml_backend_t backend, ggml_cgraph * cgraph) {
    ggml_backend_cuda_context * cuda_ctx = (ggml_backend_cuda_context *)backend->context;
    ggml_cuda_set_device(cuda_ctx->device);
// #ifdef USE_CUDA_GRAPH
    static const bool disable_cuda_graphs_due_to_env = (getenv("GGML_CUDA_DISABLE_GRAPHS") != nullptr);
    // Objects required for CUDA Graph
    // 需要warmup的原因找到了
    // 第二次可能就不执行了 *key*
    if (cuda_ctx->cuda_graph == nullptr) --> cuda_ctx->cuda_graph.reset(new ggml_cuda_graph());
    bool use_cuda_graph = true;
    bool cuda_graph_update_required = false;
    if (cuda_ctx->cuda_graph->graph == nullptr);
        // 我们不会用到AMPERE以下的
        // if (ggml_cuda_info().devices[cuda_ctx->device].cc < GGML_CUDA_CC_AMPERE) cuda_ctx->cuda_graph->disable_due_to_gpu_arch = true; // GGML_LOG_DEBUG("%s: disabling CUDA graphs due to GPU architecture\n", __func__);
    // Disable CUDA graphs in presence of env var, old GPU, use-case which is changing too rapidly,
    // or previous graph capture failure.
    // Also disable for multi-gpu for now. TO DO investigate
    // 不会被执行到
    if (disable_cuda_graphs_due_to_env
        || cuda_ctx->cuda_graph->disable_due_to_gpu_arch
        || cuda_ctx->cuda_graph->disable_due_to_too_many_updates
        || cuda_ctx->cuda_graph->disable_due_to_failed_graph_capture)
        use_cuda_graph = false;
    
    // 默认会被执行到
    // if (use_cuda_graph) {
    --> is_cuda_graph_update_required(cuda_ctx, cgraph) --> cuda_graph_update_required, cuda_ctx->cuda_graph->ggml_graph_properties;
    --> check_node_graph_compatibility_and_refresh_copy_ops(cuda_ctx, cgraph, use_cuda_graph) --> use_cuda_graph, cuda_ctx->cuda_graph|->cpy_dest_ptrs/dest_ptrs_d/dest_ptrs_size/use_cpy_indirection/graph_cpynode_index;
    // Disable CUDA graphs (from the next token) if the use-case is demanding too many consecutive graph updates.
    if (use_cuda_graph && cuda_graph_update_required) cuda_ctx->cuda_graph->number_consecutive_updates++;
    else cuda_ctx->cuda_graph->number_consecutive_updates = 0;
    // GGML_LOG_DEBUG("%s: disabling CUDA graphs due to too many consecutive updates\n", __func__);
    if (cuda_ctx->cuda_graph->number_consecutive_updates >= 4) cuda_ctx->cuda_graph->disable_due_to_too_many_updates = true;
    // }
    // 第二次可能不是执行到
    if (use_cuda_graph && cuda_graph_update_required)
        // Start CUDA graph capture
        std::lock_guard<std::mutex> lock(ggml_cuda_lock);
        ggml_cuda_lock_counter.fetch_add(1, std::memory_order_relaxed);
        cudaStreamBeginCapture(cuda_ctx->stream(), cudaStreamCaptureModeRelaxed);
    // if (!use_cuda_graph) cuda_ctx->cuda_graph->use_cpy_indirection = false; //大概率不会执行到
// #endif // USE_CUDA_GRAPH
    --> evaluate_and_capture_cuda_graph(cuda_ctx, cgraph, graph_evaluated_or_captured=false, use_cuda_graph, cuda_graph_update_required) --> cuda_ctx->cuda_graph->graph;
    return GGML_STATUS_SUCCESS;
}
```

###### [`is_cuda_graph_update_required`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L2623)

<details>
<summary>struct ggml_graph_node_properties</summary>

```c++
struct ggml_graph_node_properties {
    void * node_address;
    ggml_op node_op;
    int64_t ne[GGML_MAX_DIMS];
    size_t nb[GGML_MAX_DIMS];
    void * src_address[GGML_MAX_SRC];
    int32_t op_params[GGML_MAX_OP_PARAMS / sizeof(int32_t)];
};
```

</details>

```c++
static bool is_cuda_graph_update_required(ggml_backend_cuda_context * cuda_ctx, ggml_cgraph * cgraph) {
    bool cuda_graph_update_required = false;
    if (cuda_ctx->cuda_graph->instance == nullptr) cuda_graph_update_required = true;
    // Check if the graph size has changed
    if (cuda_ctx->cuda_graph->ggml_graph_properties.size() != (size_t)cgraph->n_nodes)
        cuda_graph_update_required = true;
        cuda_ctx->cuda_graph->ggml_graph_properties.resize(cgraph->n_nodes);
    // Loop over nodes in GGML graph to determine if CUDA graph update is required
    // and store properties to allow this comparison for the next token
    for (int i = 0; i < cgraph->n_nodes; i++)
        bool has_matching_properties = true;
        if (!cuda_graph_update_required)
            ggml_graph_node_has_matching_properties(cgraph->nodes[i], &cuda_ctx->cuda_graph->ggml_graph_properties[i]) --> has_matching_properties;
            --->if (node->data != graph_node_properties->node_address &&
                    node->op != GGML_OP_CPY &&
                    node->op != GGML_OP_VIEW) return false;
                if (node->op != graph_node_properties->node_op) return false;
                for (int i = 0; i < GGML_MAX_DIMS; i++)
                    if (node->ne[i] != graph_node_properties->ne[i]) return false;
                    if (node->nb[i] != graph_node_properties->nb[i]) return false;
                for (int i = 0; i < GGML_MAX_SRC; i++)
                    if (node->src[i] &&
                        node->src[i]->data != graph_node_properties->src_address[i] &&
                        node->op != GGML_OP_CPY &&
                        node->op != GGML_OP_VIEW
                    ) return false;
                if (node->op == GGML_OP_SCALE &&
                    memcmp(graph_node_properties->op_params, node->op_params, GGML_MAX_OP_PARAMS) != 0) return false;
                return true;
        if (!has_matching_properties)
            cuda_graph_update_required = true;
            set_ggml_graph_node_properties(node=cgraph->nodes[i], p=&cuda_ctx->cuda_graph->ggml_graph_properties[i]);
            --->p->node_address = node->data; p->node_op = node->op;
                for (int i = 0; i < GGML_MAX_DIMS; i++) p->ne[i] = node->ne[i]; p->nb[i] = node->nb[i];
                for (int i = 0; i < GGML_MAX_SRC; i++) p->src_address[i] = node->src[i] ? node->src[i]->data : nullptr;
                memcpy(p->op_params, node->op_params, GGML_MAX_OP_PARAMS);
    return cuda_graph_update_required;
}
```

###### [`check_node_graph_compatibility_and_refresh_copy_ops`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L2506)

```c++
static bool check_node_graph_compatibility_and_refresh_copy_ops(ggml_backend_cuda_context * cuda_ctx, ggml_cgraph * cgraph, bool use_cuda_graph) {
    // Loop over nodes in GGML graph to obtain info needed for CUDA graph
    cuda_ctx->cuda_graph->cpy_dest_ptrs.clear();
    for (int i = 0; i < cgraph->n_nodes; i++)
        ggml_tensor * node = cgraph->nodes[i];
        if (ggml_is_empty(node) || node->op == GGML_OP_RESHAPE || node->op == GGML_OP_TRANSPOSE || \
            node->op == GGML_OP_VIEW || node->op == GGML_OP_PERMUTE || node->op == GGML_OP_NONE) continue;
        if (node->src[0] && node->src[0]->buffer && ggml_backend_buft_is_cuda_split(node->src[0]->buffer->buft))
            use_cuda_graph = false; // Split buffers are not supported by CUDA graph capture
            // GGML_LOG_DEBUG("%s: disabling CUDA graphs due to split buffer\n", __func__);
        if (node->op == GGML_OP_MUL_MAT_ID && node->ne[2] != 1)
            use_cuda_graph = false; // This node type is not supported by CUDA graph capture
            // GGML_LOG_DEBUG("%s: disabling CUDA graphs due to unsupported node type\n", __func__);
        if (node->op == GGML_OP_ADD && node->src[1] && node->src[1]->ne[1] > 1)
            // disable CUDA graphs for batch size > 1 for now.
            // Changes in batch size or context size can cause changes to the grid size of some kernels.
            use_cuda_graph = false;
            // GGML_LOG_DEBUG("%s: disabling CUDA graphs due to batch size > 1 [%s] [%ld %ld %ld %ld]\n", __func__, node->name, node->ne[0], node->ne[1], node->ne[2], node->ne[3]);
        if (node->op == GGML_OP_CPY) // cache_k, cache_v
            // Store the pointers which are updated for each token, such that these can be sent
            // to the device and accessed using indirection from CUDA graph
            cuda_ctx->cuda_graph->cpy_dest_ptrs.push_back((char *) node->src[1]->data);
            // store a pointer to each copy op CUDA kernel to identify it later ??? 看起来也没用到啊
            void * ptr = ggml_cuda_cpy_fn(node->src[0], node->src[1]); // return (void*) cpy_f32_f16<cpy_1_f32_f16>;
            if (!ptr) use_cuda_graph = false;
            // GGML_LOG_DEBUG("%s: disabling CUDA graphs due to unsupported copy op\n", __func__);
        if (!use_cuda_graph) break;
    if (use_cuda_graph)
        cuda_ctx->cuda_graph->use_cpy_indirection = true;
        // copy pointers to GPU so they can be accessed via indirection within CUDA graph
        ggml_cuda_cpy_dest_ptrs_copy(
            cuda_graph=cuda_ctx->cuda_graph.get(),
            host_dest_ptrs=cuda_ctx->cuda_graph->cpy_dest_ptrs.data(),
            host_dest_ptrs_size=cuda_ctx->cuda_graph->cpy_dest_ptrs.size(),
            stream=cuda_ctx->stream());
        --->// 下面这个if第二次可能不会执行
            if (cuda_graph->dest_ptrs_size < host_dest_ptrs_size) // (re-)allocate GPU memory for 
                cudaStreamSynchronize(stream);
                if (cuda_graph->dest_ptrs_d != nullptr) cudaFree(cuda_graph->dest_ptrs_d);
                cudaMalloc(&cuda_graph->dest_ptrs_d, host_dest_ptrs_size*sizeof(char *));
                cuda_graph->dest_ptrs_size = host_dest_ptrs_size;
            cudaMemcpyAsync(cuda_graph->dest_ptrs_d, host_dest_ptrs, host_dest_ptrs_size*sizeof(char *), cudaMemcpyHostToDevice, stream);
            cuda_graph->graph_cpynode_index = 0; // reset index
    return use_cuda_graph;
}
```

###### [`evaluate_and_capture_cuda_graph`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L2681)

```c++
static void evaluate_and_capture_cuda_graph(ggml_backend_cuda_context * cuda_ctx, ggml_cgraph * cgraph,
    bool & graph_evaluated_or_captured, bool & use_cuda_graph, bool & cuda_graph_update_required) {
    // flag used to determine whether it is an integrated_gpu
    const bool integrated = ggml_cuda_info().devices[cuda_ctx->device].integrated;

    while (!graph_evaluated_or_captured) // while 循环只会被执行一次
        // Only perform the graph execution if CUDA graphs are not enabled, or we are capturing the graph.
        // With the use of CUDA graphs, the execution will be performed by the graph launch.
        // 第二次大概率不会被执行到
        if (!use_cuda_graph || cuda_graph_update_required)
            for (int i = 0; i < cgraph->n_nodes; i++)
                ggml_tensor * node = cgraph->nodes[i];
                if (ggml_is_empty(node) || node->op == GGML_OP_RESHAPE || node->op == GGML_OP_TRANSPOSE || node->op == GGML_OP_VIEW || node->op == GGML_OP_PERMUTE || node->op == GGML_OP_NONE) continue;
                --> ggml_cuda_compute_forward(*cuda_ctx, node) --> bool ok;
        // 第二次大概率不会被执行到
        if (use_cuda_graph && cuda_graph_update_required) // End CUDA graph capture
            // if (cuda_ctx->cuda_graph->graph != nullptr);// 不会为true
            cudaStreamEndCapture(cuda_ctx->stream(), &cuda_ctx->cuda_graph->graph);
            graph_evaluated_or_captured = true; // CUDA graph has been captured
            std::lock_guard<std::mutex> lock(ggml_cuda_lock);
            if (ggml_cuda_lock_counter.fetch_sub(1, std::memory_order_relaxed) == 1) ggml_cuda_lock_cv.notify_all();
        else //第二次大概率直接会到这里
            graph_evaluated_or_captured = true; // ggml graph has been directly evaluated
    if (use_cuda_graph)
        // 第二次大概率不会被执行到
        if (cuda_ctx->cuda_graph->instance == nullptr) // Create executable graph from captured graph.
            cudaGraphInstantiate(&cuda_ctx->cuda_graph->instance, cuda_ctx->cuda_graph->graph, NULL, NULL, 0);
        // 第二次大概率不会被执行到
        if (cuda_graph_update_required) // Update graph executable
            update_cuda_graph_executable(cuda_ctx);
            --->//#if CUDART_VERSION >= 12000
                cudaGraphExecUpdate(cuda_ctx->cuda_graph->instance, cuda_ctx->cuda_graph->graph, cudaGraphExecUpdateResultInfo &result_info); //cudaSuccess
        // Launch graph
        cudaGraphLaunch(cuda_ctx->cuda_graph->instance, cuda_ctx->stream());
}
```

### [`ggml_backend_tensor_get_async`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#245)

```c++
ggml_backend_tensor_get_async(backend_res, t_logits, logits_out, 0, n_outputs*n_vocab*sizeof(float));
----------
void ggml_backend_tensor_get_async(ggml_backend_t backend, const struct ggml_tensor * tensor, void * data, size_t offset, size_t size) {
    if (backend->iface.get_tensor_async == NULL) ggml_backend_tensor_get(tensor, data, offset, size);
    else backend->iface.get_tensor_async(backend, tensor, data, offset, size); // output 层也在GPU上
}
```

[`ggml_backend_cpu_buffer_get_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1896)

```c++
void ggml_backend_tensor_get(const struct ggml_tensor * tensor, void * data, size_t offset, size_t size) {
    ggml_backend_buffer_t buf = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
    --> buf->iface.get_tensor(buf, tensor, data, offset, size);
        --> static void ggml_backend_cpu_buffer_get_tensor(ggml_backend_buffer_t buffer, const struct ggml_tensor * tensor, void * data, size_t offset, size_t size)
            memcpy(data, (const char *)tensor->data + offset, size);
}
```

[`ggml_backend_cuda_get_tensor_async`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cuda/ggml-cuda.cu#L2507)

```c++
static void ggml_backend_cuda_get_tensor_async(ggml_backend_t backend, const ggml_tensor * tensor, void * data, size_t offset, size_t size) {
    cudaMemcpyAsync(data, (const char *)tensor->data + offset, size, cudaMemcpyDeviceToHost, ((ggml_backend_cuda_context *)backend->context;)->stream());
}
```

# [`perplexity`](https://github.com/ggml-org/llama.cpp/blob/master/tools/perplexity/perplexity.cpp#L441)

```c++
results = perplexity(ctx, params, n_ctx);
----------
static results_perplexity perplexity(llama_context * ctx, const common_params & params, const int32_t n_ctx) {}
```

```c++
// Download: https://huggingface.co/datasets/ggml-org/ci/resolve/main/wikitext-2-raw-v1.zip
// Run `./llama-perplexity -m models/7B/ggml-model-q4_0.bin -f wiki.test.raw`
// Output: `perplexity: 13.5106 [114/114]`
// BOS tokens will be added for each chunk before eval

const llama_model * model = llama_get_model(ctx);
const llama_vocab * vocab = llama_model_get_vocab(model);
const bool add_bos = llama_vocab_get_add_bos(vocab);
--> common_tokenize --> llama_tokenize --> vocab->tokenize --> llama_vocab::tokenize --> llama_vocab::impl::tokenize --> std::vector<llama_token> tokens;

std::vector<float> logit_history;
logit_history.resize(tokens.size());

std::vector<float> prob_history;
prob_history.resize(tokens.size());

const int n_chunk_max = tokens.size() / n_ctx;
const int n_chunk = params.n_chunks < 0 ? n_chunk_max : std::min(params.n_chunks, n_chunk_max);
const int n_batch = params.n_batch;
const int n_vocab = llama_vocab_n_tokens(vocab);
int count = 0;
double nll = 0.0;
double nll2 = 0.0;
const int num_batches = (n_ctx + n_batch - 1) / n_batch;
const int n_seq = std::max(1, n_batch / n_ctx);
--> llama_batch_init --> llama_batch batch;
std::vector<std::thread> workers(std::thread::hardware_concurrency() - 1);

// We get the logits for all the tokens in the context window (params.n_ctx)
// from llama_eval above.  Now, based on https://huggingface.co/docs/transformers/perplexity,
// calculate the perplexity over the last half of the window (so the model always has
// some context to predict the token).
//
// We rely on the fact that attention in the forward pass only looks at previous
// tokens here, so the logits returned for each token are an accurate representation
// of what the model would have predicted at that point.
//
// Example, we have a context window of 512, we will compute perplexity for each of the
// last 256 tokens.  Then, we split the input up into context window size chunks to
// process the entire prompt.
const int first = n_ctx/2;
for (int i = 0; i < n_chunk; i += n_seq)
    const int start =     i * n_ctx;
    const int end   = start + n_ctx;
    const int n_seq_batch = std::min(n_seq, n_chunk - i);
    // clear the KV cache
    llama_memory_clear(llama_get_memory(ctx), true);
    // for (int j = 0; j < num_batches; ++j)
    const int batch_start = start + j * n_batch;
    const int batch_size  = std::min(end - batch_start, n_batch);
    int n_outputs = 0;
    batch.n_tokens = 0;
    for (int seq = 0; seq < n_seq_batch; seq++)
        int seq_start = batch_start + seq*n_ctx;
        // save original token and restore it after eval
        const auto token_org = tokens[seq_start];
        if (add_bos && j == 0) tokens[seq_start] = llama_vocab_bos(vocab);
        for (int k = 0; k < batch_size; ++k) {
            const int idx = seq*n_ctx + k;
            batch.token   [idx]    = tokens[seq_start + k];
            batch.pos     [idx]    = j*n_batch + k;
            batch.n_seq_id[idx]    = 1;
            batch.seq_id  [idx][0] = seq;
            batch.logits  [idx]    = batch.pos[idx] >= first ? 1 : 0;

            n_outputs += batch.logits[idx] != 0;
        }
        batch.n_tokens += batch_size;
        // restore the original token in case it was set to BOS
        tokens[seq_start] = token_org;

    --> llama_decode(ctx, batch);
    if (i == 0) llama_synchronize(ctx);

    for (int seq = 0; seq < n_seq_batch; seq++)
        const float * all_logits = llama_get_logits_ith(ctx, seq*n_ctx + first);
            ctx->synchronize();
            return ctx:: logits + output_ids[seq*n_ctx + first]*model.vocab.n_tokens();
        llama_token * tokens_data = tokens.data() + start + seq*n_ctx + first;
        --> process_logits(n_vocab, all_logits,
                tokens_data, n_ctx - 1 - first,
                workers, nll, nll2,
                logit_history.data() + start + seq*n_ctx + first,
                prob_history.data()  + start + seq*n_ctx + first);
        count += n_ctx - first - 1;

nll2 /= count;
nll /= count;
const double ppl = exp(nll);
nll2 -= nll * nll;
nll2 = sqrt(nll2/(count-1));

--> llama_batch_free;
return {tokens, ppl, logit_history, prob_history};
```

## [`llama_vocab::impl::tokenize`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-vocab.cpp#L2390)

```c++
std::vector<llama_token> tokens = common_tokenize(ctx, params.prompt, true);
----------
```

```c++
// tokenizes a string into a vector of tokens
// should work similar to Python's `tokenizer.encode`
std::vector<llama_token> common_tokenize(
  const struct llama_context * ctx,
           const std::string & text,
                        bool   add_special,
                        bool   parse_special = false) {
    const llama_model * model = llama_get_model(ctx);
    const llama_vocab * vocab = llama_model_get_vocab(model);
    --> return common_tokenize(vocab, text, add_special, parse_special);
}

std::vector<llama_token> common_tokenize(
    const struct llama_vocab * vocab,
           const std::string & text,
                        bool   add_special,
                        bool   parse_special) {
    // upper limit for the number of tokens
    int n_tokens = text.length() + 2 * add_special;
    std::vector<llama_token> result(n_tokens);
    --> n_tokens = llama_tokenize(vocab, text.data(), text.length(), result.data(), result.size(), add_special, parse_special);
    result.resize(n_tokens);
    return result;
}

int32_t llama_tokenize(
    const struct llama_vocab * vocab,
                  const char * text,
                     int32_t   text_len,
                 llama_token * tokens,
                     int32_t   n_tokens_max,
                        bool   add_special,
                        bool   parse_special) {
    --> return vocab->tokenize(text, text_len, tokens, n_tokens_max, add_special, parse_special);
}

int32_t llama_vocab::tokenize(
                  const char * text,
                     int32_t   text_len,
                 llama_token * tokens,
                     int32_t   n_tokens_max,
                        bool   add_special,
                        bool   parse_special) const {
    --> auto res = tokenize(std::string(text, text_len), add_special, parse_special);
    for (size_t i = 0; i < res.size(); i++) {
        tokens[i] = res[i];
    }
    return res.size();
}

std::vector<llama_token> llama_vocab::tokenize(
        const std::string & raw_text,
        bool add_special,
        bool parse_special) const {
    --> return pimpl->tokenize(raw_text, add_special, parse_special);
}

std::vector<llama_token> llama_vocab::impl::tokenize(
        const std::string & raw_text,
        bool add_special,
        bool parse_special) const {}
```

```c++
std::vector<llama_token> output;
std::forward_list<fragment_buffer_variant> fragment_buffer;

fragment_buffer.emplace_front(raw_text, 0, raw_text.length());
tokenizer_st_partition(fragment_buffer, parse_special);

llm_tokenizer_bpe_session session(vocab, *static_cast<const llm_tokenizer_bpe *>(tokenizer.get()));
// it calls some other methods that are not exist in llm_tokenizer,
// here just cast it to bpe tokenizer object
if (add_special) {
    session.append_bos(output);
}
for (const auto & fragment : fragment_buffer) {
    std::string text = fragment.raw_text.substr(fragment.offset, fragment.length);
    --> llm_tokenizer_bpe_session::tokenize;
}

if (add_special) {
    session.append_eos(output);
    session.check_double_bos_eos(output);
}

return output;
```

[`llm_tokenizer_bpe_session::tokenize`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-vocab.cpp#L482)

<details>
<summary>struct llm_tokenizer_bpe_session</summary>

```c++
struct llm_tokenizer_bpe_session {
    llm_tokenizer_bpe_session(const llama_vocab & vocab, const llm_tokenizer_bpe & tokenizer) : vocab(vocab), tokenizer(tokenizer) {}

    static void append(const llama_token token_id, std::vector<llama_token> & output)  {
    bool append_bos(std::vector<llama_token> & output) const
    bool append_eos(std::vector<llama_token> & output) const
    void check_double_bos_eos(const std::vector<llama_token> & output) const
    void tokenize(const std::string & text, std::vector<llama_token> & output)

private:
    void add_new_bigram(int left, int right)
    const llama_vocab & vocab;
    const llm_tokenizer_bpe & tokenizer;

    std::vector<llm_symbol> symbols;
    std::vector<llm_symbol> symbols_final;
    llm_bigram_bpe::queue work_queue;
}
```

</details>


```c++
session.tokenize(text, output);
----------
void llm_tokenizer_bpe_session::tokenize(const std::string & text, std::vector<llama_token> & output) {}
```

## [`llama_batch_init`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L590)

```c++
llama_batch batch = llama_batch_init(std::min(n_batch, n_ctx*n_seq), 0, 1);
----------
struct llama_batch llama_batch_init(int32_t n_tokens_alloc, int32_t embd, int32_t n_seq_max) {}
```

```c++
llama_batch batch = {
    /*n_tokens       =*/ 0,
    /*tokens         =*/ nullptr,
    /*embd           =*/ nullptr,
    /*pos            =*/ nullptr,
    /*n_seq_id       =*/ nullptr,
    /*seq_id         =*/ nullptr,
    /*logits         =*/ nullptr,
};

if (embd) {
    batch.embd = (float *) malloc(sizeof(float) * n_tokens_alloc * embd);
} else {
    batch.token = (llama_token *) malloc(sizeof(llama_token) * n_tokens_alloc);
}

batch.pos      = (llama_pos *)     malloc(sizeof(llama_pos)      * n_tokens_alloc);
batch.n_seq_id = (int32_t *)       malloc(sizeof(int32_t)        * n_tokens_alloc);
batch.seq_id   = (llama_seq_id **) malloc(sizeof(llama_seq_id *) * (n_tokens_alloc + 1));
for (int i = 0; i < n_tokens_alloc; ++i) {
    batch.seq_id[i] = (llama_seq_id *) malloc(sizeof(llama_seq_id) * n_seq_max);
}
batch.seq_id[n_tokens_alloc] = nullptr;

batch.logits   = (int8_t *)        malloc(sizeof(int8_t)         * n_tokens_alloc);

return batch;
```

## [`process_logits`](https://github.com/ggml-org/llama.cpp/blob/master/tools/perplexity/perplexity.cpp#L107)

```c++
process_logits(n_vocab, all_logits,
        tokens_data, n_ctx - 1 - first,
        workers, nll, nll2,
        logit_history.data() + start + seq*n_ctx + first,
        prob_history.data()  + start + seq*n_ctx + first);
----------
static void process_logits(
    int n_vocab, const float * logits, const int * tokens, int n_token, std::vector<std::thread> & workers,
    double & nll, double & nll2, float * logit_history, float * prob_history) {}
```

```c++
std::mutex mutex;
int counter = 0;
auto compute = [&mutex, &counter, &nll, &nll2, logit_history, prob_history, n_vocab, logits, tokens, n_token] () {
    double local_nll  = 0;
    double local_nll2 = 0;
    while (true) {
        std::unique_lock<std::mutex> lock(mutex);
        int i = counter++;
        if (i >= n_token) {
            nll += local_nll; nll2 += local_nll2;
            break;
        }
        lock.unlock();
        --> log_softmax --> const results_log_softmax results;
        const double v = -results.log_softmax;
        local_nll += v;
        local_nll2 += v*v;

        logit_history[i] = results.logit;
        prob_history[i]  = results.prob;
    }
};
for (auto & w : workers) {
    w = std::thread(compute);
}
compute();
for (auto & w : workers) {
    w.join();
}
```

[`log_softmax`](https://github.com/ggml-org/llama.cpp/blob/master/tools/perplexity/perplexity.cpp#L58)

<details>
<summary>struct results_log_softmax</summary>

```c++
struct results_log_softmax {
    double log_softmax;
    float  logit;
    float  prob;
};
```

</details>


```c++
const results_log_softmax results = log_softmax(n_vocab, logits + size_t(i)*n_vocab, tokens[i+1]);
----------
static results_log_softmax log_softmax(int n_vocab, const float * logits, int tok) {
    float max_logit = logits[0];
    for (int i = 1; i < n_vocab; ++i) {
        max_logit = std::max(max_logit, logits[i]);
    }
    double sum_exp = 0.0;
    for (int i = 0; i < n_vocab; ++i) {
        sum_exp += expf(logits[i] - max_logit);
    }
    return {logits[tok] - max_logit - log(sum_exp), logits[tok], expf(logits[tok] - max_logit) / (float) sum_exp};
}

```

## [`llama_batch_free`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L620)

```c++
llama_batch_free(batch);
----------
void llama_batch_free(struct llama_batch batch) {}
```

```c++
if (batch.token)    free(batch.token);
if (batch.embd)     free(batch.embd);
if (batch.pos)      free(batch.pos);
if (batch.n_seq_id) free(batch.n_seq_id);
if (batch.seq_id) {
    for (int i = 0; batch.seq_id[i] != nullptr; ++i) {
        free(batch.seq_id[i]);
    }
    free(batch.seq_id);
}
if (batch.logits)   free(batch.logits);
```

# [`ggml_quantize_free`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L6451)

```c++
llama_backend_free();
----------
void llama_backend_free(void) {
    --> ggml_quantize_free();
}

void ggml_quantize_free(void) {
    ggml_critical_section_start();

    iq2xs_free_impl(GGML_TYPE_IQ2_XXS);
    iq2xs_free_impl(GGML_TYPE_IQ2_XS);
    iq2xs_free_impl(GGML_TYPE_IQ1_S);
    iq3xs_free_impl(256);

    ggml_critical_section_end();
}

void iq2xs_free_impl(enum ggml_type type)
    const int gindex = iq2_data_index(type);
    if (iq2_data[gindex].grid) {
        free(iq2_data[gindex].grid);       iq2_data[gindex].grid = NULL;
        free(iq2_data[gindex].map);        iq2_data[gindex].map  = NULL;
        free(iq2_data[gindex].neighbours); iq2_data[gindex].neighbours = NULL;
    }

void iq3xs_free_impl(int grid_size)
    const int gindex = iq3_data_index(grid_size);
    if (iq3_data[gindex].grid) {
        free(iq3_data[gindex].grid);       iq3_data[gindex].grid = NULL;
        free(iq3_data[gindex].map);        iq3_data[gindex].map  = NULL;
        free(iq3_data[gindex].neighbours); iq3_data[gindex].neighbours = NULL;
    }
```